<a href="https://colab.research.google.com/github/RussAbbott/Wordle/blob/main/Wordle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

guess_words_long = [
    'aahed', 'aalii', 'aapas', 'aargh', 'aarti', 'abaca', 'abaci', 'aback', 'abacs', 'abaft', 'abaht', 'abaka', 'abamp', 'aband', 'abase', 'abash', 'abask', 'abate', 'abaya', 'abbas', 'abbed', 'abbes', 'abbey', 'abbot', 'abcee', 'abeam', 'abear', 'abeat', 'abeer', 'abele', 'abeng', 'abers', 'abets', 'abeys', 'abhor', 'abide', 'abies', 'abius', 'abjad', 'abjud', 'abled', 'abler', 'ables', 'ablet', 'ablow', 'abmho', 'abnet', 'abode', 'abohm', 'aboil', 'aboma', 'aboon', 'abord', 'abore', 'aborn', 'abort', 'about', 'above', 'abram', 'abray', 'abrim', 'abrin', 'abris', 'absey', 'absit', 'abuna', 'abune', 'abura', 'aburn', 'abuse', 'abuts', 'abuzz', 'abyes', 'abysm', 'abyss', 'acais', 'acara', 'acari', 'accas', 'accha', 'accoy', 'accra', 'acedy', 'acene', 'acerb', 'acers', 'aceta', 'achar', 'ached', 'acher', 'aches', 'achey', 'achoo', 'acids', 'acidy', 'acies', 'acing', 'acini', 'ackee', 'acker', 'acmes', 'acmic', 'acned', 'acnes', 'acock', 'acoel', 'acold', 'acone', 'acorn', 'acral', 'acred', 'acres', 'acrid', 'acron', 'acros', 'acryl', 'actas', 'acted', 'actin', 'acton', 'actor', 'actus', 'acute', 'acyls', 'adage', 'adapt', 'adats', 'adawn', 'adaws', 'adays', 'adbot', 'addas', 'addax', 'added', 'adder', 'addin', 'addio', 'addle', 'addra', 'adead', 'adeem', 'adept', 'adhan', 'adhoc', 'adieu', 'adios', 'adits', 'adlib', 'adman', 'admen', 'admin', 'admit', 'admix', 'adnex', 'adobe', 'adobo', 'adoon', 'adopt', 'adorb', 'adore', 'adorn', 'adown', 'adoze', 'adrad', 'adraw', 'adred', 'adret', 'adrip', 'adsum', 'aduki', 'adult', 'adunc', 'adust', 'advew', 'advts', 'adyta', 'adyts', 'adzed', 'adzes', 'aecia', 'aedes', 'aeger', 'aegis', 'aeons', 'aerie', 'aeros', 'aesir', 'aevum', 'afald', 'afanc', 'afara', 'afars', 'afear', 'affix', 'affly', 'afion', 'afire', 'afizz', 'aflaj', 'aflap', 'aflow', 'afoam', 'afoot', 'afore', 'afoul', 'afret', 'afrit', 'afros', 'after', 'aftos', 'again', 'agals', 'agama', 'agami', 'agamy', 'agape', 'agars', 'agasp', 'agast', 'agate', 'agaty', 'agave', 'agaze', 'agbas', 'agene', 'agent', 'agers', 'aggag', 'agger', 'aggie', 'aggri', 'aggro', 'aggry', 'aghas', 'agidi', 'agila', 'agile', 'aging', 'agios', 'agism', 'agist', 'agita', 'aglee', 'aglet', 'agley', 'agloo', 'aglow', 'aglus', 'agmas', 'agoge', 'agogo', 'agone', 'agons', 'agony', 'agood', 'agora', 'agree', 'agria', 'agrin', 'agros', 'agrum', 'agued', 'agues', 'aguey', 'aguna', 'agush', 'aguti', 'ahead', 'aheap', 'ahent', 'ahigh', 'ahind', 'ahing', 'ahint', 'ahold', 'ahole', 'ahull', 'ahuru', 'aidas', 'aided', 'aider', 'aides', 'aidoi', 'aidos', 'aiery', 'aigas', 'aight', 'ailed', 'aimag', 'aimak', 'aimed', 'aimer', 'ainee', 'ainga', 'aioli', 'aired', 'airer', 'airns', 'airth', 'airts', 'aisle', 'aitch', 'aitus', 'aiver', 'aixes', 'aiyah', 'aiyee', 'aiyoh', 'aiyoo', 'aizle', 'ajies', 'ajiva', 'ajuga', 'ajupa', 'ajwan', 'akara', 'akees', 'akela', 'akene', 'aking', 'akita', 'akkas', 'akker', 'akoia', 'akoja', 'akoya', 'aksed', 'akses', 'alaap', 'alack', 'alala', 'alamo', 'aland', 'alane', 'alang', 'alans', 'alant', 'alapa', 'alaps', 'alarm', 'alary', 'alata', 'alate', 'alays', 'albas', 'albee', 'albid', 'album', 'alcea', 'alces', 'alcid', 'alcos', 'aldea', 'alder', 'aldol', 'aleak', 'aleck', 'alecs', 'aleem', 'alefs', 'aleft', 'aleph', 'alert', 'alews', 'aleye', 'alfas', 'algae', 'algal', 'algas', 'algid', 'algin', 'algor', 'algos', 'algum', 'alias', 'alibi', 'alick', 'alien', 'alifs', 'align', 'alike', 'alims', 'aline', 'alios', 'alist', 'alive', 'aliya', 'alkie', 'alkin', 'alkos', 'alkyd', 'alkyl', 'allan', 'allay', 'allee', 'allel', 'allen', 'aller', 'alley', 'allin', 'allis', 'allod', 'allot', 'allow', 'alloy', 'allus', 'allyl', 'almah', 'almas', 'almeh', 'almes', 'almud', 'almug', 'alods', 'aloed', 'aloes', 'aloft', 'aloha', 'aloin', 'alone', 'along', 'aloof', 'aloos', 'alose', 'aloud', 'alowe', 'alpha', 'altar', 'alter', 'altho', 'altos', 'alula', 'alums', 'alumy', 'alure', 'alurk', 'alvar', 'alway', 'amahs', 'amain', 'amari', 'amaro', 'amass', 'amate', 'amaut', 'amaze', 'amban', 'amber', 'ambit', 'amble', 'ambos', 'ambry', 'ameba', 'ameer', 'amend', 'amene', 'amens', 'ament', 'amias', 'amice', 'amici', 'amide', 'amido', 'amids', 'amies', 'amiga', 'amigo', 'amine', 'amino', 'amins', 'amirs', 'amiss', 'amity', 'amlas', 'amman', 'ammas', 'ammon', 'ammos', 'amnia', 'amnic', 'amnio', 'amoks', 'amole', 'among', 'amore', 'amort', 'amour', 'amove', 'amowt', 'amped', 'ample', 'amply', 'ampul', 'amrit', 'amuck', 'amuse', 'amyls', 'anana', 'anata', 'ancho', 'ancle', 'ancon', 'andic', 'andro', 'anear', 'anele', 'anent', 'angas', 'angel', 'anger', 'angle', 'anglo', 'angry', 'angst', 'anigh', 'anile', 'anils', 'anima', 'anime', 'animi', 'anion', 'anise', 'anker', 'ankhs', 'ankle', 'ankus', 'anlas', 'annal', 'annan', 'annas', 'annat', 'annex', 'annoy', 'annul', 'annum', 'annus', 'anoas', 'anode', 'anole', 'anomy', 'ansae', 'ansas', 'antae', 'antar', 'antas', 'anted', 'antes', 'antic', 'antis', 'antra', 'antre', 'antsy', 'anura', 'anvil', 'anyon', 'aorta', 'apace', 'apage', 'apaid', 'apart', 'apayd', 'apays', 'apeak', 'apeek', 'apers', 'apert', 'apery', 'apgar', 'aphid', 'aphis', 'apian', 'aping', 'apiol', 'apish', 'apism', 'apnea', 'apode', 'apods', 'apols', 'apoop', 'aport', 'appal', 'appam', 'appay', 'appel', 'apple', 'apply', 'appro', 'appts', 'appui', 'appuy', 'apres', 'apron', 'apses', 'apsis', 'apsos', 'apted', 'apter', 'aptly', 'aquae', 'aquas', 'araba', 'araks', 'arame', 'arars', 'arbah', 'arbas', 'arbor', 'arced', 'archi', 'arcos', 'arcus', 'ardeb', 'ardor', 'ardri', 'aread', 'areae', 'areal', 'arear', 'areas', 'areca', 'aredd', 'arede', 'arefy', 'areic', 'arena', 'arene', 'arepa', 'arere', 'arete', 'arets', 'arett', 'argal', 'argan', 'argil', 'argle', 'argol', 'argon', 'argot', 'argue', 'argus', 'arhat', 'arias', 'ariel', 'ariki', 'arils', 'ariot', 'arise', 'arish', 'arith', 'arked', 'arled', 'arles', 'armed', 'armer', 'armet', 'armil', 'armor', 'arnas', 'arnis', 'arnut', 'aroba', 'aroha', 'aroid', 'aroma', 'arose', 'arpas', 'arpen', 'arrah', 'arras', 'array', 'arret', 'arris', 'arrow', 'arroz', 'arsed', 'arses', 'arsey', 'arsis', 'arson', 'artal', 'artel', 'arter', 'artic', 'artis', 'artly', 'artsy', 'aruhe', 'arums', 'arval', 'arvee', 'arvos', 'aryls', 'asada', 'asana', 'ascon', 'ascot', 'ascus', 'asdic', 'ashed', 'ashen', 'ashes', 'ashet', 'aside', 'asity', 'askar', 'asked', 'asker', 'askew', 'askoi', 'askos', 'aspen', 'asper', 'aspic', 'aspie', 'aspis', 'aspro', 'assai', 'assam', 'assay', 'assed', 'asses', 'asset', 'assez', 'assot', 'aster', 'astir', 'astun', 'asura', 'asway', 'aswim', 'asyla', 'ataps', 'ataxy', 'atigi', 'atilt', 'atimy', 'atlas', 'atman', 'atmas', 'atmos', 'atocs', 'atoke', 'atoks', 'atoll', 'atoms', 'atomy', 'atone', 'atony', 'atopy', 'atria', 'atrip', 'attap', 'attar', 'attas', 'atter', 'attic', 'atuas', 'aucht', 'audad', 'audax', 'audio', 'audit', 'augen', 'auger', 'auges', 'aught', 'augur', 'aulas', 'aulic', 'auloi', 'aulos', 'aumil', 'aunes', 'aunts', 'aunty', 'aurae', 'aural', 'aurar', 'auras', 'aurei', 'aures', 'auric', 'auris', 'aurum', 'autos', 'auxin', 'avail', 'avale', 'avant', 'avast', 'avels', 'avens', 'avers', 'avert', 'avgas', 'avian', 'avine', 'avion', 'avise', 'aviso', 'avize', 'avoid', 'avows', 'avyze', 'await', 'awake', 'award', 'aware', 'awari', 'awarn', 'awash', 'awato', 'awave', 'aways', 'awdls', 'aweel', 'aweto', 'awful', 'awing', 'awkin', 'awmry', 'awned', 'awner', 'awoke', 'awols', 'awork', 'axels', 'axial', 'axile', 'axils', 'axing', 'axiom', 'axion', 'axite', 'axled', 'axles', 'axman', 'axmen', 'axoid', 'axone', 'axons', 'ayahs', 'ayaya', 'ayelp', 'aygre', 'ayins', 'aymag', 'ayont', 'ayres', 'ayrie', 'azans', 'azide', 'azido', 'azine', 'azlon', 'azoic', 'azole', 'azons', 'azote', 'azoth', 'azuki', 'azure', 'azurn', 'azury', 'azygy', 'azyme', 'azyms', 'baaed', 'baals', 'baaps', 'babas', 'babby', 'babel', 'babes', 'babka', 'baboo', 'babul', 'babus', 'bacca', 'bacco', 'baccy', 'bacha', 'bachs', 'backs', 'backy', 'bacne', 'bacon', 'badam', 'baddy', 'badge', 'badly', 'baels', 'baffs', 'baffy', 'bafta', 'bafts', 'bagel', 'baggy', 'baghs', 'bagie', 'bagsy', 'bagua', 'bahts', 'bahus', 'bahut', 'baiks', 'baile', 'bails', 'bairn', 'baisa', 'baith', 'baits', 'baiza', 'baize', 'bajan', 'bajra', 'bajri', 'bajus', 'baked', 'baken', 'baker', 'bakes', 'bakra', 'balas', 'balds', 'baldy', 'baled', 'baler', 'bales', 'balks', 'balky', 'ballo', 'balls', 'bally', 'balms', 'balmy', 'baloi', 'balon', 'baloo', 'balot', 'balsa', 'balti', 'balun', 'balus', 'balut', 'bamas', 'bambi', 'bamma', 'bammy', 'banak', 'banal', 'banco', 'bancs', 'banda', 'bandh', 'bands', 'bandy', 'baned', 'banes', 'bangs', 'bania', 'banjo', 'banks', 'banky', 'banns', 'bants', 'bantu', 'banty', 'bantz', 'banya', 'baons', 'baozi', 'bappu', 'bapus', 'barbe', 'barbs', 'barby', 'barca', 'barde', 'bardo', 'bards', 'bardy', 'bared', 'barer', 'bares', 'barfi', 'barfs', 'barfy', 'barge', 'baric', 'barks', 'barky', 'barms', 'barmy', 'barns', 'barny', 'baron', 'barps', 'barra', 'barre', 'barro', 'barry', 'barye', 'basal', 'basan', 'basas', 'based', 'basen', 'baser', 'bases', 'basha', 'basho', 'basic', 'basij', 'basil', 'basin', 'basis', 'basks', 'bason', 'basse', 'bassi', 'basso', 'bassy', 'basta', 'baste', 'basti', 'basto', 'basts', 'batch', 'bated', 'bates', 'bathe', 'baths', 'batik', 'baton', 'batos', 'batta', 'batts', 'battu', 'batty', 'bauds', 'bauks', 'baulk', 'baurs', 'bavin', 'bawds', 'bawdy', 'bawks', 'bawls', 'bawns', 'bawrs', 'bawty', 'bayas', 'bayed', 'bayer', 'bayes', 'bayle', 'bayou', 'bayts', 'bazar', 'bazas', 'bazoo', 'bball', 'bdays', 'beach', 'beads', 'beady', 'beaks', 'beaky', 'beals', 'beams', 'beamy', 'beano', 'beans', 'beany', 'beard', 'beare', 'bears', 'beast', 'beath', 'beats', 'beaty', 'beaus', 'beaut', 'beaux', 'bebop', 'becap', 'becke', 'becks', 'bedad', 'bedel', 'bedes', 'bedew', 'bedim', 'bedye', 'beech', 'beedi', 'beefs', 'beefy', 'beeps', 'beers', 'beery', 'beets', 'befit', 'befog', 'begad', 'began', 'begar', 'begat', 'begem', 'beget', 'begin', 'begob', 'begot', 'begum', 'begun', 'beige', 'beigy', 'being', 'beins', 'beira', 'beisa', 'bekah', 'belah', 'belar', 'belay', 'belch', 'belee', 'belga', 'belie', 'belit', 'belle', 'belli', 'bello', 'bells', 'belly', 'belon', 'below', 'belts', 'belve', 'bemad', 'bemas', 'bemix', 'bemud', 'bench', 'bends', 'bendy', 'benes', 'benet', 'benga', 'benis', 'benji', 'benne', 'benni', 'benny', 'bento', 'bents', 'benty', 'bepat', 'beray', 'beres', 'beret', 'bergs', 'berko', 'berks', 'berme', 'berms', 'berob', 'berry', 'berth', 'beryl', 'besat', 'besaw', 'besee', 'beses', 'beset', 'besit', 'besom', 'besot', 'besti', 'bests', 'betas', 'beted', 'betel', 'betes', 'beths', 'betid', 'beton', 'betta', 'betty', 'bevan', 'bevel', 'bever', 'bevor', 'bevue', 'bevvy', 'bewdy', 'bewet', 'bewig', 'bezel', 'bezes', 'bezil', 'bezzy', 'bhais', 'bhaji', 'bhang', 'bhats', 'bhava', 'bhels', 'bhoot', 'bhuna', 'bhuts', 'biach', 'biali', 'bialy', 'bibbs', 'bibes', 'bibis', 'bible', 'biccy', 'bicep', 'bices', 'bicky', 'biddy', 'bided', 'bider', 'bides', 'bidet', 'bidis', 'bidon', 'bidri', 'bield', 'biers', 'biffo', 'biffs', 'biffy', 'bifid', 'bigae', 'biggs', 'biggy', 'bigha', 'bight', 'bigly', 'bigos', 'bigot', 'bihon', 'bijou', 'biked', 'biker', 'bikes', 'bikie', 'bikky', 'bilal', 'bilat', 'bilbo', 'bilby', 'biled', 'biles', 'bilge', 'bilgy', 'bilks', 'bills', 'billy', 'bimah', 'bimas', 'bimbo', 'binal', 'bindi', 'binds', 'biner', 'bines', 'binge', 'bingo', 'bings', 'bingy', 'binit', 'binks', 'binky', 'bints', 'biogs', 'biome', 'bions', 'biont', 'biose', 'biota', 'biped', 'bipod', 'bippy', 'birch', 'birdo', 'birds', 'biris', 'birks', 'birle', 'birls', 'biros', 'birrs', 'birse', 'birsy', 'birth', 'birze', 'birzz', 'bises', 'bisks', 'bisom', 'bison', 'bitch', 'biter', 'bites', 'bitey', 'bitos', 'bitou', 'bitsy', 'bitte', 'bitts', 'bitty', 'bivia', 'bivvy', 'bizes', 'bizzo', 'bizzy', 'blabs', 'black', 'blade', 'blads', 'blady', 'blaer', 'blaes', 'blaff', 'blags', 'blahs', 'blain', 'blame', 'blams', 'blanc', 'bland', 'blank', 'blare', 'blart', 'blase', 'blash', 'blast', 'blate', 'blats', 'blatt', 'blaud', 'blawn', 'blaws', 'blays', 'blaze', 'bleah', 'bleak', 'blear', 'bleat', 'blebs', 'blech', 'bleed', 'bleep', 'blees', 'blend', 'blent', 'blert', 'bless', 'blest', 'blets', 'bleys', 'blimp', 'blimy', 'blind', 'bling', 'blini', 'blink', 'blins', 'bliny', 'blips', 'bliss', 'blist', 'blite', 'blits', 'blitz', 'blive', 'bloat', 'blobs', 'block', 'blocs', 'blogs', 'bloke', 'blond', 'blonx', 'blood', 'blook', 'bloom', 'bloop', 'blore', 'blots', 'blown', 'blows', 'blowy', 'blubs', 'blude', 'bluds', 'bludy', 'blued', 'bluer', 'blues', 'bluet', 'bluey', 'bluff', 'bluid', 'blume', 'blunk', 'blunt', 'blurb', 'blurs', 'blurt', 'blush', 'blype', 'boabs', 'boaks', 'board', 'boars', 'boart', 'boast', 'boats', 'boaty', 'bobac', 'bobak', 'bobas', 'bobby', 'bobol', 'bobos', 'bocca', 'bocce', 'bocci', 'boche', 'bocks', 'boded', 'bodes', 'bodge', 'bodgy', 'bodhi', 'bodle', 'bodoh', 'boeps', 'boers', 'boeti', 'boets', 'boeuf', 'boffo', 'boffs', 'bogan', 'bogey', 'boggy', 'bogie', 'bogle', 'bogue', 'bogus', 'bohea', 'bohos', 'boils', 'boing', 'boink', 'boite', 'boked', 'bokeh', 'bokes', 'bokos', 'bolar', 'bolas', 'boldo', 'bolds', 'boles', 'bolet', 'bolix', 'bolks', 'bolls', 'bolos', 'bolts', 'bolus', 'bomas', 'bombe', 'bombo', 'bombs', 'bomoh', 'bomor', 'bonce', 'bonds', 'boned', 'boner', 'bones', 'boney', 'bongo', 'bongs', 'bonie', 'bonks', 'bonne', 'bonny', 'bonum', 'bonus', 'bonza', 'bonze', 'booai', 'booay', 'boobs', 'booby', 'boody', 'booed', 'boofy', 'boogy', 'boohs', 'books', 'booky', 'bools', 'booms', 'boomy', 'boong', 'boons', 'boord', 'boors', 'boose', 'boost', 'booth', 'boots', 'booty', 'booze', 'boozy', 'boppy', 'borak', 'boral', 'boras', 'borax', 'borde', 'bords', 'bored', 'boree', 'borek', 'borel', 'borer', 'bores', 'borgo', 'boric', 'borks', 'borms', 'borna', 'borne', 'boron', 'borts', 'borty', 'bortz', 'bosey', 'bosie', 'bosks', 'bosky', 'bosom', 'boson', 'bossa', 'bossy', 'bosun', 'botas', 'botch', 'boteh', 'botel', 'botes', 'botew', 'bothy', 'botos', 'botte', 'botts', 'botty', 'bouge', 'bough', 'bouks', 'boule', 'boult', 'bound', 'bouns', 'bourd', 'bourg', 'bourn', 'bouse', 'bousy', 'bouts', 'boutu', 'bovid', 'bowat', 'bowed', 'bowel', 'bower', 'bowes', 'bowet', 'bowie', 'bowls', 'bowne', 'bowrs', 'bowse', 'boxed', 'boxen', 'boxer', 'boxes', 'boxla', 'boxty', 'boyar', 'boyau', 'boyed', 'boyey', 'boyfs', 'boygs', 'boyla', 'boyly', 'boyos', 'boysy', 'bozos', 'braai', 'brace', 'brach', 'brack', 'bract', 'brads', 'braes', 'brags', 'brahs', 'braid', 'brail', 'brain', 'brake', 'braks', 'braky', 'brame', 'brand', 'brane', 'brank', 'brans', 'brant', 'brash', 'brass', 'brast', 'brats', 'brava', 'brave', 'bravi', 'bravo', 'brawl', 'brawn', 'braws', 'braxy', 'brays', 'braza', 'braze', 'bread', 'break', 'bream', 'brede', 'breds', 'breed', 'breem', 'breer', 'brees', 'breid', 'breis', 'breme', 'brens', 'brent', 'brere', 'brers', 'breve', 'brews', 'breys', 'briar', 'bribe', 'brick', 'bride', 'brief', 'brier', 'bries', 'brigs', 'briki', 'briks', 'brill', 'brims', 'brine', 'bring', 'brink', 'brins', 'briny', 'brios', 'brise', 'brisk', 'briss', 'brith', 'brits', 'britt', 'brize', 'broad', 'broch', 'brock', 'brods', 'brogh', 'brogs', 'broil', 'broke', 'brome', 'bromo', 'bronc', 'brond', 'brood', 'brook', 'brool', 'broom', 'broos', 'brose', 'brosy', 'broth', 'brown', 'brows', 'bruck', 'brugh', 'bruhs', 'bruin', 'bruit', 'bruja', 'brujo', 'brule', 'brume', 'brung', 'brunt', 'brush', 'brusk', 'brust', 'brute', 'bruts', 'bruvs', 'buats', 'buaze', 'bubal', 'bubas', 'bubba', 'bubbe', 'bubby', 'bubus', 'buchu', 'bucko', 'bucks', 'bucku', 'budas', 'buddy', 'buded', 'budes', 'budge', 'budis', 'budos', 'buena', 'buffa', 'buffe', 'buffi', 'buffo', 'buffs', 'buffy', 'bufos', 'bufty', 'bugan', 'buggy', 'bugle', 'buhls', 'buhrs', 'buiks', 'build', 'built', 'buist', 'bukes', 'bukos', 'bulbs', 'bulge', 'bulgy', 'bulks', 'bulky', 'bulla', 'bulls', 'bully', 'bulse', 'bumbo', 'bumfs', 'bumph', 'bumps', 'bumpy', 'bunas', 'bunce', 'bunch', 'bunco', 'bunde', 'bundh', 'bunds', 'bundt', 'bundu', 'bundy', 'bungs', 'bungy', 'bunia', 'bunje', 'bunjy', 'bunko', 'bunks', 'bunns', 'bunny', 'bunts', 'bunty', 'bunya', 'buoys', 'buppy', 'buran', 'buras', 'burbs', 'burds', 'buret', 'burfi', 'burgh', 'burgs', 'burin', 'burka', 'burke', 'burks', 'burls', 'burly', 'burns', 'burnt', 'buroo', 'burps', 'burqa', 'burra', 'burro', 'burrs', 'burry', 'bursa', 'burse', 'burst', 'busby', 'bused', 'buses', 'bushy', 'busks', 'busky', 'bussu', 'busti', 'busts', 'busty', 'butch', 'buteo', 'butes', 'butle', 'butoh', 'butte', 'butts', 'butty', 'butut', 'butyl', 'buxom', 'buyer', 'buyin', 'buzzy', 'bwana', 'bwazi', 'byded', 'bydes', 'byked', 'bykes', 'bylaw', 'byres', 'byrls', 'byssi', 'bytes', 'byway', 'caaed', 'cabal', 'cabas', 'cabby', 'caber', 'cabin', 'cable', 'cabob', 'caboc', 'cabre', 'cacao', 'cacas', 'cache', 'cacks', 'cacky', 'cacti', 'caddy', 'cadee', 'cades', 'cadet', 'cadge', 'cadgy', 'cadie', 'cadis', 'cadre', 'caeca', 'caese', 'cafes', 'caffe', 'caffs', 'caged', 'cager', 'cages', 'cagey', 'cagot', 'cahow', 'caids', 'cains', 'caird', 'cairn', 'cajon', 'cajun', 'caked', 'cakes', 'cakey', 'calfs', 'calid', 'calif', 'calix', 'calks', 'calla', 'calle', 'calls', 'calms', 'calmy', 'calos', 'calpa', 'calps', 'calve', 'calyx', 'caman', 'camas', 'camel', 'cameo', 'cames', 'camis', 'camos', 'campi', 'campo', 'camps', 'campy', 'camus', 'canal', 'cando', 'candy', 'caned', 'caneh', 'caner', 'canes', 'cangs', 'canid', 'canna', 'canns', 'canny', 'canoe', 'canon', 'canso', 'canst', 'canti', 'canto', 'cants', 'canty', 'capas', 'capax', 'caped', 'caper', 'capes', 'capex', 'caphs', 'capiz', 'caple', 'capon', 'capos', 'capot', 'capri', 'capul', 'caput', 'carap', 'carat', 'carbo', 'carbs', 'carby', 'cardi', 'cards', 'cardy', 'cared', 'carer', 'cares', 'caret', 'carex', 'cargo', 'carks', 'carle', 'carls', 'carne', 'carns', 'carny', 'carob', 'carol', 'carom', 'caron', 'carpe', 'carpi', 'carps', 'carrs', 'carry', 'carse', 'carta', 'carte', 'carts', 'carve', 'carvy', 'casas', 'casco', 'cased', 'caser', 'cases', 'casks', 'casky', 'caste', 'casts', 'casus', 'catch', 'cater', 'cates', 'catty', 'cauda', 'cauks', 'cauld', 'caulk', 'cauls', 'caums', 'caups', 'cauri', 'causa', 'cause', 'cavas', 'caved', 'cavel', 'caver', 'caves', 'cavie', 'cavil', 'cavus', 'cawed', 'cawks', 'caxon', 'cease', 'ceaze', 'cebid', 'cecal', 'cecum', 'cedar', 'ceded', 'ceder', 'cedes', 'cedis', 'ceiba', 'ceili', 'ceils', 'celeb', 'cella', 'celli', 'cello', 'cells', 'celly', 'celom', 'celts', 'cense', 'cento', 'cents', 'centu', 'ceorl', 'cepes', 'cerci', 'cered', 'ceres', 'cerge', 'ceria', 'ceric', 'cerne', 'ceroc', 'ceros', 'certs', 'certy', 'cesse', 'cesta', 'cesti', 'cetes', 'cetyl', 'cezve', 'chaap', 'chaat', 'chace', 'chack', 'chaco', 'chado', 'chads', 'chafe', 'chaff', 'chaft', 'chain', 'chair', 'chais', 'chalk', 'chals', 'champ', 'chams', 'chana', 'chang', 'chank', 'chant', 'chaos', 'chape', 'chaps', 'chapt', 'chara', 'chard', 'chare', 'chark', 'charm', 'charr', 'chars', 'chart', 'chary', 'chase', 'chasm', 'chats', 'chava', 'chave', 'chavs', 'chawk', 'chawl', 'chaws', 'chaya', 'chays', 'cheap', 'cheat', 'cheba', 'check', 'chedi', 'cheeb', 'cheek', 'cheep', 'cheer', 'cheet', 'chefs', 'cheka', 'chela', 'chelp', 'chemo', 'chems', 'chere', 'chert', 'chess', 'chest', 'cheth', 'chevy', 'chews', 'chewy', 'chiao', 'chias', 'chiba', 'chibs', 'chica', 'chich', 'chick', 'chico', 'chics', 'chide', 'chief', 'chiel', 'chiko', 'chiks', 'child', 'chile', 'chili', 'chill', 'chimb', 'chime', 'chimo', 'chimp', 'china', 'chine', 'ching', 'chink', 'chino', 'chins', 'chips', 'chirk', 'chirl', 'chirm', 'chiro', 'chirp', 'chirr', 'chirt', 'chiru', 'chiti', 'chits', 'chiva', 'chive', 'chivs', 'chivy', 'chizz', 'chock', 'choco', 'chocs', 'chode', 'chogs', 'choil', 'choir', 'choke', 'choko', 'choky', 'chola', 'choli', 'cholo', 'chomp', 'chons', 'choof', 'chook', 'choom', 'choon', 'chops', 'chord', 'chore', 'chose', 'choss', 'chota', 'chott', 'chout', 'choux', 'chowk', 'chows', 'chubs', 'chuck', 'chufa', 'chuff', 'chugs', 'chump', 'chums', 'chunk', 'churl', 'churn', 'churr', 'chuse', 'chute', 'chuts', 'chyle', 'chyme', 'chynd', 'cibol', 'cided', 'cider', 'cides', 'ciels', 'cigar', 'ciggy', 'cilia', 'cills', 'cimar', 'cimex', 'cinch', 'cinct', 'cines', 'cinqs', 'cions', 'cippi', 'circa', 'circs', 'cires', 'cirls', 'cirri', 'cisco', 'cissy', 'cists', 'cital', 'cited', 'citee', 'citer', 'cites', 'cives', 'civet', 'civic', 'civie', 'civil', 'civvy', 'clach', 'clack', 'clade', 'clads', 'claes', 'clags', 'claim', 'clair', 'clame', 'clamp', 'clams', 'clang', 'clank', 'clans', 'claps', 'clapt', 'claro', 'clart', 'clary', 'clash', 'clasp', 'class', 'clast', 'clats', 'claut', 'clave', 'clavi', 'claws', 'clays', 'clean', 'clear', 'cleat', 'cleck', 'cleek', 'cleep', 'clefs', 'cleft', 'clegs', 'cleik', 'clems', 'clepe', 'clept', 'clerk', 'cleve', 'clews', 'click', 'clied', 'clies', 'cliff', 'clift', 'climb', 'clime', 'cline', 'cling', 'clink', 'clint', 'clipe', 'clips', 'clipt', 'clits', 'cloak', 'cloam', 'clock', 'clods', 'cloff', 'clogs', 'cloke', 'clomb', 'clomp', 'clone', 'clonk', 'clons', 'cloop', 'cloot', 'clops', 'close', 'clote', 'cloth', 'clots', 'cloud', 'clour', 'clous', 'clout', 'clove', 'clown', 'clows', 'cloye', 'cloys', 'cloze', 'clubs', 'cluck', 'clued', 'clues', 'cluey', 'clump', 'clung', 'clunk', 'clype', 'cnida', 'coach', 'coact', 'coady', 'coala', 'coals', 'coaly', 'coapt', 'coarb', 'coast', 'coate', 'coati', 'coats', 'cobbs', 'cobby', 'cobia', 'coble', 'cobot', 'cobra', 'cobza', 'cocas', 'cocci', 'cocco', 'cocks', 'cocky', 'cocoa', 'cocos', 'cocus', 'codas', 'codec', 'coded', 'coden', 'coder', 'codes', 'codex', 'codon', 'coeds', 'coffs', 'cogie', 'cogon', 'cogue', 'cohab', 'cohen', 'cohoe', 'cohog', 'cohos', 'coifs', 'coign', 'coils', 'coins', 'coirs', 'coits', 'coked', 'cokes', 'cokey', 'colas', 'colby', 'colds', 'coled', 'coles', 'coley', 'colic', 'colin', 'colle', 'colls', 'colly', 'colog', 'colon', 'color', 'colts', 'colza', 'comae', 'comal', 'comas', 'combe', 'combi', 'combo', 'combs', 'comby', 'comer', 'comes', 'comet', 'comfy', 'comic', 'comix', 'comma', 'comme', 'commo', 'comms', 'commy', 'compo', 'comps', 'compt', 'comte', 'comus', 'conch', 'condo', 'coned', 'cones', 'conex', 'coney', 'confs', 'conga', 'conge', 'congo', 'conia', 'conic', 'conin', 'conks', 'conky', 'conne', 'conns', 'conte', 'conto', 'conus', 'convo', 'cooch', 'cooed', 'cooee', 'cooer', 'cooey', 'coofs', 'cooks', 'cooky', 'cools', 'cooly', 'coomb', 'cooms', 'coomy', 'coons', 'coops', 'coopt', 'coost', 'coots', 'cooty', 'cooze', 'copal', 'copay', 'coped', 'copen', 'coper', 'copes', 'copha', 'coppy', 'copra', 'copse', 'copsy', 'coqui', 'coral', 'coram', 'corbe', 'corby', 'corda', 'cords', 'cored', 'corer', 'cores', 'corey', 'corgi', 'coria', 'corks', 'corky', 'corms', 'corni', 'corno', 'corns', 'cornu', 'corny', 'corps', 'corse', 'corso', 'cosec', 'cosed', 'coses', 'coset', 'cosey', 'cosie', 'costa', 'coste', 'costs', 'cotan', 'cotch', 'coted', 'cotes', 'coths', 'cotta', 'cotts', 'couch', 'coude', 'cough', 'could', 'count', 'coupe', 'coups', 'courb', 'courd', 'coure', 'cours', 'court', 'couta', 'couth', 'coved', 'coven', 'cover', 'coves', 'covet', 'covey', 'covin', 'cowal', 'cowan', 'cowed', 'cower', 'cowks', 'cowls', 'cowps', 'cowry', 'coxae', 'coxal', 'coxed', 'coxes', 'coxib', 'coyau', 'coyed', 'coyer', 'coyly', 'coypu', 'cozed', 'cozen', 'cozes', 'cozey', 'cozie', 'craal', 'crabs', 'crack', 'craft', 'crags', 'craic', 'craig', 'crake', 'crame', 'cramp', 'crams', 'crane', 'crank', 'crans', 'crape', 'craps', 'crapy', 'crare', 'crash', 'crass', 'crate', 'crave', 'crawl', 'craws', 'crays', 'craze', 'crazy', 'creak', 'cream', 'credo', 'creds', 'creed', 'creek', 'creel', 'creep', 'crees', 'crein', 'crema', 'creme', 'crems', 'crena', 'crepe', 'creps', 'crept', 'crepy', 'cress', 'crest', 'crewe', 'crews', 'crias', 'cribo', 'cribs', 'crick', 'cried', 'crier', 'cries', 'crime', 'crimp', 'crims', 'crine', 'crink', 'crins', 'crios', 'cripe', 'crips', 'crise', 'crisp', 'criss', 'crith', 'crits', 'croak', 'croci', 'crock', 'crocs', 'croft', 'crogs', 'cromb', 'crome', 'crone', 'cronk', 'crons', 'crony', 'crook', 'crool', 'croon', 'crops', 'crore', 'cross', 'crost', 'croup', 'crout', 'crowd', 'crowl', 'crown', 'crows', 'croze', 'cruck', 'crude', 'crudo', 'cruds', 'crudy', 'cruel', 'crues', 'cruet', 'cruft', 'crumb', 'crump', 'crunk', 'cruor', 'crura', 'cruse', 'crush', 'crust', 'crusy', 'cruve', 'crwth', 'cryer', 'cryne', 'crypt', 'ctene', 'cubby', 'cubeb', 'cubed', 'cuber', 'cubes', 'cubic', 'cubit', 'cucks', 'cudda', 'cuddy', 'cueca', 'cuffo', 'cuffs', 'cuifs', 'cuing', 'cuish', 'cuits', 'cukes', 'culch', 'culet', 'culex', 'culls', 'cully', 'culms', 'culpa', 'culti', 'cults', 'culty', 'cumec', 'cumin', 'cundy', 'cunei', 'cunit', 'cunny', 'cunts', 'cupel', 'cupid', 'cuppa', 'cuppy', 'cupro', 'curat', 'curbs', 'curch', 'curds', 'curdy', 'cured', 'curer', 'cures', 'curet', 'curfs', 'curia', 'curie', 'curio', 'curli', 'curls', 'curly', 'curns', 'curny', 'currs', 'curry', 'curse', 'cursi', 'curst', 'curve', 'curvy', 'cusec', 'cushy', 'cusks', 'cusps', 'cuspy', 'cusso', 'cusum', 'cutch', 'cuter', 'cutes', 'cutey', 'cutie', 'cutin', 'cutis', 'cutto', 'cutty', 'cutup', 'cuvee', 'cuzes', 'cwtch', 'cyano', 'cyans', 'cyber', 'cycad', 'cycas', 'cycle', 'cyclo', 'cyder', 'cylix', 'cymae', 'cymar', 'cymas', 'cymes', 'cymol', 'cynic', 'cysts', 'cytes', 'cyton', 'czars', 'daals', 'dabba', 'daces', 'dacha', 'dacks', 'dadah', 'dadas', 'daddy', 'dadis', 'dadla', 'dados', 'daffs', 'daffy', 'dagga', 'daggy', 'dagos', 'dahis', 'dahls', 'daiko', 'daily', 'daine', 'daint', 'dairy', 'daisy', 'daker', 'daled', 'dalek', 'dales', 'dalis', 'dalle', 'dally', 'dalts', 'daman', 'damar', 'dames', 'damme', 'damna', 'damns', 'damps', 'dampy', 'dance', 'dancy', 'danda', 'dandy', 'dangs', 'danio', 'danks', 'danny', 'danse', 'dants', 'dappy', 'daraf', 'darbs', 'darcy', 'dared', 'darer', 'dares', 'darga', 'dargs', 'daric', 'daris', 'darks', 'darky', 'darls', 'darns', 'darre', 'darts', 'darzi', 'dashi', 'dashy', 'datal', 'dated', 'dater', 'dates', 'datil', 'datos', 'datto', 'datum', 'daube', 'daubs', 'dauby', 'dauds', 'dault', 'daunt', 'daurs', 'dauts', 'daven', 'davit', 'dawah', 'dawds', 'dawed', 'dawen', 'dawgs', 'dawks', 'dawns', 'dawts', 'dayal', 'dayan', 'daych', 'daynt', 'dazed', 'dazer', 'dazes', 'dbags', 'deads', 'deair', 'deals', 'dealt', 'deans', 'deare', 'dearn', 'dears', 'deary', 'deash', 'death', 'deave', 'deaws', 'deawy', 'debag', 'debar', 'debby', 'debel', 'debes', 'debit', 'debts', 'debud', 'debug', 'debur', 'debus', 'debut', 'debye', 'decad', 'decaf', 'decal', 'decan', 'decay', 'decim', 'decko', 'decks', 'decor', 'decos', 'decoy', 'decry', 'decyl', 'dedal', 'deeds', 'deedy', 'deely', 'deems', 'deens', 'deeps', 'deere', 'deers', 'deets', 'deeve', 'deevs', 'defat', 'defer', 'deffo', 'defis', 'defog', 'degas', 'degum', 'degus', 'deice', 'deids', 'deify', 'deign', 'deils', 'deink', 'deism', 'deist', 'deity', 'deked', 'dekes', 'dekko', 'delay', 'deled', 'deles', 'delfs', 'delft', 'delis', 'della', 'dells', 'delly', 'delos', 'delph', 'delta', 'delts', 'delve', 'deman', 'demes', 'demic', 'demit', 'demob', 'demoi', 'demon', 'demos', 'demot', 'dempt', 'demur', 'denar', 'denay', 'dench', 'denes', 'denet', 'denim', 'denis', 'dense', 'dente', 'dents', 'deoch', 'deoxy', 'depot', 'depth', 'derat', 'deray', 'derby', 'dered', 'deres', 'derig', 'derma', 'derms', 'derns', 'derny', 'deros', 'derpy', 'derro', 'derry', 'derth', 'dervs', 'desex', 'deshi', 'desis', 'desks', 'desse', 'detag', 'deter', 'detox', 'deuce', 'devas', 'devel', 'devil', 'devis', 'devon', 'devos', 'devot', 'dewan', 'dewar', 'dewax', 'dewed', 'dexes', 'dexie', 'dexys', 'dhaba', 'dhaks', 'dhals', 'dhikr', 'dhobi', 'dhole', 'dholl', 'dhols', 'dhoni', 'dhoti', 'dhows', 'dhuti', 'diact', 'dials', 'diana', 'diane', 'diary', 'diazo', 'dibbs', 'diced', 'dicer', 'dices', 'dicey', 'dicht', 'dicks', 'dicky', 'dicot', 'dicta', 'dicto', 'dicts', 'dictu', 'dicty', 'diddy', 'didie', 'didis', 'didos', 'didst', 'diebs', 'diels', 'diene', 'diets', 'diffs', 'dight', 'digit', 'dikas', 'diked', 'diker', 'dikes', 'dikey', 'dildo', 'dilli', 'dills', 'dilly', 'dimbo', 'dimer', 'dimes', 'dimly', 'dimps', 'dinar', 'dined', 'diner', 'dines', 'dinge', 'dingo', 'dings', 'dingy', 'dinic', 'dinks', 'dinky', 'dinlo', 'dinna', 'dinos', 'dints', 'dioch', 'diode', 'diols', 'diota', 'dippy', 'dipso', 'diram', 'direr', 'dirge', 'dirke', 'dirks', 'dirls', 'dirts', 'dirty', 'disas', 'disci', 'disco', 'discs', 'dishy', 'disks', 'disme', 'dital', 'ditas', 'ditch', 'dited', 'dites', 'ditsy', 'ditto', 'ditts', 'ditty', 'ditzy', 'divan', 'divas', 'dived', 'diver', 'dives', 'divey', 'divis', 'divna', 'divos', 'divot', 'divvy', 'diwan', 'dixie', 'dixit', 'diyas', 'dizen', 'dizzy', 'djinn', 'djins', 'doabs', 'doats', 'dobby', 'dobes', 'dobie', 'dobla', 'doble', 'dobra', 'dobro', 'docht', 'docks', 'docos', 'docus', 'doddy', 'dodge', 'dodgy', 'dodos', 'doeks', 'doers', 'doest', 'doeth', 'doffs', 'dogal', 'dogan', 'doges', 'dogey', 'doggo', 'doggy', 'dogie', 'dogly', 'dogma', 'dohyo', 'doilt', 'doily', 'doing', 'doits', 'dojos', 'dolce', 'dolci', 'doled', 'dolee', 'doles', 'doley', 'dolia', 'dolie', 'dolls', 'dolly', 'dolma', 'dolor', 'dolos', 'dolts', 'domal', 'domed', 'domes', 'domic', 'donah', 'donas', 'donee', 'doner', 'donga', 'dongs', 'donko', 'donna', 'donne', 'donny', 'donor', 'donsy', 'donut', 'doobs', 'dooce', 'doody', 'doofs', 'dooks', 'dooky', 'doole', 'dools', 'dooly', 'dooms', 'doomy', 'doona', 'doorn', 'doors', 'doozy', 'dopas', 'doped', 'doper', 'dopes', 'dopey', 'doppe', 'dorad', 'dorba', 'dorbs', 'doree', 'dores', 'doric', 'doris', 'dorje', 'dorks', 'dorky', 'dorms', 'dormy', 'dorps', 'dorrs', 'dorsa', 'dorse', 'dorts', 'dorty', 'dosai', 'dosas', 'dosed', 'doseh', 'doser', 'doses', 'dosha', 'dotal', 'doted', 'doter', 'dotes', 'dotty', 'douar', 'doubt', 'douce', 'doucs', 'dough', 'douks', 'doula', 'douma', 'doums', 'doups', 'doura', 'douse', 'douts', 'doved', 'doven', 'dover', 'doves', 'dovie', 'dowak', 'dowar', 'dowds', 'dowdy', 'dowed', 'dowel', 'dower', 'dowfs', 'dowie', 'dowle', 'dowls', 'dowly', 'downa', 'downs', 'downy', 'dowps', 'dowry', 'dowse', 'dowts', 'doxed', 'doxes', 'doxie', 'doyen', 'doyly', 'dozed', 'dozen', 'dozer', 'dozes', 'drabs', 'drack', 'draco', 'draff', 'draft', 'drags', 'drail', 'drain', 'drake', 'drama', 'drams', 'drank', 'drant', 'drape', 'draps', 'drapy', 'drats', 'drave', 'drawl', 'drawn', 'draws', 'drays', 'dread', 'dream', 'drear', 'dreck', 'dreed', 'dreer', 'drees', 'dregs', 'dreks', 'drent', 'drere', 'dress', 'drest', 'dreys', 'dribs', 'drice', 'dried', 'drier', 'dries', 'drift', 'drill', 'drily', 'drink', 'drips', 'dript', 'drive', 'drock', 'droid', 'droil', 'droit', 'droke', 'drole', 'droll', 'drome', 'drone', 'drony', 'droob', 'droog', 'drook', 'drool', 'droop', 'drops', 'dropt', 'dross', 'drouk', 'drove', 'drown', 'drows', 'drubs', 'drugs', 'druid', 'drums', 'drunk', 'drupe', 'druse', 'drusy', 'druxy', 'dryad', 'dryas', 'dryer', 'dryly', 'dsobo', 'dsomo', 'duads', 'duals', 'duans', 'duars', 'dubbo', 'dubby', 'ducal', 'ducat', 'duces', 'duchy', 'ducks', 'ducky', 'ducti', 'ducts', 'duddy', 'duded', 'dudes', 'duels', 'duets', 'duett', 'duffs', 'dufus', 'duing', 'duits', 'dukas', 'duked', 'dukes', 'dukka', 'dukun', 'dulce', 'dules', 'dulia', 'dulls', 'dully', 'dulse', 'dumas', 'dumbo', 'dumbs', 'dumka', 'dumky', 'dummy', 'dumps', 'dumpy', 'dunam', 'dunce', 'dunch', 'dunes', 'dungs', 'dungy', 'dunks', 'dunno', 'dunny', 'dunsh', 'dunts', 'duomi', 'duomo', 'duped', 'duper', 'dupes', 'duple', 'duply', 'duppy', 'dural', 'duras', 'dured', 'dures', 'durgy', 'durns', 'duroc', 'duros', 'duroy', 'durra', 'durrs', 'durry', 'durst', 'durum', 'durzi', 'dusks', 'dusky', 'dusts', 'dusty', 'dutch', 'duvet', 'duxes', 'dwaal', 'dwale', 'dwalm', 'dwams', 'dwamy', 'dwang', 'dwarf', 'dwaum', 'dweeb', 'dwell', 'dwelt', 'dwile', 'dwine', 'dyads', 'dyers', 'dying', 'dyked', 'dykes', 'dykey', 'dykon', 'dynel', 'dynes', 'dynos', 'dzhos', 'eager', 'eagle', 'eagly', 'eagre', 'ealed', 'eales', 'eaned', 'eards', 'eared', 'earls', 'early', 'earns', 'earnt', 'earst', 'earth', 'eased', 'easel', 'easer', 'eases', 'easle', 'easts', 'eaten', 'eater', 'eathe', 'eatin', 'eaved', 'eaver', 'eaves', 'ebank', 'ebbed', 'ebbet', 'ebena', 'ebene', 'ebike', 'ebons', 'ebony', 'ebook', 'ecads', 'ecard', 'ecash', 'eched', 'eches', 'echos', 'ecigs', 'eclat', 'ecole', 'ecrus', 'edema', 'edged', 'edger', 'edges', 'edict', 'edify', 'edile', 'edits', 'educe', 'educt', 'eejit', 'eensy', 'eerie', 'eeven', 'eever', 'eevns', 'effed', 'effer', 'efits', 'egads', 'egers', 'egest', 'eggar', 'egged', 'egger', 'egmas', 'egret', 'ehing', 'eider', 'eidos', 'eight', 'eigne', 'eiked', 'eikon', 'eilds', 'eiron', 'eisel', 'eject', 'ejido', 'ekdam', 'eking', 'ekkas', 'elain', 'eland', 'elans', 'elate', 'elbow', 'elchi', 'elder', 'eldin', 'elect', 'eleet', 'elegy', 'elemi', 'elfed', 'elfin', 'eliad', 'elide', 'elint', 'elite', 'elmen', 'eloge', 'elogy', 'eloin', 'elope', 'elops', 'elpee', 'elsin', 'elude', 'elute', 'elvan', 'elven', 'elver', 'elves', 'emacs', 'email', 'embar', 'embay', 'embed', 'ember', 'embog', 'embow', 'embox', 'embus', 'emcee', 'emeer', 'emend', 'emerg', 'emery', 'emeus', 'emics', 'emirs', 'emits', 'emmas', 'emmer', 'emmet', 'emmew', 'emmys', 'emoji', 'emong', 'emote', 'emove', 'empts', 'empty', 'emule', 'emure', 'emyde', 'emyds', 'enact', 'enarm', 'enate', 'ended', 'ender', 'endew', 'endow', 'endue', 'enema', 'enemy', 'enews', 'enfix', 'eniac', 'enjoy', 'enlit', 'enmew', 'ennog', 'ennui', 'enoki', 'enols', 'enorm', 'enows', 'enrol', 'ensew', 'ensky', 'ensue', 'enter', 'entia', 'entre', 'entry', 'enure', 'enurn', 'envoi', 'envoy', 'enzym', 'eolid', 'eorls', 'eosin', 'epact', 'epees', 'epena', 'epene', 'ephah', 'ephas', 'ephod', 'ephor', 'epics', 'epoch', 'epode', 'epopt', 'epoxy', 'eppie', 'epris', 'equal', 'eques', 'equid', 'equip', 'erase', 'erbia', 'erect', 'erevs', 'ergon', 'ergos', 'ergot', 'erhus', 'erica', 'erick', 'erics', 'ering', 'erned', 'ernes', 'erode', 'erose', 'erred', 'error', 'erses', 'eruct', 'erugo', 'erupt', 'eruvs', 'erven', 'ervil', 'escar', 'escot', 'esile', 'eskar', 'esker', 'esnes', 'esrog', 'essay', 'esses', 'ester', 'estoc', 'estop', 'estro', 'etage', 'etape', 'etats', 'etens', 'ethal', 'ether', 'ethic', 'ethne', 'ethos', 'ethyl', 'etics', 'etnas', 'etrog', 'ettin', 'ettle', 'etude', 'etuis', 'etwee', 'etyma', 'eughs', 'euked', 'eupad', 'euros', 'eusol', 'evade', 'evegs', 'evens', 'event', 'evert', 'every', 'evets', 'evhoe', 'evict', 'evils', 'evite', 'evohe', 'evoke', 'ewers', 'ewest', 'ewhow', 'ewked', 'exact', 'exalt', 'exams', 'excel', 'exeat', 'execs', 'exeem', 'exeme', 'exert', 'exfil', 'exier', 'exies', 'exile', 'exine', 'exing', 'exist', 'exite', 'exits', 'exode', 'exome', 'exons', 'expat', 'expel', 'expos', 'extol', 'extra', 'exude', 'exuls', 'exult', 'exurb', 'eyass', 'eyers', 'eying', 'eyots', 'eyras', 'eyres', 'eyrie', 'eyrir', 'ezine', 'fabbo', 'fabby', 'fable', 'faced', 'facer', 'faces', 'facet', 'facey', 'facia', 'facie', 'facta', 'facto', 'facts', 'facty', 'faddy', 'faded', 'fader', 'fades', 'fadge', 'fados', 'faena', 'faery', 'faffs', 'faffy', 'faggy', 'fagin', 'fagot', 'faiks', 'fails', 'faine', 'fains', 'faint', 'faire', 'fairs', 'fairy', 'faith', 'faked', 'faker', 'fakes', 'fakey', 'fakie', 'fakir', 'falaj', 'fales', 'falls', 'false', 'falsy', 'famed', 'fames', 'fanal', 'fancy', 'fands', 'fanes', 'fanga', 'fango', 'fangs', 'fanks', 'fanny', 'fanon', 'fanos', 'fanum', 'faqir', 'farad', 'farce', 'farci', 'farcy', 'fards', 'fared', 'farer', 'fares', 'farle', 'farls', 'farms', 'faros', 'farro', 'farse', 'farts', 'fasci', 'fasti', 'fasts', 'fatal', 'fated', 'fates', 'fatly', 'fatso', 'fatty', 'fatwa', 'fauch', 'faugh', 'fauld', 'fault', 'fauna', 'fauns', 'faurd', 'faute', 'fauts', 'fauve', 'favas', 'favel', 'faver', 'faves', 'favor', 'favus', 'fawns', 'fawny', 'faxed', 'faxes', 'fayed', 'fayer', 'fayne', 'fayre', 'fazed', 'fazes', 'feals', 'feard', 'feare', 'fears', 'feart', 'fease', 'feast', 'feats', 'feaze', 'fecal', 'feces', 'fecht', 'fecit', 'fecks', 'fedai', 'fedex', 'feebs', 'feeds', 'feels', 'feely', 'feens', 'feers', 'feese', 'feeze', 'fehme', 'feign', 'feint', 'feist', 'felch', 'felid', 'felix', 'fella', 'fells', 'felly', 'felon', 'felts', 'felty', 'femal', 'femes', 'femic', 'femme', 'femmy', 'femur', 'fence', 'fends', 'fendy', 'fenis', 'fenks', 'fenny', 'fents', 'feods', 'feoff', 'feral', 'ferer', 'feres', 'feria', 'ferly', 'fermi', 'ferms', 'ferns', 'ferny', 'ferox', 'ferry', 'fesse', 'festa', 'fests', 'festy', 'fetal', 'fetas', 'fetch', 'feted', 'fetes', 'fetid', 'fetor', 'fetta', 'fetts', 'fetus', 'fetwa', 'feuar', 'feuds', 'feued', 'fever', 'fewer', 'feyed', 'feyer', 'feyly', 'fezes', 'fezzy', 'fiars', 'fiats', 'fiber', 'fibre', 'fibro', 'fices', 'fiche', 'fichu', 'ficin', 'ficos', 'ficta', 'ficus', 'fides', 'fidge', 'fidos', 'fidus', 'fiefs', 'field', 'fiend', 'fient', 'fiere', 'fieri', 'fiers', 'fiery', 'fiest', 'fifed', 'fifer', 'fifes', 'fifis', 'fifth', 'fifty', 'figgy', 'fight', 'figos', 'fiked', 'fikes', 'filar', 'filch', 'filed', 'filer', 'files', 'filet', 'filii', 'filks', 'fille', 'fillo', 'fills', 'filly', 'filmi', 'films', 'filmy', 'filon', 'filos', 'filth', 'filum', 'final', 'finca', 'finch', 'finds', 'fined', 'finer', 'fines', 'finis', 'finks', 'finny', 'finos', 'fiord', 'fiqhs', 'fique', 'fired', 'firer', 'fires', 'firie', 'firks', 'firma', 'firms', 'firni', 'firns', 'firry', 'first', 'firth', 'fiscs', 'fisho', 'fishy', 'fisks', 'fists', 'fisty', 'fitch', 'fitly', 'fitna', 'fitte', 'fitts', 'fiver', 'fives', 'fixed', 'fixer', 'fixes', 'fixie', 'fixit', 'fizzy', 'fjeld', 'fjord', 'flabs', 'flack', 'flaff', 'flags', 'flail', 'flair', 'flake', 'flaks', 'flaky', 'flame', 'flamm', 'flams', 'flamy', 'flane', 'flank', 'flans', 'flaps', 'flare', 'flary', 'flash', 'flask', 'flats', 'flava', 'flawn', 'flaws', 'flawy', 'flaxy', 'flays', 'fleam', 'fleas', 'fleck', 'fleek', 'fleer', 'flees', 'fleet', 'flegs', 'fleme', 'flesh', 'fleur', 'flews', 'flexi', 'flexo', 'fleys', 'flick', 'flics', 'flied', 'flier', 'flies', 'flimp', 'flims', 'fling', 'flint', 'flips', 'flirs', 'flirt', 'flisk', 'flite', 'flits', 'flitt', 'float', 'flobs', 'flock', 'flocs', 'floes', 'flogs', 'flong', 'flood', 'floor', 'flops', 'flora', 'flore', 'flors', 'flory', 'flosh', 'floss', 'flota', 'flote', 'flour', 'flout', 'flown', 'flows', 'flowy', 'flubs', 'flued', 'flues', 'fluey', 'fluff', 'fluid', 'fluke', 'fluky', 'flume', 'flump', 'flung', 'flunk', 'fluor', 'flurr', 'flush', 'flute', 'fluty', 'fluyt', 'flyby', 'flyer', 'flyin', 'flype', 'flyte', 'fnarr', 'foals', 'foams', 'foamy', 'focal', 'focus', 'foehn', 'fogey', 'foggy', 'fogie', 'fogle', 'fogos', 'fogou', 'fohns', 'foids', 'foils', 'foins', 'foist', 'folds', 'foley', 'folia', 'folic', 'folie', 'folio', 'folks', 'folky', 'folly', 'fomes', 'fonda', 'fonds', 'fondu', 'fones', 'fonio', 'fonly', 'fonts', 'foods', 'foody', 'fools', 'foots', 'footy', 'foram', 'foray', 'forbs', 'forby', 'force', 'fordo', 'fords', 'forel', 'fores', 'forex', 'forge', 'forgo', 'forks', 'forky', 'forma', 'forme', 'forms', 'forte', 'forth', 'forts', 'forty', 'forum', 'forza', 'forze', 'fossa', 'fosse', 'fouat', 'fouds', 'fouer', 'fouet', 'foule', 'fouls', 'found', 'fount', 'fours', 'fouth', 'fovea', 'fowls', 'fowth', 'foxed', 'foxes', 'foxie', 'foyer', 'foyle', 'foyne', 'frabs', 'frack', 'fract', 'frags', 'frail', 'fraim', 'frais', 'frame', 'franc', 'frank', 'frape', 'fraps', 'frass', 'frate', 'frati', 'frats', 'fraud', 'fraus', 'frays', 'freak', 'freed', 'freer', 'frees', 'freet', 'freit', 'fremd', 'frena', 'freon', 'frere', 'fresh', 'frets', 'friar', 'fribs', 'fried', 'frier', 'fries', 'frigs', 'frill', 'frise', 'frisk', 'frist', 'frita', 'frite', 'frith', 'frits', 'fritt', 'fritz', 'frize', 'frizz', 'frock', 'froes', 'frogs', 'fromm', 'frond', 'frons', 'front', 'froom', 'frore', 'frorn', 'frory', 'frosh', 'frost', 'froth', 'frown', 'frows', 'frowy', 'froyo', 'froze', 'frugs', 'fruit', 'frump', 'frush', 'frust', 'fryer', 'fubar', 'fubby', 'fubsy', 'fucks', 'fucus', 'fuddy', 'fudge', 'fudgy', 'fuels', 'fuero', 'fuffs', 'fuffy', 'fugal', 'fuggy', 'fugie', 'fugio', 'fugis', 'fugle', 'fugly', 'fugue', 'fugus', 'fujis', 'fulla', 'fulls', 'fully', 'fulth', 'fulwa', 'fumed', 'fumer', 'fumes', 'fumet', 'funda', 'fundi', 'fundo', 'funds', 'fundy', 'fungi', 'fungo', 'fungs', 'funic', 'funis', 'funks', 'funky', 'funny', 'funsy', 'funts', 'fural', 'furan', 'furca', 'furls', 'furol', 'furor', 'furos', 'furrs', 'furry', 'furth', 'furze', 'furzy', 'fused', 'fusee', 'fusel', 'fuses', 'fusil', 'fusks', 'fussy', 'fusts', 'fusty', 'futon', 'fuzed', 'fuzee', 'fuzes', 'fuzil', 'fuzzy', 'fyces', 'fyked', 'fykes', 'fyles', 'fyrds', 'fytte', 'gabba', 'gabby', 'gable', 'gaddi', 'gades', 'gadge', 'gadgy', 'gadid', 'gadis', 'gadje', 'gadjo', 'gadso', 'gaffe', 'gaffs', 'gaged', 'gager', 'gages', 'gaids', 'gaily', 'gains', 'gairs', 'gaita', 'gaits', 'gaitt', 'gajos', 'galah', 'galas', 'galax', 'galea', 'galed', 'gales', 'galia', 'galis', 'galls', 'gally', 'galop', 'galut', 'galvo', 'gamas', 'gamay', 'gamba', 'gambe', 'gambo', 'gambs', 'gamed', 'gamer', 'games', 'gamey', 'gamic', 'gamin', 'gamma', 'gamme', 'gammy', 'gamps', 'gamut', 'ganch', 'gandy', 'ganef', 'ganev', 'gangs', 'ganja', 'ganks', 'ganof', 'gants', 'gaols', 'gaped', 'gaper', 'gapes', 'gapos', 'gappy', 'garam', 'garba', 'garbe', 'garbo', 'garbs', 'garda', 'garde', 'gares', 'garis', 'garms', 'garni', 'garre', 'garri', 'garth', 'garum', 'gases', 'gashy', 'gasps', 'gaspy', 'gassy', 'gasts', 'gatch', 'gated', 'gater', 'gates', 'gaths', 'gator', 'gauch', 'gaucy', 'gauds', 'gaudy', 'gauge', 'gauje', 'gault', 'gaums', 'gaumy', 'gaunt', 'gaups', 'gaurs', 'gauss', 'gauze', 'gauzy', 'gavel', 'gavot', 'gawcy', 'gawds', 'gawks', 'gawky', 'gawps', 'gawsy', 'gayal', 'gayer', 'gayly', 'gazal', 'gazar', 'gazed', 'gazer', 'gazes', 'gazon', 'gazoo', 'geals', 'geans', 'geare', 'gears', 'geasa', 'geats', 'gebur', 'gecko', 'gecks', 'geeks', 'geeky', 'geeps', 'geese', 'geest', 'geist', 'geits', 'gelds', 'gelee', 'gelid', 'gelly', 'gelts', 'gemel', 'gemma', 'gemmy', 'gemot', 'genae', 'genal', 'genas', 'genes', 'genet', 'genic', 'genie', 'genii', 'genin', 'genio', 'genip', 'genny', 'genoa', 'genom', 'genre', 'genro', 'gents', 'genty', 'genua', 'genus', 'geode', 'geoid', 'gerah', 'gerbe', 'geres', 'gerle', 'germs', 'germy', 'gerne', 'gesse', 'gesso', 'geste', 'gests', 'getas', 'getup', 'geums', 'geyan', 'geyer', 'ghast', 'ghats', 'ghaut', 'ghazi', 'ghees', 'ghest', 'ghost', 'ghoul', 'ghusl', 'ghyll', 'giant', 'gibed', 'gibel', 'giber', 'gibes', 'gibli', 'gibus', 'giddy', 'gifts', 'gigas', 'gighe', 'gigot', 'gigue', 'gilas', 'gilds', 'gilet', 'gilia', 'gills', 'gilly', 'gilpy', 'gilts', 'gimel', 'gimme', 'gimps', 'gimpy', 'ginch', 'ginga', 'ginge', 'gings', 'ginks', 'ginny', 'ginzo', 'gipon', 'gippo', 'gippy', 'gipsy', 'girds', 'girlf', 'girls', 'girly', 'girns', 'giron', 'giros', 'girrs', 'girsh', 'girth', 'girts', 'gismo', 'gisms', 'gists', 'gitch', 'gites', 'giust', 'gived', 'given', 'giver', 'gives', 'gizmo', 'glace', 'glade', 'glads', 'glady', 'glaik', 'glair', 'glamp', 'glams', 'gland', 'glans', 'glare', 'glary', 'glass', 'glatt', 'glaum', 'glaur', 'glaze', 'glazy', 'gleam', 'glean', 'gleba', 'glebe', 'gleby', 'glede', 'gleds', 'gleed', 'gleek', 'glees', 'gleet', 'gleis', 'glens', 'glent', 'gleys', 'glial', 'glias', 'glibs', 'glide', 'gliff', 'glift', 'glike', 'glime', 'glims', 'glint', 'glisk', 'glits', 'glitz', 'gloam', 'gloat', 'globe', 'globi', 'globs', 'globy', 'glode', 'glogg', 'gloms', 'gloom', 'gloop', 'glops', 'glory', 'gloss', 'glost', 'glout', 'glove', 'glows', 'glowy', 'gloze', 'glued', 'gluer', 'glues', 'gluey', 'glugg', 'glugs', 'glume', 'glums', 'gluon', 'glute', 'gluts', 'glyph', 'gnapi', 'gnarl', 'gnarr', 'gnars', 'gnash', 'gnats', 'gnawn', 'gnaws', 'gnome', 'gnows', 'goads', 'goafs', 'goaft', 'goals', 'goary', 'goats', 'goaty', 'goave', 'goban', 'gobar', 'gobbe', 'gobbi', 'gobbo', 'gobby', 'gobis', 'gobos', 'godet', 'godly', 'godso', 'goels', 'goers', 'goest', 'goeth', 'goety', 'gofer', 'goffs', 'gogga', 'gogos', 'goier', 'going', 'gojis', 'gokes', 'golds', 'goldy', 'golem', 'goles', 'golfs', 'golly', 'golpe', 'golps', 'gombo', 'gomer', 'gompa', 'gonad', 'gonch', 'gonef', 'goner', 'gongs', 'gonia', 'gonif', 'gonks', 'gonna', 'gonof', 'gonys', 'gonzo', 'gooby', 'goodo', 'goods', 'goody', 'gooey', 'goofs', 'goofy', 'googs', 'gooks', 'gooky', 'goold', 'gools', 'gooly', 'goomy', 'goons', 'goony', 'goops', 'goopy', 'goors', 'goory', 'goose', 'goosy', 'gopak', 'gopik', 'goral', 'goras', 'goray', 'gorbs', 'gordo', 'gored', 'gores', 'gorge', 'goris', 'gorms', 'gormy', 'gorps', 'gorse', 'gorsy', 'gosht', 'gosse', 'gotch', 'goths', 'gothy', 'gotta', 'gouch', 'gouge', 'gouks', 'goura', 'gourd', 'gouts', 'gouty', 'goved', 'goves', 'gowan', 'gowds', 'gowfs', 'gowks', 'gowls', 'gowns', 'goxes', 'goyim', 'goyle', 'graal', 'grabs', 'grace', 'grade', 'grads', 'graff', 'graft', 'grail', 'grain', 'graip', 'grama', 'grame', 'gramp', 'grams', 'grana', 'grand', 'grano', 'grans', 'grant', 'grape', 'graph', 'grapy', 'grasp', 'grass', 'grata', 'grate', 'grats', 'grave', 'gravs', 'gravy', 'grays', 'graze', 'great', 'grebe', 'grebo', 'grece', 'greed', 'greek', 'green', 'grees', 'greet', 'grege', 'grego', 'grein', 'grens', 'greps', 'grese', 'greve', 'grews', 'greys', 'grice', 'gride', 'grids', 'grief', 'griff', 'grift', 'grigs', 'grike', 'grill', 'grime', 'grimy', 'grind', 'grins', 'griot', 'gripe', 'grips', 'gript', 'gripy', 'grise', 'grist', 'grisy', 'grith', 'grits', 'grize', 'groan', 'groat', 'grody', 'grogs', 'groin', 'groks', 'groma', 'groms', 'grone', 'groof', 'groom', 'grope', 'gross', 'grosz', 'grots', 'grouf', 'group', 'grout', 'grove', 'grovy', 'growl', 'grown', 'grows', 'grrls', 'grrrl', 'grubs', 'grued', 'gruel', 'grues', 'grufe', 'gruff', 'grume', 'grump', 'grund', 'grunt', 'gryce', 'gryde', 'gryke', 'grype', 'grypt', 'guaco', 'guana', 'guano', 'guans', 'guard', 'guars', 'guava', 'gubba', 'gucks', 'gucky', 'gudes', 'guess', 'guest', 'guffs', 'gugas', 'guggl', 'guide', 'guido', 'guids', 'guild', 'guile', 'guilt', 'guimp', 'guiro', 'guise', 'gulab', 'gulag', 'gular', 'gulas', 'gulch', 'gules', 'gulet', 'gulfs', 'gulfy', 'gulls', 'gully', 'gulph', 'gulps', 'gulpy', 'gumbo', 'gumma', 'gummi', 'gummy', 'gumps', 'gunas', 'gundi', 'gundy', 'gunge', 'gungy', 'gunks', 'gunky', 'gunny', 'guppy', 'guqin', 'gurdy', 'gurge', 'gurks', 'gurls', 'gurly', 'gurns', 'gurry', 'gursh', 'gurus', 'gushy', 'gusla', 'gusle', 'gusli', 'gussy', 'gusto', 'gusts', 'gusty', 'gutsy', 'gutta', 'gutty', 'guyed', 'guyle', 'guyot', 'guyse', 'gwine', 'gyals', 'gyans', 'gybed', 'gybes', 'gyeld', 'gymps', 'gynae', 'gynie', 'gynny', 'gynos', 'gyoza', 'gypes', 'gypos', 'gyppo', 'gyppy', 'gypsy', 'gyral', 'gyred', 'gyres', 'gyron', 'gyros', 'gyrus', 'gytes', 'gyved', 'gyver', 'gyves', 'haafs', 'haars', 'haats', 'habit', 'hable', 'habus', 'hacek', 'hacks', 'hacky', 'hadal', 'haded', 'hades', 'hadji', 'hadst', 'haems', 'haere', 'haets', 'haffs', 'hafiz', 'hafta', 'hafts', 'haggs', 'haham', 'hahas', 'haick', 'haika', 'haiks', 'haiku', 'hails', 'haily', 'hains', 'haint', 'hairs', 'hairy', 'haith', 'hajes', 'hajis', 'hajji', 'hakam', 'hakas', 'hakea', 'hakes', 'hakim', 'hakus', 'halal', 'haldi', 'haled', 'haler', 'hales', 'halfa', 'halfs', 'halid', 'hallo', 'halls', 'halma', 'halms', 'halon', 'halos', 'halse', 'halsh', 'halts', 'halva', 'halve', 'halwa', 'hamal', 'hamba', 'hamed', 'hamel', 'hames', 'hammy', 'hamza', 'hanap', 'hance', 'hanch', 'handi', 'hands', 'handy', 'hangi', 'hangs', 'hanks', 'hanky', 'hansa', 'hanse', 'hants', 'haole', 'haoma', 'hapas', 'hapax', 'haply', 'happi', 'happy', 'hapus', 'haram', 'hards', 'hardy', 'hared', 'harem', 'hares', 'harim', 'harks', 'harls', 'harms', 'harns', 'haros', 'harps', 'harpy', 'harry', 'harsh', 'harts', 'hashy', 'hasks', 'hasps', 'hasta', 'haste', 'hasty', 'hatch', 'hated', 'hater', 'hates', 'hatha', 'hathi', 'hatty', 'hauds', 'haufs', 'haugh', 'haugo', 'hauld', 'haulm', 'hauls', 'hault', 'hauns', 'haunt', 'hause', 'haute', 'havan', 'havel', 'haven', 'haver', 'haves', 'havoc', 'hawed', 'hawks', 'hawms', 'hawse', 'hayed', 'hayer', 'hayey', 'hayle', 'hazan', 'hazed', 'hazel', 'hazer', 'hazes', 'hazle', 'heads', 'heady', 'heald', 'heals', 'heame', 'heaps', 'heapy', 'heard', 'heare', 'hears', 'heart', 'heast', 'heath', 'heats', 'heaty', 'heave', 'heavy', 'heben', 'hebes', 'hecht', 'hecks', 'heder', 'hedge', 'hedgy', 'heeds', 'heedy', 'heels', 'heeze', 'hefte', 'hefts', 'hefty', 'heiau', 'heids', 'heigh', 'heils', 'heirs', 'heist', 'hejab', 'hejra', 'heled', 'heles', 'helio', 'helix', 'hella', 'hello', 'hells', 'helly', 'helms', 'helos', 'helot', 'helps', 'helve', 'hemal', 'hemes', 'hemic', 'hemin', 'hemps', 'hempy', 'hence', 'hench', 'hends', 'henge', 'henna', 'henny', 'henry', 'hents', 'hepar', 'herbs', 'herby', 'herds', 'heres', 'herls', 'herma', 'herms', 'herns', 'heron', 'heros', 'herps', 'herry', 'herse', 'hertz', 'herye', 'hesps', 'hests', 'hetes', 'heths', 'heuch', 'heugh', 'hevea', 'hevel', 'hewed', 'hewer', 'hewgh', 'hexad', 'hexed', 'hexer', 'hexes', 'hexyl', 'heyed', 'hiant', 'hibas', 'hicks', 'hided', 'hider', 'hides', 'hiems', 'hifis', 'highs', 'hight', 'hijab', 'hijra', 'hiked', 'hiker', 'hikes', 'hikoi', 'hilar', 'hilch', 'hillo', 'hills', 'hilly', 'hilsa', 'hilts', 'hilum', 'hilus', 'himbo', 'hinau', 'hinds', 'hinge', 'hings', 'hinky', 'hinny', 'hints', 'hiois', 'hiped', 'hiper', 'hipes', 'hiply', 'hippo', 'hippy', 'hired', 'hiree', 'hirer', 'hires', 'hissy', 'hists', 'hitch', 'hithe', 'hived', 'hiver', 'hives', 'hizen', 'hoach', 'hoaed', 'hoagy', 'hoard', 'hoars', 'hoary', 'hoast', 'hobby', 'hobos', 'hocks', 'hocus', 'hodad', 'hodja', 'hoers', 'hogan', 'hogen', 'hoggs', 'hoghs', 'hogoh', 'hogos', 'hohed', 'hoick', 'hoied', 'hoiks', 'hoing', 'hoise', 'hoist', 'hokas', 'hoked', 'hokes', 'hokey', 'hokis', 'hokku', 'hokum', 'holds', 'holed', 'holes', 'holey', 'holks', 'holla', 'hollo', 'holly', 'holme', 'holms', 'holon', 'holos', 'holts', 'homas', 'homed', 'homer', 'homes', 'homey', 'homie', 'homme', 'homos', 'honan', 'honda', 'honds', 'honed', 'honer', 'hones', 'honey', 'hongi', 'hongs', 'honks', 'honky', 'honor', 'hooch', 'hoods', 'hoody', 'hooey', 'hoofs', 'hoogo', 'hooha', 'hooka', 'hooks', 'hooky', 'hooly', 'hoons', 'hoops', 'hoord', 'hoors', 'hoosh', 'hoots', 'hooty', 'hoove', 'hopak', 'hoped', 'hoper', 'hopes', 'hoppy', 'horah', 'horal', 'horas', 'horde', 'horis', 'horks', 'horme', 'horns', 'horny', 'horse', 'horst', 'horsy', 'hosed', 'hosel', 'hosen', 'hoser', 'hoses', 'hosey', 'hosta', 'hosts', 'hotch', 'hotel', 'hoten', 'hotis', 'hotly', 'hotte', 'hotty', 'houff', 'houfs', 'hough', 'hound', 'houri', 'hours', 'house', 'houts', 'hovea', 'hoved', 'hovel', 'hoven', 'hover', 'hoves', 'howay', 'howbe', 'howdy', 'howes', 'howff', 'howfs', 'howks', 'howls', 'howre', 'howso', 'howto', 'hoxed', 'hoxes', 'hoyas', 'hoyed', 'hoyle', 'hubba', 'hubby', 'hucks', 'hudna', 'hudud', 'huers', 'huffs', 'huffy', 'huger', 'huggy', 'huhus', 'huias', 'huies', 'hukou', 'hulas', 'hules', 'hulks', 'hulky', 'hullo', 'hulls', 'hully', 'human', 'humas', 'humfs', 'humic', 'humid', 'humor', 'humph', 'humps', 'humpy', 'humus', 'hunch', 'hundo', 'hunks', 'hunky', 'hunts', 'hurds', 'hurls', 'hurly', 'hurra', 'hurry', 'hurst', 'hurts', 'hurty', 'hushy', 'husks', 'husky', 'husos', 'hussy', 'hutch', 'hutia', 'huzza', 'huzzy', 'hwyls', 'hydel', 'hydra', 'hydro', 'hyena', 'hyens', 'hygge', 'hying', 'hykes', 'hylas', 'hyleg', 'hyles', 'hylic', 'hymen', 'hymns', 'hynde', 'hyoid', 'hyped', 'hyper', 'hypes', 'hypha', 'hyphy', 'hypos', 'hyrax', 'hyson', 'hythe', 'iambi', 'iambs', 'ibrik', 'icers', 'iched', 'iches', 'ichor', 'icier', 'icily', 'icing', 'icker', 'ickle', 'icons', 'ictal', 'ictic', 'ictus', 'idant', 'iddah', 'iddat', 'iddut', 'ideal', 'ideas', 'idees', 'ident', 'idiom', 'idiot', 'idled', 'idler', 'idles', 'idlis', 'idola', 'idols', 'idyll', 'idyls', 'iftar', 'igapo', 'igged', 'igloo', 'iglus', 'ignis', 'ihram', 'iiwis', 'ikans', 'ikats', 'ikons', 'ileac', 'ileal', 'ileum', 'ileus', 'iliac', 'iliad', 'ilial', 'ilium', 'iller', 'illth', 'image', 'imago', 'imagy', 'imams', 'imari', 'imaum', 'imbar', 'imbed', 'imbos', 'imbue', 'imide', 'imido', 'imids', 'imine', 'imino', 'imlis', 'immew', 'immit', 'immix', 'imped', 'impel', 'impis', 'imply', 'impot', 'impro', 'imshi', 'imshy', 'inane', 'inapt', 'inarm', 'inbox', 'inbye', 'incas', 'incel', 'incle', 'incog', 'incur', 'incus', 'incut', 'indew', 'index', 'india', 'indie', 'indol', 'indow', 'indri', 'indue', 'inept', 'inerm', 'inert', 'infer', 'infix', 'infos', 'infra', 'ingan', 'ingle', 'ingot', 'inion', 'inked', 'inker', 'inkle', 'inlay', 'inlet', 'inned', 'inner', 'innie', 'innit', 'inorb', 'input', 'inros', 'inrun', 'insee', 'inset', 'inspo', 'intel', 'inter', 'intil', 'intis', 'intra', 'intro', 'inula', 'inure', 'inurn', 'inust', 'invar', 'inver', 'inwit', 'iodic', 'iodid', 'iodin', 'ionic', 'ioras', 'iotas', 'ippon', 'irade', 'irate', 'irids', 'iring', 'irked', 'iroko', 'irone', 'irons', 'irony', 'isbas', 'ishes', 'isled', 'isles', 'islet', 'isnae', 'issei', 'issue', 'istle', 'itchy', 'items', 'ither', 'ivied', 'ivies', 'ivory', 'ixias', 'ixnay', 'ixora', 'ixtle', 'izard', 'izars', 'izzat', 'jaaps', 'jabot', 'jacal', 'jacet', 'jacks', 'jacky', 'jaded', 'jades', 'jafas', 'jaffa', 'jagas', 'jager', 'jaggs', 'jaggy', 'jagir', 'jagra', 'jails', 'jaker', 'jakes', 'jakey', 'jakie', 'jalap', 'jaleo', 'jalop', 'jambe', 'jambo', 'jambs', 'jambu', 'james', 'jammy', 'jamon', 'jamun', 'janes', 'janky', 'janns', 'janny', 'janty', 'japan', 'japed', 'japer', 'japes', 'jarks', 'jarls', 'jarps', 'jarta', 'jarul', 'jasey', 'jaspe', 'jasps', 'jatha', 'jatis', 'jatos', 'jauks', 'jaune', 'jaunt', 'jaups', 'javas', 'javel', 'jawan', 'jawed', 'jawns', 'jaxie', 'jazzy', 'jeans', 'jeats', 'jebel', 'jedis', 'jeels', 'jeely', 'jeeps', 'jeera', 'jeers', 'jeeze', 'jefes', 'jeffs', 'jehad', 'jehus', 'jelab', 'jello', 'jells', 'jelly', 'jembe', 'jemmy', 'jenny', 'jeons', 'jerid', 'jerks', 'jerky', 'jerry', 'jesse', 'jessy', 'jests', 'jesus', 'jetee', 'jetes', 'jeton', 'jetty', 'jeune', 'jewed', 'jewel', 'jewie', 'jhala', 'jheel', 'jhils', 'jiaos', 'jibba', 'jibbs', 'jibed', 'jiber', 'jibes', 'jiffs', 'jiffy', 'jiggy', 'jigot', 'jihad', 'jills', 'jilts', 'jimmy', 'jimpy', 'jingo', 'jings', 'jinks', 'jinne', 'jinni', 'jinns', 'jirds', 'jirga', 'jirre', 'jisms', 'jitis', 'jitty', 'jived', 'jiver', 'jives', 'jivey', 'jnana', 'jobed', 'jobes', 'jocko', 'jocks', 'jocky', 'jocos', 'jodel', 'joeys', 'johns', 'joins', 'joint', 'joist', 'joked', 'joker', 'jokes', 'jokey', 'jokol', 'joled', 'joles', 'jolie', 'jollo', 'jolls', 'jolly', 'jolts', 'jolty', 'jomon', 'jomos', 'jones', 'jongs', 'jonty', 'jooks', 'joram', 'jorts', 'jorum', 'jotas', 'jotty', 'jotun', 'joual', 'jougs', 'jouks', 'joule', 'jours', 'joust', 'jowar', 'jowed', 'jowls', 'jowly', 'joyed', 'jubas', 'jubes', 'jucos', 'judas', 'judge', 'judgy', 'judos', 'jugal', 'jugum', 'juice', 'juicy', 'jujus', 'juked', 'jukes', 'jukus', 'julep', 'julia', 'jumar', 'jumbo', 'jumby', 'jumps', 'jumpy', 'junco', 'junks', 'junky', 'junta', 'junto', 'jupes', 'jupon', 'jural', 'jurat', 'jurel', 'jures', 'juris', 'juror', 'juste', 'justs', 'jutes', 'jutty', 'juves', 'juvie', 'kaama', 'kabab', 'kabar', 'kabob', 'kacha', 'kacks', 'kadai', 'kades', 'kadis', 'kafir', 'kagos', 'kagus', 'kahal', 'kaiak', 'kaids', 'kaies', 'kaifs', 'kaika', 'kaiks', 'kails', 'kaims', 'kaing', 'kains', 'kajal', 'kakas', 'kakis', 'kalam', 'kalas', 'kales', 'kalif', 'kalis', 'kalpa', 'kalua', 'kamas', 'kames', 'kamik', 'kamis', 'kamme', 'kanae', 'kanal', 'kanas', 'kanat', 'kandy', 'kaneh', 'kanes', 'kanga', 'kangs', 'kanji', 'kants', 'kanzu', 'kaons', 'kapai', 'kapas', 'kapha', 'kaphs', 'kapok', 'kapow', 'kappa', 'kapur', 'kapus', 'kaput', 'karai', 'karas', 'karat', 'karee', 'karez', 'karks', 'karma', 'karns', 'karoo', 'karos', 'karri', 'karst', 'karsy', 'karts', 'karzy', 'kasha', 'kasme', 'katal', 'katas', 'katis', 'katti', 'kaugh', 'kauri', 'kauru', 'kaury', 'kaval', 'kavas', 'kawas', 'kawau', 'kawed', 'kayak', 'kayle', 'kayos', 'kazis', 'kazoo', 'kbars', 'kcals', 'keaki', 'kebab', 'kebar', 'kebob', 'kecks', 'kedge', 'kedgy', 'keech', 'keefs', 'keeks', 'keels', 'keema', 'keeno', 'keens', 'keeps', 'keets', 'keeve', 'kefir', 'kehua', 'keirs', 'kelep', 'kelim', 'kells', 'kelly', 'kelps', 'kelpy', 'kelts', 'kelty', 'kembo', 'kembs', 'kemps', 'kempt', 'kempy', 'kenaf', 'kench', 'kendo', 'kenos', 'kente', 'kents', 'kepis', 'kerbs', 'kerel', 'kerfs', 'kerky', 'kerma', 'kerne', 'kerns', 'keros', 'kerry', 'kerve', 'kesar', 'kests', 'ketas', 'ketch', 'ketes', 'ketol', 'kevel', 'kevil', 'kexes', 'keyed', 'keyer', 'khadi', 'khads', 'khafs', 'khaki', 'khana', 'khans', 'khaph', 'khats', 'khaya', 'khazi', 'kheda', 'kheer', 'kheth', 'khets', 'khirs', 'khoja', 'khors', 'khoum', 'khuds', 'khula', 'khyal', 'kiaat', 'kiack', 'kiaki', 'kiang', 'kiasu', 'kibbe', 'kibbi', 'kibei', 'kibes', 'kibla', 'kicks', 'kicky', 'kiddo', 'kiddy', 'kidel', 'kideo', 'kidge', 'kiefs', 'kiers', 'kieve', 'kievs', 'kight', 'kikay', 'kikes', 'kikoi', 'kiley', 'kilig', 'kilim', 'kills', 'kilns', 'kilos', 'kilps', 'kilts', 'kilty', 'kimbo', 'kimet', 'kinas', 'kinda', 'kinds', 'kindy', 'kines', 'kings', 'kingy', 'kinin', 'kinks', 'kinky', 'kinos', 'kiore', 'kiosk', 'kipah', 'kipas', 'kipes', 'kippa', 'kipps', 'kipsy', 'kirby', 'kirks', 'kirns', 'kirri', 'kisan', 'kissy', 'kists', 'kitab', 'kited', 'kiter', 'kites', 'kithe', 'kiths', 'kitke', 'kitty', 'kitul', 'kivas', 'kiwis', 'klang', 'klaps', 'klett', 'klick', 'klieg', 'kliks', 'klong', 'kloof', 'kluge', 'klutz', 'knack', 'knags', 'knaps', 'knarl', 'knars', 'knaur', 'knave', 'knawe', 'knead', 'kneed', 'kneel', 'knees', 'knell', 'knelt', 'knick', 'knife', 'knish', 'knits', 'knive', 'knobs', 'knock', 'knoll', 'knoop', 'knops', 'knosp', 'knots', 'knoud', 'knout', 'knowd', 'knowe', 'known', 'knows', 'knubs', 'knule', 'knurl', 'knurr', 'knurs', 'knuts', 'koala', 'koans', 'koaps', 'koban', 'kobos', 'koels', 'koffs', 'kofta', 'kogal', 'kohas', 'kohen', 'kohls', 'koine', 'koiwi', 'kojis', 'kokam', 'kokas', 'koker', 'kokra', 'kokum', 'kolas', 'kolos', 'kombi', 'kombu', 'konbu', 'kondo', 'konks', 'kooks', 'kooky', 'koori', 'kopek', 'kophs', 'kopje', 'koppa', 'korai', 'koran', 'koras', 'korat', 'kores', 'koris', 'korma', 'koros', 'korun', 'korus', 'koses', 'kotch', 'kotos', 'kotow', 'koura', 'kraal', 'krabs', 'kraft', 'krais', 'krait', 'krang', 'krans', 'kranz', 'kraut', 'krays', 'kreef', 'kreen', 'kreep', 'kreng', 'krewe', 'krill', 'kriol', 'krona', 'krone', 'kroon', 'krubi', 'krump', 'krunk', 'ksars', 'kubie', 'kudos', 'kudus', 'kudzu', 'kufis', 'kugel', 'kuias', 'kukri', 'kukus', 'kulak', 'kulan', 'kulas', 'kulfi', 'kumis', 'kumys', 'kunas', 'kunds', 'kuris', 'kurre', 'kurta', 'kurus', 'kusso', 'kusti', 'kutai', 'kutas', 'kutch', 'kutis', 'kutus', 'kuyas', 'kuzus', 'kvass', 'kvell', 'kwaai', 'kwela', 'kwink', 'kwirl', 'kyack', 'kyaks', 'kyang', 'kyars', 'kyats', 'kybos', 'kydst', 'kyles', 'kylie', 'kylin', 'kylix', 'kyloe', 'kynde', 'kynds', 'kypes', 'kyrie', 'kytes', 'kythe', 'kyudo', 'laarf', 'laari', 'labda', 'label', 'labia', 'labis', 'labne', 'labor', 'labra', 'laccy', 'laced', 'lacer', 'laces', 'lacet', 'lacey', 'lacis', 'lacka', 'lacks', 'lacky', 'laddu', 'laddy', 'laded', 'ladee', 'laden', 'lader', 'lades', 'ladle', 'ladoo', 'laers', 'laevo', 'lagan', 'lagar', 'lager', 'laggy', 'lahal', 'lahar', 'laich', 'laics', 'laide', 'laids', 'laigh', 'laika', 'laiks', 'laird', 'lairs', 'lairy', 'laith', 'laity', 'laked', 'laker', 'lakes', 'lakhs', 'lakin', 'laksa', 'laldy', 'lalls', 'lamas', 'lambs', 'lamby', 'lamed', 'lamer', 'lames', 'lamia', 'lammy', 'lamps', 'lanai', 'lanas', 'lance', 'lanch', 'lande', 'lands', 'laned', 'lanes', 'lanks', 'lanky', 'lants', 'lapas', 'lapel', 'lapin', 'lapis', 'lapje', 'lappa', 'lappy', 'lapse', 'larch', 'lards', 'lardy', 'laree', 'lares', 'larfs', 'larga', 'large', 'largo', 'laris', 'larks', 'larky', 'larns', 'larnt', 'larum', 'larva', 'lased', 'laser', 'lases', 'lassi', 'lasso', 'lassu', 'lassy', 'lasts', 'latah', 'latch', 'lated', 'laten', 'later', 'latex', 'lathe', 'lathi', 'laths', 'lathy', 'latke', 'latte', 'latus', 'lauan', 'lauch', 'laude', 'lauds', 'laufs', 'laugh', 'laund', 'laura', 'laval', 'lavas', 'laved', 'laver', 'laves', 'lavra', 'lavvy', 'lawed', 'lawer', 'lawin', 'lawks', 'lawns', 'lawny', 'lawsy', 'laxed', 'laxer', 'laxes', 'laxly', 'layby', 'layed', 'layer', 'layin', 'layup', 'lazar', 'lazed', 'lazes', 'lazos', 'lazzi', 'lazzo', 'leach', 'leads', 'leady', 'leafs', 'leafy', 'leaks', 'leaky', 'leams', 'leans', 'leant', 'leany', 'leaps', 'leapt', 'leare', 'learn', 'lears', 'leary', 'lease', 'leash', 'least', 'leats', 'leave', 'leavy', 'leaze', 'leben', 'leccy', 'leche', 'ledes', 'ledge', 'ledgy', 'ledum', 'leear', 'leech', 'leeks', 'leeps', 'leers', 'leery', 'leese', 'leets', 'leeze', 'lefte', 'lefts', 'lefty', 'legal', 'leger', 'leges', 'legge', 'leggo', 'leggy', 'legit', 'legno', 'lehrs', 'lehua', 'leirs', 'leish', 'leman', 'lemed', 'lemel', 'lemes', 'lemma', 'lemme', 'lemon', 'lemur', 'lends', 'lenes', 'lengs', 'lenis', 'lenos', 'lense', 'lenti', 'lento', 'leone', 'lepak', 'leper', 'lepid', 'lepra', 'lepta', 'lered', 'leres', 'lerps', 'lesbo', 'leses', 'lesos', 'lests', 'letch', 'lethe', 'letty', 'letup', 'leuch', 'leuco', 'leuds', 'leugh', 'levas', 'levee', 'level', 'lever', 'leves', 'levin', 'levis', 'lewis', 'lexes', 'lexis', 'lezes', 'lezza', 'lezzo', 'lezzy', 'liana', 'liane', 'liang', 'liard', 'liars', 'liart', 'libel', 'liber', 'libor', 'libra', 'libre', 'libri', 'licet', 'lichi', 'licht', 'licit', 'licks', 'lidar', 'lidos', 'liefs', 'liege', 'liens', 'liers', 'lieus', 'lieve', 'lifer', 'lifes', 'lifey', 'lifts', 'ligan', 'liger', 'ligge', 'light', 'ligne', 'liked', 'liken', 'liker', 'likes', 'likin', 'lilac', 'lills', 'lilos', 'lilts', 'lilty', 'liman', 'limas', 'limax', 'limba', 'limbi', 'limbo', 'limbs', 'limby', 'limed', 'limen', 'limes', 'limey', 'limit', 'limma', 'limns', 'limos', 'limpa', 'limps', 'linac', 'linch', 'linds', 'lindy', 'lined', 'linen', 'liner', 'lines', 'liney', 'linga', 'lingo', 'lings', 'lingy', 'linin', 'links', 'linky', 'linns', 'linny', 'linos', 'lints', 'linty', 'linum', 'linux', 'lions', 'lipas', 'lipes', 'lipid', 'lipin', 'lipos', 'lippy', 'liras', 'lirks', 'lirot', 'lises', 'lisks', 'lisle', 'lisps', 'lists', 'litai', 'litas', 'lited', 'litem', 'liter', 'lites', 'lithe', 'litho', 'liths', 'litie', 'litre', 'lived', 'liven', 'liver', 'lives', 'livid', 'livor', 'livre', 'liwaa', 'liwas', 'llama', 'llano', 'loach', 'loads', 'loafs', 'loams', 'loamy', 'loans', 'loast', 'loath', 'loave', 'lobar', 'lobby', 'lobed', 'lobes', 'lobos', 'lobus', 'local', 'loche', 'lochs', 'lochy', 'locie', 'locis', 'locks', 'locky', 'locos', 'locum', 'locus', 'loden', 'lodes', 'lodge', 'loess', 'lofts', 'lofty', 'logan', 'loges', 'loggy', 'logia', 'logic', 'logie', 'login', 'logoi', 'logon', 'logos', 'lohan', 'loids', 'loins', 'loipe', 'loirs', 'lokes', 'lokey', 'lokum', 'lolas', 'loled', 'lollo', 'lolls', 'lolly', 'lolog', 'lolos', 'lomas', 'lomed', 'lomes', 'loner', 'longa', 'longe', 'longs', 'looby', 'looed', 'looey', 'loofa', 'loofs', 'looie', 'looks', 'looky', 'looms', 'loons', 'loony', 'loops', 'loopy', 'loord', 'loose', 'loots', 'loped', 'loper', 'lopes', 'loppy', 'loral', 'loran', 'lords', 'lordy', 'lorel', 'lores', 'loric', 'loris', 'lorry', 'losed', 'losel', 'losen', 'loser', 'loses', 'lossy', 'lotah', 'lotas', 'lotes', 'lotic', 'lotos', 'lotsa', 'lotta', 'lotte', 'lotto', 'lotus', 'loued', 'lough', 'louie', 'louis', 'louma', 'lound', 'louns', 'loupe', 'loups', 'loure', 'lours', 'loury', 'louse', 'lousy', 'louts', 'lovat', 'loved', 'lovee', 'lover', 'loves', 'lovey', 'lovie', 'lowan', 'lowed', 'lowen', 'lower', 'lowes', 'lowly', 'lownd', 'lowne', 'lowns', 'lowps', 'lowry', 'lowse', 'lowth', 'lowts', 'loxed', 'loxes', 'loyal', 'lozen', 'luach', 'luaus', 'lubed', 'lubes', 'lubra', 'luces', 'lucid', 'lucks', 'lucky', 'lucre', 'ludes', 'ludic', 'ludos', 'luffa', 'luffs', 'luged', 'luger', 'luges', 'lulls', 'lulus', 'lumas', 'lumbi', 'lumen', 'lumme', 'lummy', 'lumps', 'lumpy', 'lunar', 'lunas', 'lunch', 'lunes', 'lunet', 'lunge', 'lungi', 'lungs', 'lunks', 'lunts', 'lupin', 'lupus', 'lurch', 'lured', 'lurer', 'lures', 'lurex', 'lurgi', 'lurgy', 'lurid', 'lurks', 'lurry', 'lurve', 'luser', 'lushy', 'lusks', 'lusts', 'lusty', 'lusus', 'lutea', 'luted', 'luter', 'lutes', 'luvvy', 'luxed', 'luxer', 'luxes', 'lweis', 'lyams', 'lyard', 'lyart', 'lyase', 'lycea', 'lycee', 'lycra', 'lying', 'lymes', 'lymph', 'lynch', 'lynes', 'lyres', 'lyric', 'lysed', 'lyses', 'lysin', 'lysis', 'lysol', 'lyssa', 'lyted', 'lytes', 'lythe', 'lytic', 'lytta', 'maaed', 'maare', 'maars', 'maban', 'mabes', 'macas', 'macaw', 'macca', 'maced', 'macer', 'maces', 'mache', 'machi', 'macho', 'machs', 'macka', 'macks', 'macle', 'macon', 'macro', 'macte', 'madal', 'madam', 'madar', 'maddy', 'madge', 'madid', 'madly', 'mados', 'madre', 'maedi', 'maerl', 'mafia', 'mafic', 'mafts', 'magas', 'mages', 'maggs', 'magic', 'magma', 'magna', 'magot', 'magus', 'mahal', 'mahem', 'mahis', 'mahoe', 'mahrs', 'mahua', 'mahwa', 'maids', 'maiko', 'maiks', 'maile', 'maill', 'mailo', 'mails', 'maims', 'mains', 'maire', 'mairs', 'maise', 'maist', 'maize', 'majas', 'majat', 'majoe', 'major', 'majos', 'makaf', 'makai', 'makan', 'makar', 'makee', 'maker', 'makes', 'makie', 'makis', 'makos', 'malae', 'malai', 'malam', 'malar', 'malas', 'malax', 'maleo', 'males', 'malic', 'malik', 'malis', 'malky', 'malls', 'malms', 'malmy', 'malts', 'malty', 'malus', 'malva', 'malwa', 'mamak', 'mamas', 'mamba', 'mambo', 'mambu', 'mamee', 'mamey', 'mamie', 'mamil', 'mamma', 'mammy', 'manas', 'manat', 'mandi', 'mands', 'mandy', 'maneb', 'maned', 'maneh', 'manes', 'manet', 'manga', 'mange', 'mangi', 'mango', 'mangs', 'mangy', 'mania', 'manic', 'manie', 'manis', 'manks', 'manky', 'manly', 'manna', 'manny', 'manoa', 'manor', 'manos', 'manse', 'manso', 'manta', 'mante', 'manto', 'mants', 'manty', 'manul', 'manus', 'manzo', 'mapau', 'mapes', 'maple', 'mapou', 'mappy', 'maqam', 'maqui', 'marae', 'marah', 'maral', 'maran', 'maras', 'maray', 'march', 'marcs', 'mards', 'mardy', 'mares', 'marga', 'marge', 'margo', 'margs', 'maria', 'marid', 'maril', 'marka', 'marks', 'marle', 'marls', 'marly', 'marma', 'marms', 'maron', 'maror', 'marra', 'marri', 'marry', 'marse', 'marsh', 'marts', 'marua', 'marvy', 'masas', 'mased', 'maser', 'mases', 'masha', 'mashy', 'masks', 'mason', 'massa', 'masse', 'massy', 'masts', 'masty', 'masur', 'masus', 'masut', 'matai', 'match', 'mated', 'mater', 'mates', 'matey', 'mathe', 'maths', 'matin', 'matlo', 'matra', 'matsu', 'matte', 'matts', 'matty', 'matza', 'matzo', 'mauby', 'mauds', 'mauka', 'maula', 'mauls', 'maums', 'maumy', 'maund', 'maunt', 'mauri', 'mausy', 'mauts', 'mauve', 'mauvy', 'mauzy', 'maven', 'mavie', 'mavin', 'mavis', 'mawed', 'mawks', 'mawky', 'mawla', 'mawns', 'mawps', 'mawrs', 'maxed', 'maxes', 'maxim', 'maxis', 'mayan', 'mayas', 'maybe', 'mayed', 'mayor', 'mayos', 'mayst', 'mazac', 'mazak', 'mazar', 'mazas', 'mazed', 'mazel', 'mazer', 'mazes', 'mazet', 'mazey', 'mazut', 'mbari', 'mbars', 'mbila', 'mbira', 'mbret', 'mbube', 'mbuga', 'meads', 'meake', 'meaks', 'meals', 'mealy', 'meane', 'means', 'meant', 'meany', 'meare', 'mease', 'meath', 'meats', 'meaty', 'mebbe', 'mebos', 'mecca', 'mecha', 'mechs', 'mecks', 'mecum', 'medal', 'media', 'medic', 'medii', 'medin', 'medle', 'meech', 'meeds', 'meeja', 'meeps', 'meers', 'meets', 'meffs', 'meids', 'meiko', 'meils', 'meins', 'meint', 'meiny', 'meism', 'meith', 'mekka', 'melam', 'melas', 'melba', 'melch', 'melds', 'melee', 'meles', 'melic', 'melik', 'mells', 'meloe', 'melon', 'melos', 'melts', 'melty', 'memes', 'memic', 'memos', 'menad', 'mence', 'mends', 'mened', 'menes', 'menge', 'mengs', 'menil', 'mensa', 'mense', 'mensh', 'menta', 'mento', 'ments', 'menus', 'meous', 'meows', 'merch', 'mercs', 'mercy', 'merde', 'merds', 'mered', 'merel', 'merer', 'meres', 'merge', 'meril', 'meris', 'merit', 'merks', 'merle', 'merls', 'merry', 'merse', 'mersk', 'mesad', 'mesal', 'mesas', 'mesca', 'mesel', 'mesem', 'meses', 'meshy', 'mesia', 'mesic', 'mesne', 'meson', 'messy', 'mesto', 'mesyl', 'metal', 'metas', 'meted', 'meteg', 'metel', 'meter', 'metes', 'methi', 'metho', 'meths', 'methy', 'metic', 'metif', 'metis', 'metol', 'metre', 'metro', 'metta', 'meums', 'meuse', 'meved', 'meves', 'mewed', 'mewls', 'meynt', 'mezes', 'mezza', 'mezze', 'mezzo', 'mgals', 'mhorr', 'miais', 'miaou', 'miaow', 'miasm', 'miaul', 'micas', 'miche', 'michi', 'micht', 'micks', 'micky', 'micos', 'micra', 'micro', 'middy', 'midge', 'midgy', 'midis', 'midst', 'miens', 'mieux', 'mieve', 'miffs', 'miffy', 'mifty', 'miggs', 'might', 'migma', 'migod', 'mihas', 'mihis', 'mikan', 'miked', 'mikes', 'mikos', 'mikra', 'mikva', 'milch', 'milds', 'miler', 'miles', 'milfs', 'milia', 'milko', 'milks', 'milky', 'mille', 'mills', 'milly', 'milor', 'milos', 'milpa', 'milts', 'milty', 'miltz', 'mimed', 'mimeo', 'mimer', 'mimes', 'mimic', 'mimis', 'mimsy', 'minae', 'minar', 'minas', 'mince', 'mincy', 'mindi', 'minds', 'mined', 'miner', 'mines', 'minge', 'mingi', 'mings', 'mingy', 'minim', 'minis', 'minke', 'minks', 'minny', 'minor', 'minos', 'minse', 'mints', 'minty', 'minus', 'minxy', 'miraa', 'mirah', 'mirch', 'mired', 'mires', 'mirex', 'mirid', 'mirin', 'mirkn', 'mirks', 'mirky', 'mirls', 'mirly', 'miros', 'mirrl', 'mirrs', 'mirth', 'mirvs', 'mirza', 'misal', 'misch', 'misdo', 'miser', 'mises', 'misgo', 'misky', 'misls', 'misos', 'missa', 'missy', 'misto', 'mists', 'misty', 'mitas', 'mitch', 'miter', 'mites', 'mitey', 'mitie', 'mitis', 'mitre', 'mitry', 'mitta', 'mitts', 'mivey', 'mivvy', 'mixed', 'mixen', 'mixer', 'mixes', 'mixie', 'mixis', 'mixte', 'mixup', 'miyas', 'mizen', 'mizes', 'mizzy', 'mmkay', 'mneme', 'moais', 'moaky', 'moals', 'moana', 'moans', 'moany', 'moars', 'moats', 'mobby', 'mobed', 'mobee', 'mobes', 'mobey', 'mobie', 'moble', 'mobos', 'mocap', 'mocha', 'mochi', 'mochs', 'mochy', 'mocks', 'mocky', 'mocos', 'mocus', 'modal', 'model', 'modem', 'moder', 'modes', 'modge', 'modii', 'modin', 'modoc', 'modom', 'modus', 'moeni', 'moers', 'mofos', 'mogar', 'mogas', 'moggy', 'mogos', 'mogra', 'mogue', 'mogul', 'mohar', 'mohel', 'mohos', 'mohrs', 'mohua', 'mohur', 'moile', 'moils', 'moira', 'moire', 'moist', 'moits', 'moity', 'mojos', 'moker', 'mokes', 'mokey', 'mokis', 'mokky', 'mokos', 'mokus', 'molal', 'molar', 'molas', 'molds', 'moldy', 'moled', 'moler', 'moles', 'moley', 'molie', 'molla', 'molle', 'mollo', 'molls', 'molly', 'moloi', 'molos', 'molto', 'molts', 'molue', 'molvi', 'molys', 'momes', 'momie', 'momma', 'momme', 'mommy', 'momos', 'mompe', 'momus', 'monad', 'monal', 'monas', 'monde', 'mondo', 'moner', 'money', 'mongo', 'mongs', 'monic', 'monie', 'monks', 'monos', 'monpe', 'monte', 'month', 'monty', 'moobs', 'mooch', 'moods', 'moody', 'mooed', 'mooey', 'mooks', 'moola', 'mooli', 'mools', 'mooly', 'moong', 'mooni', 'moons', 'moony', 'moops', 'moors', 'moory', 'moose', 'mooth', 'moots', 'moove', 'moped', 'moper', 'mopes', 'mopey', 'moppy', 'mopsy', 'mopus', 'morae', 'morah', 'moral', 'moran', 'moras', 'morat', 'moray', 'moree', 'morel', 'mores', 'morgy', 'moria', 'morin', 'mormo', 'morna', 'morne', 'morns', 'moron', 'moror', 'morph', 'morra', 'morro', 'morse', 'morts', 'moruk', 'mosed', 'moses', 'mosey', 'mosks', 'mosso', 'mossy', 'moste', 'mosto', 'mosts', 'moted', 'motel', 'moten', 'motes', 'motet', 'motey', 'moths', 'mothy', 'motif', 'motis', 'moton', 'motor', 'motte', 'motto', 'motts', 'motty', 'motus', 'motza', 'mouch', 'moues', 'moufs', 'mould', 'moule', 'mouls', 'moult', 'mouly', 'mound', 'mount', 'moups', 'mourn', 'mouse', 'moust', 'mousy', 'mouth', 'moved', 'mover', 'moves', 'movie', 'mowas', 'mowed', 'mower', 'mowie', 'mowra', 'moxas', 'moxie', 'moyas', 'moyle', 'moyls', 'mozed', 'mozes', 'mozos', 'mpret', 'mrads', 'msasa', 'mtepe', 'mucho', 'mucic', 'mucid', 'mucin', 'mucko', 'mucks', 'mucky', 'mucor', 'mucro', 'mucus', 'mudar', 'muddy', 'mudge', 'mudif', 'mudim', 'mudir', 'mudra', 'muffs', 'muffy', 'mufti', 'mugga', 'muggs', 'muggy', 'mugho', 'mugil', 'mugos', 'muhly', 'muids', 'muils', 'muirs', 'muiry', 'muist', 'mujik', 'mukim', 'mukti', 'mulai', 'mulch', 'mulct', 'muled', 'mules', 'muley', 'mulga', 'mulie', 'mulla', 'mulls', 'mulse', 'mulsh', 'mumbo', 'mumms', 'mummy', 'mumph', 'mumps', 'mumsy', 'mumus', 'munch', 'munds', 'mundu', 'munga', 'munge', 'mungi', 'mungo', 'mungs', 'mungy', 'munia', 'munis', 'munja', 'munjs', 'munts', 'muntu', 'muons', 'mural', 'muras', 'mured', 'mures', 'murex', 'murgh', 'murgi', 'murid', 'murks', 'murky', 'murls', 'murly', 'murra', 'murre', 'murri', 'murrs', 'murry', 'murth', 'murti', 'muruk', 'murva', 'musar', 'musca', 'mused', 'musee', 'muser', 'muses', 'muset', 'musha', 'mushy', 'music', 'musit', 'musks', 'musky', 'musos', 'musse', 'mussy', 'musta', 'musth', 'musts', 'musty', 'mutas', 'mutch', 'muted', 'muter', 'mutes', 'mutha', 'mutic', 'mutis', 'muton', 'mutti', 'mutts', 'mutum', 'muvva', 'muxed', 'muxes', 'muzak', 'muzzy', 'mvula', 'mvule', 'mvuli', 'myall', 'myals', 'mylar', 'mynah', 'mynas', 'myoid', 'myoma', 'myons', 'myope', 'myops', 'myopy', 'myrrh', 'mysid', 'mysie', 'mythi', 'myths', 'mythy', 'myxos', 'mzees', 'naams', 'naans', 'naats', 'nabam', 'nabby', 'nabes', 'nabis', 'nabks', 'nabla', 'nabob', 'nache', 'nacho', 'nacre', 'nadas', 'nadir', 'naeve', 'naevi', 'naffs', 'nagar', 'nagas', 'nages', 'naggy', 'nagor', 'nahal', 'naiad', 'naibs', 'naice', 'naids', 'naieo', 'naifs', 'naiks', 'nails', 'naily', 'nains', 'naios', 'naira', 'nairu', 'naive', 'najib', 'nakas', 'naked', 'naker', 'nakfa', 'nalas', 'naled', 'nalla', 'namad', 'namak', 'namaz', 'named', 'namer', 'names', 'namma', 'namus', 'nanas', 'nance', 'nancy', 'nandu', 'nanna', 'nanny', 'nanos', 'nante', 'nanti', 'nanto', 'nants', 'nanty', 'nanua', 'napas', 'naped', 'napes', 'napoh', 'napoo', 'nappa', 'nappe', 'nappy', 'naras', 'narco', 'narcs', 'nards', 'nares', 'naric', 'naris', 'narks', 'narky', 'narod', 'narra', 'narre', 'nasal', 'nashi', 'nasho', 'nasis', 'nason', 'nasty', 'nasus', 'natak', 'natal', 'natch', 'nates', 'natis', 'natto', 'natty', 'natya', 'nauch', 'naunt', 'naval', 'navar', 'naved', 'navel', 'naves', 'navew', 'navvy', 'nawab', 'nawal', 'nazar', 'nazes', 'nazir', 'nazis', 'nazzy', 'nduja', 'neafe', 'neals', 'neant', 'neaps', 'nears', 'neath', 'neato', 'neats', 'nebby', 'nebek', 'nebel', 'neche', 'necks', 'neddy', 'neebs', 'needs', 'needy', 'neefs', 'neeld', 'neele', 'neemb', 'neems', 'neeps', 'neese', 'neeze', 'nefie', 'negri', 'negro', 'negus', 'neifs', 'neigh', 'neist', 'neive', 'nelia', 'nelis', 'nelly', 'nemas', 'nemic', 'nemns', 'nempt', 'nenes', 'nenta', 'neons', 'neosa', 'neoza', 'neper', 'nepit', 'neral', 'neram', 'nerds', 'nerdy', 'nerfs', 'nerka', 'nerks', 'nerol', 'nerts', 'nertz', 'nerve', 'nervy', 'neski', 'nests', 'nesty', 'netas', 'netes', 'netop', 'netta', 'netts', 'netty', 'neuks', 'neume', 'neums', 'nevel', 'never', 'neves', 'nevis', 'nevus', 'nevvy', 'newbs', 'newed', 'newel', 'newer', 'newie', 'newly', 'newsy', 'newts', 'nexal', 'nexin', 'nexts', 'nexum', 'nexus', 'ngaio', 'ngaka', 'ngana', 'ngapi', 'ngati', 'ngege', 'ngoma', 'ngoni', 'ngram', 'ngwee', 'nibby', 'nicad', 'niced', 'nicer', 'nicey', 'niche', 'nicht', 'nicks', 'nicky', 'nicol', 'nidal', 'nided', 'nides', 'nidor', 'nidus', 'niece', 'niefs', 'niess', 'nieve', 'nifes', 'niffs', 'niffy', 'nifle', 'nifty', 'niger', 'nigga', 'nighs', 'night', 'nigre', 'nigua', 'nihil', 'nikab', 'nikah', 'nikau', 'nilas', 'nills', 'nimbi', 'nimbs', 'nimby', 'nimps', 'niner', 'nines', 'ninja', 'ninny', 'ninon', 'ninta', 'ninth', 'niopo', 'nioza', 'nipas', 'nipet', 'nippy', 'niqab', 'nirls', 'nirly', 'nisei', 'nisin', 'nisse', 'nisus', 'nital', 'niter', 'nites', 'nitid', 'niton', 'nitre', 'nitro', 'nitry', 'nitta', 'nitto', 'nitty', 'nival', 'nivas', 'nivel', 'nixed', 'nixer', 'nixes', 'nixie', 'nizam', 'njirl', 'nkosi', 'nmoli', 'nmols', 'noahs', 'nobby', 'noble', 'nobly', 'nocks', 'nodal', 'noddy', 'noded', 'nodes', 'nodum', 'nodus', 'noels', 'noema', 'noeme', 'nogal', 'noggs', 'noggy', 'nohow', 'noias', 'noils', 'noily', 'noint', 'noire', 'noirs', 'noise', 'noisy', 'nokes', 'noles', 'nolle', 'nolls', 'nolos', 'nomad', 'nomas', 'nomen', 'nomes', 'nomic', 'nomoi', 'nomos', 'nonan', 'nonas', 'nonce', 'noncy', 'nonda', 'nondo', 'nones', 'nonet', 'nongs', 'nonic', 'nonis', 'nonna', 'nonno', 'nonny', 'nonyl', 'noobs', 'noois', 'nooit', 'nooks', 'nooky', 'noone', 'noons', 'noops', 'noose', 'noove', 'nopal', 'noria', 'norie', 'noris', 'norks', 'norma', 'norms', 'north', 'nosed', 'noser', 'noses', 'nosey', 'noshi', 'nosir', 'notal', 'notam', 'notch', 'noted', 'noter', 'notes', 'notum', 'nougs', 'nouja', 'nould', 'noule', 'nouls', 'nouns', 'nouny', 'noups', 'noust', 'novae', 'novas', 'novel', 'novia', 'novio', 'novum', 'noway', 'nowds', 'nowed', 'nowls', 'nowts', 'nowty', 'noxal', 'noxas', 'noxes', 'noyau', 'noyed', 'noyes', 'nrtta', 'nrtya', 'nsima', 'nubby', 'nubia', 'nucha', 'nucin', 'nuddy', 'nuder', 'nudes', 'nudge', 'nudgy', 'nudie', 'nudzh', 'nuevo', 'nuffs', 'nugae', 'nujol', 'nuked', 'nukes', 'nulla', 'nullo', 'nulls', 'nully', 'numbs', 'numen', 'nummy', 'numps', 'nunks', 'nunky', 'nunny', 'nunus', 'nuque', 'nurds', 'nurdy', 'nurls', 'nurrs', 'nurse', 'nurts', 'nurtz', 'nused', 'nuses', 'nutso', 'nutsy', 'nutty', 'nyaff', 'nyala', 'nyams', 'nying', 'nylon', 'nymph', 'nyong', 'nyssa', 'nyung', 'nyuse', 'nyuze', 'oafos', 'oaked', 'oaken', 'oaker', 'oakum', 'oared', 'oarer', 'oasal', 'oases', 'oasis', 'oasts', 'oaten', 'oater', 'oaths', 'oaves', 'obang', 'obbos', 'obeah', 'obeli', 'obese', 'obeys', 'obias', 'obied', 'obiit', 'obits', 'objet', 'oboes', 'obole', 'oboli', 'obols', 'occam', 'occur', 'ocean', 'ocher', 'oches', 'ochre', 'ochry', 'ocker', 'ocote', 'ocrea', 'octad', 'octal', 'octan', 'octas', 'octet', 'octic', 'octli', 'octyl', 'oculi', 'odahs', 'odals', 'odder', 'oddly', 'odeon', 'odeum', 'odism', 'odist', 'odium', 'odoom', 'odors', 'odour', 'odums', 'odyle', 'odyls', 'ofays', 'offal', 'offed', 'offer', 'offie', 'oflag', 'often', 'ofter', 'ofuro', 'ogams', 'ogeed', 'ogees', 'oggin', 'ogham', 'ogive', 'ogled', 'ogler', 'ogles', 'ogmic', 'ogres', 'ohelo', 'ohias', 'ohing', 'ohmic', 'ohone', 'oicks', 'oidia', 'oiled', 'oiler', 'oilet', 'oinks', 'oints', 'oiran', 'ojime', 'okapi', 'okays', 'okehs', 'okies', 'oking', 'okole', 'okras', 'okrug', 'oktas', 'olate', 'olden', 'older', 'oldie', 'oldly', 'olehs', 'oleic', 'olein', 'olent', 'oleos', 'oleum', 'oleyl', 'oligo', 'olios', 'oliva', 'olive', 'ollas', 'ollav', 'oller', 'ollie', 'ology', 'olona', 'olpae', 'olpes', 'omasa', 'omber', 'ombre', 'ombus', 'omdah', 'omdas', 'omdda', 'omdeh', 'omees', 'omega', 'omens', 'omers', 'omiai', 'omits', 'omlah', 'ommel', 'ommin', 'omnes', 'omovs', 'omrah', 'omuls', 'oncer', 'onces', 'oncet', 'oncus', 'ondes', 'ondol', 'onely', 'oners', 'onery', 'ongon', 'onion', 'onium', 'onkus', 'onlap', 'onlay', 'onmun', 'onned', 'onsen', 'onset', 'ontal', 'ontic', 'ooaas', 'oobit', 'oohed', 'ooids', 'oojah', 'oomph', 'oonts', 'oopak', 'ooped', 'oopsy', 'oorie', 'ooses', 'ootid', 'ooyah', 'oozed', 'oozes', 'oozie', 'oozle', 'opahs', 'opals', 'opens', 'opepe', 'opera', 'opery', 'opgaf', 'opihi', 'opine', 'oping', 'opium', 'oppos', 'opsat', 'opsin', 'opsit', 'opted', 'opter', 'optic', 'opzit', 'orach', 'oracy', 'orals', 'orang', 'orans', 'orant', 'orate', 'orbat', 'orbed', 'orbic', 'orbit', 'orcas', 'orcin', 'order', 'ordie', 'ordos', 'oread', 'orfes', 'orful', 'organ', 'orgia', 'orgic', 'orgue', 'oribi', 'oriel', 'origo', 'orixa', 'orles', 'orlon', 'orlop', 'ormer', 'ornee', 'ornis', 'orped', 'orpin', 'orris', 'ortet', 'ortho', 'orval', 'orzos', 'osars', 'oscar', 'osetr', 'oseys', 'oshac', 'osier', 'oskin', 'oslin', 'osmic', 'osmol', 'osone', 'ossia', 'ostia', 'otaku', 'otary', 'other', 'othyl', 'otium', 'ottar', 'otter', 'ottos', 'oubit', 'ouche', 'oucht', 'oueds', 'ouens', 'ought', 'ouija', 'oulks', 'oumas', 'ounce', 'oundy', 'oupas', 'ouped', 'ouphe', 'ouphs', 'ourey', 'ourie', 'ousel', 'ousia', 'ousts', 'outby', 'outdo', 'outed', 'outen', 'outer', 'outgo', 'outie', 'outre', 'outro', 'outta', 'ouzel', 'ouzos', 'ovals', 'ovary', 'ovate', 'ovels', 'ovens', 'overs', 'overt', 'ovine', 'ovism', 'ovist', 'ovoid', 'ovoli', 'ovolo', 'ovule', 'oware', 'owari', 'owche', 'owers', 'owies', 'owing', 'owled', 'owler', 'owlet', 'owned', 'owner', 'ownio', 'owres', 'owrie', 'owsen', 'oxbow', 'oxeas', 'oxers', 'oxeye', 'oxide', 'oxids', 'oxies', 'oxime', 'oxims', 'oxine', 'oxlip', 'oxman', 'oxmen', 'oxter', 'oyama', 'oyers', 'ozeki', 'ozena', 'ozone', 'ozzie', 'paaho', 'paals', 'paans', 'pacai', 'pacas', 'pacay', 'paced', 'pacer', 'paces', 'pacey', 'pacha', 'packs', 'packy', 'pacos', 'pacta', 'pacts', 'padam', 'padas', 'paddo', 'paddy', 'padis', 'padle', 'padma', 'padou', 'padre', 'padri', 'paean', 'paedo', 'paeon', 'pagan', 'paged', 'pager', 'pages', 'pagle', 'pagne', 'pagod', 'pagri', 'pahit', 'pahos', 'pahus', 'paiks', 'pails', 'pains', 'paint', 'paipe', 'paips', 'paire', 'pairs', 'paisa', 'paise', 'pakay', 'pakka', 'pakki', 'pakua', 'pakul', 'palak', 'palar', 'palas', 'palay', 'palea', 'paled', 'paler', 'pales', 'palet', 'palis', 'palki', 'palla', 'palls', 'pallu', 'pally', 'palms', 'palmy', 'palpi', 'palps', 'palsa', 'palsy', 'palus', 'pamby', 'pampa', 'panax', 'pance', 'panch', 'panda', 'pands', 'pandy', 'paned', 'panel', 'panes', 'panga', 'pangs', 'panic', 'panim', 'panir', 'panko', 'panks', 'panna', 'panne', 'panni', 'panny', 'pansy', 'panto', 'pants', 'panty', 'paoli', 'paolo', 'papad', 'papal', 'papas', 'papaw', 'paper', 'papes', 'papey', 'pappi', 'pappy', 'papri', 'parae', 'paras', 'parch', 'parcs', 'pardi', 'pards', 'pardy', 'pared', 'paren', 'pareo', 'parer', 'pares', 'pareu', 'parev', 'parge', 'pargo', 'parid', 'paris', 'parka', 'parki', 'parks', 'parky', 'parle', 'parly', 'parma', 'parmo', 'parms', 'parol', 'parps', 'parra', 'parrs', 'parry', 'parse', 'parte', 'parti', 'parts', 'party', 'parve', 'parvo', 'pasag', 'pasar', 'pasch', 'paseo', 'pases', 'pasha', 'pashm', 'paska', 'pasmo', 'paspy', 'passe', 'passu', 'pasta', 'paste', 'pasts', 'pasty', 'patas', 'patch', 'pated', 'patee', 'patel', 'paten', 'pater', 'pates', 'paths', 'patia', 'patin', 'patio', 'patka', 'patly', 'patsy', 'patta', 'patte', 'pattu', 'patty', 'patus', 'pauas', 'pauls', 'pause', 'pauxi', 'pavan', 'pavas', 'paved', 'paven', 'paver', 'paves', 'pavid', 'pavie', 'pavin', 'pavis', 'pavon', 'pavvy', 'pawas', 'pawaw', 'pawed', 'pawer', 'pawks', 'pawky', 'pawls', 'pawns', 'paxes', 'payed', 'payee', 'payer', 'payor', 'paysd', 'peace', 'peach', 'peage', 'peags', 'peake', 'peaks', 'peaky', 'peals', 'peans', 'peare', 'pearl', 'pears', 'peart', 'pease', 'peasy', 'peats', 'peaty', 'peavy', 'peaze', 'pebas', 'pecan', 'pechs', 'pecia', 'pecke', 'pecks', 'pecky', 'pects', 'pedal', 'pedes', 'pedis', 'pedon', 'pedos', 'pedro', 'peece', 'peeks', 'peeky', 'peels', 'peely', 'peens', 'peent', 'peeoy', 'peepe', 'peeps', 'peepy', 'peers', 'peery', 'peeve', 'peevo', 'peggy', 'peghs', 'pegma', 'pegos', 'peine', 'peins', 'peise', 'peisy', 'peize', 'pekan', 'pekau', 'pekea', 'pekes', 'pekid', 'pekin', 'pekoe', 'pelas', 'pelau', 'pelch', 'peles', 'pelfs', 'pells', 'pelma', 'pelog', 'pelon', 'pelsh', 'pelta', 'pelts', 'pelus', 'penal', 'pence', 'pends', 'pendu', 'pened', 'penes', 'pengo', 'penie', 'penis', 'penks', 'penna', 'penne', 'penni', 'penny', 'pense', 'pensy', 'pents', 'peola', 'peons', 'peony', 'pepla', 'peple', 'pepon', 'pepos', 'peppy', 'pepsi', 'pequi', 'perae', 'perai', 'perce', 'perch', 'percs', 'perdu', 'perdy', 'perea', 'peres', 'perfs', 'peril', 'peris', 'perks', 'perky', 'perle', 'perls', 'perms', 'permy', 'perne', 'perns', 'perog', 'perps', 'perry', 'perse', 'persp', 'perst', 'perts', 'perve', 'pervo', 'pervs', 'pervy', 'pesch', 'pesky', 'pesos', 'pesta', 'pesto', 'pests', 'pesty', 'petal', 'petar', 'peter', 'petit', 'petos', 'petre', 'petri', 'petti', 'petto', 'petty', 'pewed', 'pewee', 'pewit', 'peyse', 'pfftt', 'phage', 'phang', 'phare', 'pharm', 'phase', 'phasm', 'pheer', 'pheme', 'phene', 'pheon', 'phese', 'phial', 'phies', 'phish', 'phizz', 'phlox', 'phobe', 'phoca', 'phone', 'phono', 'phons', 'phony', 'phooh', 'phooo', 'phota', 'photo', 'phots', 'photy', 'phpht', 'phubs', 'phuts', 'phutu', 'phwat', 'phyla', 'phyle', 'phyma', 'phynx', 'physa', 'piais', 'piani', 'piano', 'pians', 'pibal', 'pical', 'picas', 'piccy', 'picey', 'pichi', 'picks', 'picky', 'picon', 'picot', 'picra', 'picul', 'piece', 'pieds', 'piend', 'piers', 'piert', 'pieta', 'piets', 'piety', 'piezo', 'piggy', 'pight', 'pigly', 'pigmy', 'piing', 'pikas', 'pikau', 'piked', 'pikel', 'piker', 'pikes', 'pikey', 'pikis', 'pikul', 'pilae', 'pilaf', 'pilao', 'pilar', 'pilau', 'pilaw', 'pilch', 'pilea', 'piled', 'pilei', 'piler', 'piles', 'piley', 'pilin', 'pilis', 'pills', 'pilon', 'pilot', 'pilow', 'pilum', 'pilus', 'pimas', 'pimps', 'pinas', 'pinax', 'pince', 'pinch', 'pinda', 'pinds', 'pined', 'piner', 'pines', 'piney', 'pinga', 'pinge', 'pingo', 'pings', 'pinko', 'pinks', 'pinky', 'pinna', 'pinny', 'pinol', 'pinon', 'pinot', 'pinta', 'pinto', 'pints', 'pinup', 'pions', 'piony', 'pious', 'pioye', 'pioys', 'pipal', 'pipas', 'piped', 'piper', 'pipes', 'pipet', 'pipid', 'pipis', 'pipit', 'pippy', 'pipul', 'pique', 'piqui', 'pirai', 'pirks', 'pirls', 'pirns', 'pirog', 'pirre', 'pirri', 'pirrs', 'pisco', 'pises', 'pisky', 'pisos', 'pissy', 'piste', 'pitas', 'pitch', 'piths', 'pithy', 'piton', 'pitot', 'pitso', 'pitsu', 'pitta', 'pittu', 'piuma', 'piums', 'pivos', 'pivot', 'pixel', 'pixes', 'pixie', 'piyut', 'pized', 'pizer', 'pizes', 'pizza', 'plaas', 'place', 'plack', 'plaga', 'plage', 'plaid', 'plaig', 'plain', 'plait', 'planc', 'plane', 'planh', 'plank', 'plans', 'plant', 'plaps', 'plash', 'plasm', 'plast', 'plate', 'plats', 'platt', 'platy', 'plaud', 'plaur', 'plavs', 'playa', 'plays', 'plaza', 'plead', 'pleas', 'pleat', 'plebe', 'plebs', 'pleck', 'pleep', 'plein', 'plena', 'plene', 'pleno', 'pleon', 'plesh', 'plets', 'plews', 'plexi', 'plica', 'plied', 'plier', 'plies', 'pligs', 'plims', 'pling', 'plink', 'plips', 'plish', 'ploat', 'ploce', 'plock', 'plods', 'ploit', 'plomb', 'plong', 'plonk', 'plook', 'ploot', 'plops', 'plore', 'plots', 'plotz', 'plouk', 'plout', 'plows', 'plowt', 'ploye', 'ploys', 'pluck', 'pluds', 'plues', 'pluff', 'plugs', 'pluke', 'plumb', 'plume', 'plump', 'plums', 'plumy', 'plung', 'plunk', 'pluot', 'plups', 'plush', 'plute', 'pluto', 'pluty', 'plyer', 'pneus', 'poach', 'poaka', 'poake', 'poalo', 'pobby', 'poboy', 'pocan', 'poche', 'pocho', 'pocks', 'pocky', 'podal', 'poddy', 'podex', 'podge', 'podgy', 'podia', 'podos', 'podus', 'poems', 'poena', 'poeps', 'poesy', 'poete', 'poets', 'pogey', 'pogge', 'poggy', 'pogos', 'pogue', 'pohed', 'poilu', 'poind', 'point', 'poire', 'poise', 'pokal', 'poked', 'poker', 'pokes', 'pokey', 'pokie', 'pokit', 'polar', 'poled', 'poler', 'poles', 'poley', 'polio', 'polis', 'polje', 'polka', 'polks', 'pollo', 'polls', 'polly', 'polos', 'polts', 'polyp', 'polys', 'pomas', 'pombe', 'pomes', 'pomme', 'pommy', 'pomos', 'pompa', 'pomps', 'ponce', 'poncy', 'ponds', 'pondy', 'pones', 'poney', 'ponga', 'pongo', 'pongs', 'pongy', 'ponks', 'ponor', 'ponto', 'ponts', 'ponty', 'ponzu', 'pooay', 'pooch', 'poods', 'pooed', 'pooey', 'poofs', 'poofy', 'poohs', 'poohy', 'pooja', 'pooka', 'pooks', 'pools', 'pooly', 'poons', 'poopa', 'poops', 'poopy', 'poori', 'poort', 'poots', 'pooty', 'poove', 'poovy', 'popes', 'popia', 'popos', 'poppa', 'poppy', 'popsy', 'popup', 'porae', 'poral', 'porch', 'pored', 'porer', 'pores', 'porey', 'porge', 'porgy', 'porin', 'porks', 'porky', 'porno', 'porns', 'porny', 'porta', 'porte', 'porth', 'ports', 'porty', 'porus', 'posca', 'posed', 'poser', 'poses', 'poset', 'posey', 'posho', 'posit', 'posol', 'posse', 'poste', 'posts', 'potae', 'potai', 'potch', 'poted', 'potes', 'potin', 'potoo', 'potro', 'potsy', 'potto', 'potts', 'potty', 'pouce', 'pouch', 'pouff', 'poufs', 'poufy', 'pouis', 'pouke', 'pouks', 'poule', 'poulp', 'poult', 'pound', 'poupe', 'poupt', 'pours', 'pousy', 'pouts', 'pouty', 'povos', 'powan', 'power', 'powie', 'powin', 'powis', 'powlt', 'pownd', 'powns', 'powny', 'powre', 'powsy', 'poxed', 'poxes', 'poyas', 'poynt', 'poyou', 'poyse', 'pozzy', 'praam', 'prads', 'prags', 'prahu', 'prams', 'prana', 'prang', 'prank', 'praos', 'praps', 'prase', 'prate', 'prats', 'pratt', 'praty', 'praus', 'prawn', 'prays', 'preak', 'predy', 'preed', 'preem', 'preen', 'prees', 'preif', 'preke', 'prems', 'premy', 'prent', 'preon', 'preop', 'preps', 'presa', 'prese', 'press', 'prest', 'preta', 'preux', 'preve', 'prexy', 'preys', 'prial', 'prian', 'price', 'prick', 'pricy', 'pride', 'pridy', 'pried', 'prief', 'prier', 'pries', 'prigs', 'prill', 'prima', 'prime', 'primi', 'primo', 'primp', 'prims', 'primy', 'pring', 'prink', 'print', 'prion', 'prior', 'prise', 'prism', 'priss', 'prius', 'privy', 'prize', 'proal', 'proas', 'probe', 'probs', 'proby', 'prodd', 'prods', 'proem', 'profs', 'progs', 'proin', 'proke', 'prole', 'proll', 'promo', 'proms', 'prone', 'prong', 'pronk', 'proof', 'prook', 'proot', 'props', 'prora', 'prore', 'prose', 'proso', 'pross', 'prost', 'prosy', 'proto', 'proud', 'proul', 'prove', 'prowk', 'prowl', 'prows', 'proxy', 'proyn', 'prude', 'prune', 'pruno', 'prunt', 'pruny', 'pruta', 'pryan', 'pryer', 'pryse', 'psalm', 'pseud', 'pshaw', 'pshut', 'psias', 'psion', 'psoae', 'psoai', 'psoas', 'psora', 'psych', 'psyop', 'ptish', 'ptype', 'pubby', 'pubco', 'pubes', 'pubic', 'pubis', 'pubsy', 'pucan', 'pucer', 'puces', 'pucka', 'pucks', 'puddy', 'pudge', 'pudgy', 'pudic', 'pudor', 'pudsy', 'pudus', 'puers', 'puffa', 'puffs', 'puffy', 'puggy', 'pugil', 'puhas', 'pujah', 'pujas', 'pukas', 'puked', 'puker', 'pukes', 'pukey', 'pukka', 'pukus', 'pulao', 'pulas', 'puled', 'puler', 'pules', 'pulik', 'pulis', 'pulka', 'pulks', 'pulli', 'pulls', 'pully', 'pulmo', 'pulps', 'pulpy', 'pulse', 'pulus', 'pulut', 'pumas', 'pumie', 'pumps', 'pumpy', 'punas', 'punce', 'punch', 'punga', 'pungi', 'pungo', 'pungs', 'pungy', 'punim', 'punji', 'punka', 'punks', 'punky', 'punny', 'punto', 'punts', 'punty', 'pupae', 'pupal', 'pupas', 'pupil', 'puppa', 'puppy', 'pupus', 'purao', 'purau', 'purda', 'purdy', 'pured', 'puree', 'purer', 'pures', 'purga', 'purge', 'purin', 'puris', 'purls', 'puros', 'purps', 'purpy', 'purre', 'purrs', 'purry', 'purse', 'pursy', 'purty', 'puses', 'pushy', 'pusle', 'pussy', 'putas', 'puter', 'putid', 'putin', 'puton', 'putos', 'putti', 'putto', 'putts', 'puttu', 'putty', 'putza', 'puuko', 'puyas', 'puzel', 'puzta', 'pwned', 'pyats', 'pyets', 'pygal', 'pygmy', 'pyins', 'pylon', 'pyned', 'pynes', 'pyoid', 'pyots', 'pyral', 'pyran', 'pyres', 'pyrex', 'pyric', 'pyros', 'pyrus', 'pyuff', 'pyxed', 'pyxes', 'pyxie', 'pyxis', 'pzazz', 'qadis', 'qaids', 'qajaq', 'qanat', 'qapik', 'qibla', 'qilas', 'qipao', 'qophs', 'qorma', 'quabs', 'quack', 'quads', 'quaff', 'quags', 'quail', 'quair', 'quais', 'quake', 'quaky', 'quale', 'qualm', 'qualy', 'quank', 'quant', 'quare', 'quark', 'quarl', 'quart', 'quash', 'quasi', 'quass', 'quate', 'quats', 'quawk', 'quaws', 'quayd', 'quays', 'qubit', 'quean', 'queck', 'queek', 'queem', 'queen', 'queer', 'quell', 'queme', 'quena', 'quern', 'query', 'queso', 'quest', 'quete', 'queue', 'queyn', 'queys', 'queyu', 'quibs', 'quich', 'quick', 'quids', 'quies', 'quiet', 'quiff', 'quila', 'quill', 'quilt', 'quims', 'quina', 'quine', 'quink', 'quino', 'quins', 'quint', 'quipo', 'quips', 'quipu', 'quire', 'quirk', 'quirl', 'quirt', 'quist', 'quite', 'quits', 'quoad', 'quods', 'quoif', 'quoin', 'quois', 'quoit', 'quoll', 'quonk', 'quops', 'quork', 'quorl', 'quota', 'quote', 'quoth', 'quouk', 'quoys', 'quran', 'qursh', 'quyte', 'raads', 'raake', 'rabat', 'rabbi', 'rabic', 'rabid', 'rabis', 'raced', 'racer', 'races', 'rache', 'racks', 'racon', 'radar', 'raddi', 'raddy', 'radge', 'radgy', 'radif', 'radii', 'radio', 'radix', 'radon', 'rafee', 'raffs', 'raffy', 'rafik', 'rafiq', 'rafts', 'rafty', 'ragas', 'ragde', 'raged', 'ragee', 'rager', 'rages', 'ragga', 'raggs', 'raggy', 'ragis', 'ragus', 'rahed', 'rahui', 'raiah', 'raias', 'raids', 'raike', 'raiks', 'raile', 'rails', 'raine', 'rains', 'rainy', 'raird', 'raise', 'raita', 'raith', 'raits', 'rajah', 'rajas', 'rajes', 'raked', 'rakee', 'raker', 'rakes', 'rakhi', 'rakia', 'rakis', 'rakki', 'raksi', 'rakus', 'rales', 'ralli', 'rally', 'ralph', 'ramal', 'ramee', 'ramen', 'rames', 'ramet', 'ramie', 'ramin', 'ramis', 'rammy', 'ramon', 'ramps', 'ramse', 'ramsh', 'ramus', 'ranas', 'rance', 'ranch', 'rando', 'rands', 'randy', 'raned', 'ranee', 'ranes', 'ranga', 'range', 'rangi', 'rangs', 'rangy', 'ranid', 'ranis', 'ranke', 'ranks', 'ranns', 'ranny', 'ranse', 'rants', 'ranty', 'raped', 'rapee', 'raper', 'rapes', 'raphe', 'rapid', 'rapin', 'rappe', 'rapso', 'rared', 'raree', 'rarer', 'rares', 'rarks', 'rasam', 'rasas', 'rased', 'raser', 'rases', 'rasps', 'raspy', 'rasse', 'rasta', 'ratal', 'ratan', 'ratas', 'ratch', 'rated', 'ratel', 'rater', 'rates', 'ratha', 'rathe', 'raths', 'ratio', 'ratoo', 'ratos', 'ratti', 'ratty', 'ratus', 'rauli', 'rauns', 'raupo', 'raved', 'ravel', 'raven', 'raver', 'raves', 'ravey', 'ravin', 'rawdy', 'rawer', 'rawin', 'rawks', 'rawly', 'rawns', 'raxed', 'raxes', 'rayah', 'rayas', 'rayed', 'rayle', 'rayls', 'rayne', 'rayon', 'razai', 'razed', 'razee', 'razer', 'razes', 'razet', 'razoo', 'razor', 'reach', 'react', 'readd', 'reads', 'ready', 'reais', 'reaks', 'realm', 'realo', 'reals', 'reame', 'reams', 'reamy', 'reans', 'reaps', 'reard', 'rearm', 'rears', 'reast', 'reata', 'reate', 'reave', 'rebab', 'rebar', 'rebbe', 'rebec', 'rebel', 'rebid', 'rebit', 'rebop', 'rebud', 'rebus', 'rebut', 'rebuy', 'recal', 'recap', 'recce', 'recco', 'reccy', 'recep', 'recit', 'recks', 'recon', 'recta', 'recte', 'recti', 'recto', 'recue', 'recur', 'recut', 'redan', 'redds', 'reddy', 'reded', 'redes', 'redia', 'redid', 'redif', 'redig', 'redip', 'redly', 'redon', 'redos', 'redox', 'redry', 'redub', 'redug', 'redux', 'redye', 'reeaf', 'reech', 'reede', 'reeds', 'reedy', 'reefs', 'reefy', 'reeks', 'reeky', 'reels', 'reely', 'reems', 'reens', 'reerd', 'reest', 'reeve', 'reeze', 'refan', 'refed', 'refel', 'refer', 'reffo', 'refis', 'refit', 'refix', 'refly', 'refry', 'regal', 'regar', 'reges', 'reget', 'regex', 'reggo', 'regia', 'regie', 'regle', 'regma', 'regna', 'regos', 'regot', 'regur', 'rehab', 'rehem', 'reifs', 'reify', 'reign', 'reiki', 'reiks', 'reine', 'reing', 'reink', 'reins', 'reird', 'reist', 'reive', 'rejas', 'rejig', 'rejon', 'reked', 'rekes', 'rekey', 'relax', 'relay', 'relet', 'relic', 'relie', 'relit', 'rello', 'relos', 'reman', 'remap', 'remen', 'remet', 'remex', 'remit', 'remix', 'remou', 'renal', 'renay', 'rends', 'rendu', 'renew', 'reney', 'renga', 'rengs', 'renig', 'renin', 'renks', 'renne', 'renos', 'rente', 'rents', 'reoil', 'reorg', 'repas', 'repat', 'repay', 'repeg', 'repel', 'repen', 'repin', 'repla', 'reply', 'repos', 'repot', 'repps', 'repro', 'repun', 'reput', 'reran', 'rerig', 'rerun', 'resam', 'resat', 'resaw', 'resay', 'resee', 'reses', 'reset', 'resew', 'resid', 'resin', 'resit', 'resod', 'resol', 'resow', 'resto', 'rests', 'resty', 'resue', 'resus', 'retag', 'retam', 'retax', 'retch', 'retem', 'retia', 'retie', 'retin', 'retip', 'retox', 'retro', 'retry', 'reune', 'reups', 'reuse', 'revel', 'revet', 'revie', 'revow', 'revue', 'rewan', 'rewax', 'rewed', 'rewet', 'rewin', 'rewon', 'rewth', 'rexes', 'rezes', 'rhabd', 'rheas', 'rheid', 'rheme', 'rheum', 'rhies', 'rhime', 'rhine', 'rhino', 'rhody', 'rhomb', 'rhone', 'rhumb', 'rhyme', 'rhymy', 'rhyne', 'rhyta', 'riads', 'rials', 'riant', 'riata', 'riato', 'ribas', 'ribby', 'ribes', 'riced', 'ricer', 'rices', 'ricey', 'riche', 'richt', 'ricin', 'ricks', 'rider', 'rides', 'ridge', 'ridgy', 'ridic', 'riels', 'riems', 'rieve', 'rifer', 'riffs', 'riffy', 'rifle', 'rifte', 'rifts', 'rifty', 'riggs', 'right', 'rigid', 'rigmo', 'rigol', 'rigor', 'rikka', 'rikwa', 'riled', 'riles', 'riley', 'rille', 'rills', 'rilly', 'rimae', 'rimed', 'rimer', 'rimes', 'rimon', 'rimus', 'rince', 'rinds', 'rindy', 'rines', 'ringe', 'rings', 'ringy', 'rinks', 'rinse', 'rioja', 'rione', 'riots', 'rioty', 'riped', 'ripen', 'riper', 'ripes', 'ripps', 'riqqs', 'risen', 'riser', 'rises', 'rishi', 'risks', 'risky', 'risps', 'rists', 'risus', 'rites', 'rithe', 'ritts', 'ritzy', 'rival', 'rivas', 'rived', 'rivel', 'riven', 'river', 'rives', 'rivet', 'riyal', 'rizas', 'roach', 'roads', 'roady', 'roake', 'roaky', 'roams', 'roans', 'roany', 'roars', 'roary', 'roast', 'roate', 'robbo', 'robed', 'rober', 'robes', 'robin', 'roble', 'robot', 'robug', 'robur', 'roche', 'rocks', 'rocky', 'roded', 'rodeo', 'rodes', 'rodny', 'roers', 'rogan', 'roger', 'rogue', 'roguy', 'rohan', 'rohes', 'rohun', 'rohus', 'roids', 'roils', 'roily', 'roins', 'roist', 'rojak', 'rojis', 'roked', 'roker', 'rokes', 'rokey', 'rokos', 'rolag', 'roleo', 'roles', 'rolfs', 'rolls', 'rolly', 'romal', 'roman', 'romeo', 'romer', 'romps', 'rompu', 'rompy', 'ronde', 'rondo', 'roneo', 'rones', 'ronin', 'ronne', 'ronte', 'ronts', 'ronuk', 'roods', 'roofs', 'roofy', 'rooks', 'rooky', 'rooms', 'roomy', 'roons', 'roops', 'roopy', 'roosa', 'roose', 'roost', 'roots', 'rooty', 'roped', 'roper', 'ropes', 'ropey', 'roque', 'roral', 'rores', 'roric', 'rorid', 'rorie', 'rorts', 'rorty', 'rosal', 'rosco', 'rosed', 'roses', 'roset', 'rosha', 'roshi', 'rosin', 'rosit', 'rosps', 'rossa', 'rosso', 'rosti', 'rosts', 'rotal', 'rotan', 'rotas', 'rotch', 'roted', 'rotes', 'rotis', 'rotls', 'roton', 'rotor', 'rotos', 'rotta', 'rotte', 'rotto', 'rotty', 'rouen', 'roues', 'rouet', 'roufs', 'rouge', 'rough', 'rougy', 'rouks', 'rouky', 'roule', 'rouls', 'roums', 'round', 'roups', 'roupy', 'rouse', 'roust', 'route', 'routh', 'routs', 'roved', 'roven', 'rover', 'roves', 'rowan', 'rowdy', 'rowed', 'rowel', 'rowen', 'rower', 'rowet', 'rowie', 'rowme', 'rownd', 'rowns', 'rowth', 'rowts', 'royal', 'royet', 'royne', 'royst', 'rozes', 'rozet', 'rozit', 'ruach', 'ruana', 'rubai', 'ruban', 'rubby', 'rubel', 'rubes', 'rubin', 'rubio', 'ruble', 'rubli', 'rubor', 'rubus', 'ruche', 'ruchy', 'rucks', 'rudas', 'rudds', 'ruddy', 'ruder', 'rudes', 'rudie', 'rudis', 'rueda', 'ruers', 'ruffe', 'ruffs', 'ruffy', 'rufus', 'rugae', 'rugal', 'rugas', 'rugby', 'ruggy', 'ruice', 'ruing', 'ruins', 'rukhs', 'ruled', 'ruler', 'rules', 'rully', 'rumal', 'rumba', 'rumbo', 'rumen', 'rumes', 'rumly', 'rummy', 'rumor', 'rumpo', 'rumps', 'rumpy', 'runce', 'runch', 'runds', 'runed', 'runer', 'runes', 'rungs', 'runic', 'runny', 'runos', 'runts', 'runty', 'runup', 'ruote', 'rupee', 'rupia', 'rural', 'rurps', 'rurus', 'rusas', 'ruses', 'rushy', 'rusks', 'rusky', 'rusma', 'russe', 'rusts', 'rusty', 'ruths', 'rutin', 'rutty', 'ruvid', 'ryals', 'rybat', 'ryiji', 'ryijy', 'ryked', 'rykes', 'rymer', 'rymme', 'rynds', 'ryoti', 'ryots', 'ryper', 'rypin', 'rythe', 'ryugi', 'saags', 'sabal', 'sabed', 'saber', 'sabes', 'sabha', 'sabin', 'sabir', 'sabji', 'sable', 'sabos', 'sabot', 'sabra', 'sabre', 'sabzi', 'sacks', 'sacra', 'sacre', 'saddo', 'saddy', 'sades', 'sadhe', 'sadhu', 'sadic', 'sadis', 'sadly', 'sados', 'sadza', 'saeta', 'safed', 'safer', 'safes', 'sagar', 'sagas', 'sager', 'sages', 'saggy', 'sagos', 'sagum', 'sahab', 'saheb', 'sahib', 'saice', 'saick', 'saics', 'saids', 'saiga', 'sails', 'saims', 'saine', 'sains', 'saint', 'sairs', 'saist', 'saith', 'sajou', 'sakai', 'saker', 'sakes', 'sakia', 'sakis', 'sakti', 'salad', 'salal', 'salas', 'salat', 'salep', 'sales', 'salet', 'salic', 'salis', 'salix', 'salle', 'sally', 'salmi', 'salol', 'salon', 'salop', 'salpa', 'salps', 'salsa', 'salse', 'salto', 'salts', 'salty', 'salud', 'salue', 'salut', 'salve', 'salvo', 'saman', 'samas', 'samba', 'sambo', 'samek', 'samel', 'samen', 'sames', 'samey', 'samfi', 'samfu', 'sammy', 'sampi', 'samps', 'sanad', 'sands', 'sandy', 'saned', 'saner', 'sanes', 'sanga', 'sangh', 'sango', 'sangs', 'sanko', 'sansa', 'santo', 'sants', 'saola', 'sapan', 'sapid', 'sapor', 'sappy', 'saran', 'sards', 'sared', 'saree', 'sarge', 'sargo', 'sarin', 'sarir', 'saris', 'sarks', 'sarky', 'sarod', 'saros', 'sarus', 'sarvo', 'saser', 'sasin', 'sasse', 'sassy', 'satai', 'satay', 'sated', 'satem', 'sater', 'sates', 'satin', 'satis', 'satyr', 'sauba', 'sauce', 'sauch', 'saucy', 'saugh', 'sauls', 'sault', 'sauna', 'saunf', 'saunt', 'saury', 'saute', 'sauts', 'sauve', 'saved', 'saver', 'saves', 'savey', 'savin', 'savor', 'savoy', 'savvy', 'sawah', 'sawed', 'sawer', 'saxes', 'sayas', 'sayed', 'sayee', 'sayer', 'sayid', 'sayne', 'sayon', 'sayst', 'sazes', 'scabs', 'scads', 'scaff', 'scags', 'scail', 'scala', 'scald', 'scale', 'scall', 'scalp', 'scaly', 'scamp', 'scams', 'scand', 'scans', 'scant', 'scapa', 'scape', 'scapi', 'scare', 'scarf', 'scarp', 'scars', 'scart', 'scary', 'scath', 'scats', 'scatt', 'scaud', 'scaup', 'scaur', 'scaws', 'sceat', 'scena', 'scend', 'scene', 'scent', 'schav', 'schif', 'schmo', 'schul', 'schwa', 'scifi', 'scind', 'scion', 'scire', 'sclim', 'scobe', 'scody', 'scoff', 'scogs', 'scold', 'scone', 'scoog', 'scoop', 'scoot', 'scopa', 'scope', 'scops', 'score', 'scorn', 'scorp', 'scote', 'scots', 'scoug', 'scoup', 'scour', 'scout', 'scowl', 'scowp', 'scows', 'scrab', 'scrae', 'scrag', 'scram', 'scran', 'scrap', 'scrat', 'scraw', 'scray', 'scree', 'screw', 'scrim', 'scrip', 'scrob', 'scrod', 'scrog', 'scroo', 'scrow', 'scrub', 'scrum', 'scuba', 'scudi', 'scudo', 'scuds', 'scuff', 'scuft', 'scugs', 'sculk', 'scull', 'sculp', 'sculs', 'scums', 'scups', 'scurf', 'scurs', 'scuse', 'scuta', 'scute', 'scuts', 'scuzz', 'scyes', 'sdayn', 'sdein', 'seals', 'seame', 'seams', 'seamy', 'seans', 'seare', 'sears', 'sease', 'seats', 'seaze', 'sebum', 'secco', 'sechs', 'sects', 'sedan', 'seder', 'sedes', 'sedge', 'sedgy', 'sedum', 'seeds', 'seedy', 'seeks', 'seeld', 'seels', 'seely', 'seems', 'seeps', 'seepy', 'seers', 'sefer', 'segar', 'segas', 'segni', 'segno', 'segol', 'segos', 'segue', 'sehri', 'seifs', 'seils', 'seine', 'seirs', 'seise', 'seism', 'seity', 'seiza', 'seize', 'sekos', 'sekts', 'selah', 'seles', 'selfs', 'selfy', 'selky', 'sella', 'selle', 'sells', 'selva', 'semas', 'semee', 'semen', 'semes', 'semie', 'semis', 'senas', 'sends', 'senes', 'senex', 'sengi', 'senna', 'senor', 'sensa', 'sense', 'sensi', 'sensu', 'sente', 'senti', 'sents', 'senvy', 'senza', 'sepad', 'sepal', 'sepia', 'sepic', 'sepoy', 'seppo', 'septa', 'septs', 'serac', 'serai', 'seral', 'sered', 'serer', 'seres', 'serfs', 'serge', 'seria', 'seric', 'serif', 'serin', 'serir', 'serks', 'seron', 'serow', 'serra', 'serre', 'serrs', 'serry', 'serum', 'serve', 'servo', 'sesey', 'sessa', 'setae', 'setal', 'seter', 'seths', 'seton', 'setts', 'setup', 'sevak', 'seven', 'sever', 'sevir', 'sewan', 'sewar', 'sewed', 'sewel', 'sewen', 'sewer', 'sewin', 'sexed', 'sexer', 'sexes', 'sexor', 'sexto', 'sexts', 'seyen', 'sezes', 'shack', 'shade', 'shads', 'shady', 'shaft', 'shags', 'shahs', 'shaka', 'shake', 'shako', 'shakt', 'shaky', 'shale', 'shall', 'shalm', 'shalt', 'shaly', 'shama', 'shame', 'shams', 'shand', 'shank', 'shans', 'shape', 'shaps', 'shard', 'share', 'shark', 'sharn', 'sharp', 'shart', 'shash', 'shaul', 'shave', 'shawl', 'shawm', 'shawn', 'shaws', 'shaya', 'shays', 'shchi', 'sheaf', 'sheal', 'shear', 'sheas', 'sheds', 'sheel', 'sheen', 'sheep', 'sheer', 'sheet', 'sheik', 'shelf', 'shell', 'shend', 'sheng', 'shent', 'sheol', 'sherd', 'shere', 'shero', 'shets', 'sheva', 'shewn', 'shews', 'shiai', 'shied', 'shiel', 'shier', 'shies', 'shift', 'shill', 'shily', 'shims', 'shine', 'shins', 'shiny', 'shiok', 'ships', 'shire', 'shirk', 'shirr', 'shirs', 'shirt', 'shish', 'shiso', 'shist', 'shite', 'shits', 'shiur', 'shiva', 'shive', 'shivs', 'shlep', 'shlub', 'shmek', 'shmoe', 'shoal', 'shoat', 'shock', 'shoed', 'shoer', 'shoes', 'shogi', 'shogs', 'shoji', 'shojo', 'shola', 'shone', 'shonk', 'shook', 'shool', 'shoon', 'shoos', 'shoot', 'shope', 'shops', 'shore', 'shorl', 'shorn', 'short', 'shote', 'shots', 'shott', 'shoud', 'shout', 'shove', 'showd', 'shown', 'shows', 'showy', 'shoyu', 'shred', 'shrew', 'shris', 'shrow', 'shrub', 'shrug', 'shtar', 'shtik', 'shtum', 'shtup', 'shuba', 'shuck', 'shule', 'shuln', 'shuls', 'shuns', 'shunt', 'shura', 'shush', 'shute', 'shuts', 'shwas', 'shyer', 'shyly', 'sials', 'sibbs', 'sibia', 'sibyl', 'sices', 'sicht', 'sicko', 'sicks', 'sicky', 'sidas', 'sided', 'sider', 'sides', 'sidey', 'sidha', 'sidhe', 'sidle', 'siege', 'sield', 'siens', 'sient', 'sieth', 'sieur', 'sieve', 'sifts', 'sighs', 'sight', 'sigil', 'sigla', 'sigma', 'signa', 'signs', 'sigri', 'sijos', 'sikas', 'siker', 'sikes', 'silds', 'siled', 'silen', 'siler', 'siles', 'silex', 'silks', 'silky', 'sills', 'silly', 'silos', 'silts', 'silty', 'silva', 'simar', 'simas', 'simba', 'simis', 'simps', 'simul', 'since', 'sinds', 'sined', 'sines', 'sinew', 'singe', 'sings', 'sinhs', 'sinks', 'sinky', 'sinsi', 'sinus', 'siped', 'sipes', 'sippy', 'sired', 'siree', 'siren', 'sires', 'sirih', 'siris', 'siroc', 'sirra', 'sirup', 'sisal', 'sises', 'sissy', 'sista', 'sists', 'sitar', 'sitch', 'sited', 'sites', 'sithe', 'sitka', 'situp', 'situs', 'siver', 'sixer', 'sixes', 'sixmo', 'sixte', 'sixth', 'sixty', 'sizar', 'sized', 'sizel', 'sizer', 'sizes', 'skags', 'skail', 'skald', 'skank', 'skarn', 'skart', 'skate', 'skats', 'skatt', 'skaws', 'skean', 'skear', 'skeds', 'skeed', 'skeef', 'skeen', 'skeer', 'skees', 'skeet', 'skeev', 'skeez', 'skegg', 'skegs', 'skein', 'skelf', 'skell', 'skelm', 'skelp', 'skene', 'skens', 'skeos', 'skeps', 'skerm', 'skers', 'skets', 'skews', 'skids', 'skied', 'skier', 'skies', 'skiey', 'skiff', 'skill', 'skimo', 'skimp', 'skims', 'skink', 'skins', 'skint', 'skios', 'skips', 'skirl', 'skirr', 'skirt', 'skite', 'skits', 'skive', 'skivy', 'sklim', 'skoal', 'skobe', 'skody', 'skoff', 'skofs', 'skogs', 'skols', 'skool', 'skort', 'skosh', 'skran', 'skrik', 'skroo', 'skuas', 'skugs', 'skulk', 'skull', 'skunk', 'skyed', 'skyer', 'skyey', 'skyfs', 'skyre', 'skyrs', 'skyte', 'slabs', 'slack', 'slade', 'slaes', 'slags', 'slaid', 'slain', 'slake', 'slams', 'slane', 'slang', 'slank', 'slant', 'slaps', 'slart', 'slash', 'slate', 'slats', 'slaty', 'slave', 'slaws', 'slays', 'slebs', 'sleds', 'sleek', 'sleep', 'sleer', 'sleet', 'slept', 'slews', 'sleys', 'slice', 'slick', 'slide', 'slier', 'slily', 'slime', 'slims', 'slimy', 'sling', 'slink', 'slipe', 'slips', 'slipt', 'slish', 'slits', 'slive', 'sloan', 'slobs', 'sloes', 'slogs', 'sloid', 'slojd', 'sloka', 'slomo', 'sloom', 'sloop', 'sloot', 'slope', 'slops', 'slopy', 'slorm', 'slosh', 'sloth', 'slots', 'slove', 'slows', 'sloyd', 'slubb', 'slubs', 'slued', 'slues', 'sluff', 'slugs', 'sluit', 'slump', 'slums', 'slung', 'slunk', 'slurb', 'slurp', 'slurs', 'sluse', 'slush', 'sluts', 'slyer', 'slyly', 'slype', 'smaak', 'smack', 'smaik', 'small', 'smalm', 'smalt', 'smarm', 'smart', 'smash', 'smaze', 'smear', 'smeek', 'smees', 'smeik', 'smeke', 'smell', 'smelt', 'smerk', 'smews', 'smick', 'smile', 'smily', 'smirk', 'smirr', 'smirs', 'smite', 'smith', 'smits', 'smize', 'smock', 'smogs', 'smoke', 'smoko', 'smoky', 'smolt', 'smoor', 'smoot', 'smore', 'smorg', 'smote', 'smout', 'smowt', 'smugs', 'smurs', 'smush', 'smuts', 'snabs', 'snack', 'snafu', 'snags', 'snail', 'snake', 'snaky', 'snaps', 'snare', 'snarf', 'snark', 'snarl', 'snars', 'snary', 'snash', 'snath', 'snaws', 'snead', 'sneak', 'sneap', 'snebs', 'sneck', 'sneds', 'sneed', 'sneer', 'snees', 'snell', 'snibs', 'snick', 'snide', 'snied', 'snies', 'sniff', 'snift', 'snigs', 'snipe', 'snips', 'snipy', 'snirt', 'snits', 'snive', 'snobs', 'snods', 'snoek', 'snoep', 'snogs', 'snoke', 'snood', 'snook', 'snool', 'snoop', 'snoot', 'snore', 'snort', 'snots', 'snout', 'snowk', 'snows', 'snowy', 'snubs', 'snuck', 'snuff', 'snugs', 'snush', 'snyes', 'soaks', 'soaps', 'soapy', 'soare', 'soars', 'soave', 'sobas', 'sober', 'socas', 'soces', 'socia', 'socko', 'socks', 'socle', 'sodas', 'soddy', 'sodic', 'sodom', 'sofar', 'sofas', 'softa', 'softs', 'softy', 'soger', 'soggy', 'sohur', 'soils', 'soily', 'sojas', 'sojus', 'sokah', 'soken', 'sokes', 'sokol', 'solah', 'solan', 'solar', 'solas', 'solde', 'soldi', 'soldo', 'solds', 'soled', 'solei', 'soler', 'soles', 'solid', 'solon', 'solos', 'solum', 'solus', 'solve', 'soman', 'somas', 'sonar', 'sonce', 'sonde', 'sones', 'songo', 'songs', 'songy', 'sonic', 'sonly', 'sonne', 'sonny', 'sonse', 'sonsy', 'sooey', 'sooks', 'sooky', 'soole', 'sools', 'sooms', 'soops', 'soote', 'sooth', 'soots', 'sooty', 'sophs', 'sophy', 'sopor', 'soppy', 'sopra', 'soral', 'soras', 'sorbi', 'sorbo', 'sorbs', 'sorda', 'sordo', 'sords', 'sored', 'soree', 'sorel', 'sorer', 'sores', 'sorex', 'sorgo', 'sorns', 'sorra', 'sorry', 'sorta', 'sorts', 'sorus', 'soths', 'sotol', 'sotto', 'souce', 'souct', 'sough', 'souks', 'souls', 'souly', 'soums', 'sound', 'soups', 'soupy', 'sours', 'souse', 'south', 'souts', 'sowar', 'sowce', 'sowed', 'sower', 'sowff', 'sowfs', 'sowle', 'sowls', 'sowms', 'sownd', 'sowne', 'sowps', 'sowse', 'sowth', 'soxes', 'soyas', 'soyle', 'soyuz', 'sozin', 'space', 'spack', 'spacy', 'spade', 'spado', 'spads', 'spaed', 'spaer', 'spaes', 'spags', 'spahi', 'spail', 'spain', 'spait', 'spake', 'spald', 'spale', 'spall', 'spalt', 'spams', 'spane', 'spang', 'spank', 'spans', 'spard', 'spare', 'spark', 'spars', 'spart', 'spasm', 'spate', 'spats', 'spaul', 'spawl', 'spawn', 'spaws', 'spayd', 'spays', 'spaza', 'spazz', 'speak', 'speal', 'spean', 'spear', 'speat', 'speck', 'specs', 'spect', 'speed', 'speel', 'speer', 'speil', 'speir', 'speks', 'speld', 'spelk', 'spell', 'spelt', 'spend', 'spent', 'speos', 'sperm', 'spesh', 'spets', 'speug', 'spews', 'spewy', 'spial', 'spica', 'spice', 'spick', 'spics', 'spicy', 'spide', 'spied', 'spiel', 'spier', 'spies', 'spiff', 'spifs', 'spike', 'spiks', 'spiky', 'spile', 'spill', 'spilt', 'spims', 'spina', 'spine', 'spink', 'spins', 'spiny', 'spire', 'spirt', 'spiry', 'spite', 'spits', 'spitz', 'spivs', 'splat', 'splay', 'split', 'splog', 'spode', 'spods', 'spoil', 'spoke', 'spoof', 'spook', 'spool', 'spoom', 'spoon', 'spoor', 'spoot', 'spore', 'spork', 'sport', 'sposa', 'sposh', 'sposo', 'spots', 'spout', 'sprad', 'sprag', 'sprat', 'spray', 'spred', 'spree', 'sprew', 'sprig', 'sprit', 'sprod', 'sprog', 'sprue', 'sprug', 'spuds', 'spued', 'spuer', 'spues', 'spugs', 'spule', 'spume', 'spumy', 'spunk', 'spurn', 'spurs', 'spurt', 'sputa', 'spyal', 'spyre', 'squab', 'squad', 'squat', 'squaw', 'squee', 'squeg', 'squib', 'squid', 'squit', 'squiz', 'srsly', 'stabs', 'stack', 'stade', 'staff', 'stage', 'stags', 'stagy', 'staid', 'staig', 'stain', 'stair', 'stake', 'stale', 'stalk', 'stall', 'stamp', 'stand', 'stane', 'stang', 'stank', 'stans', 'staph', 'staps', 'stare', 'stark', 'starn', 'starr', 'stars', 'start', 'stary', 'stash', 'state', 'stats', 'statu', 'staun', 'stave', 'staws', 'stays', 'stead', 'steak', 'steal', 'steam', 'stean', 'stear', 'stedd', 'stede', 'steds', 'steed', 'steek', 'steel', 'steem', 'steen', 'steep', 'steer', 'steez', 'steik', 'steil', 'stein', 'stela', 'stele', 'stell', 'steme', 'stems', 'stend', 'steno', 'stens', 'stent', 'steps', 'stept', 'stere', 'stern', 'stets', 'stews', 'stewy', 'steys', 'stich', 'stick', 'stied', 'sties', 'stiff', 'stilb', 'stile', 'still', 'stilt', 'stime', 'stims', 'stimy', 'sting', 'stink', 'stint', 'stipa', 'stipe', 'stire', 'stirk', 'stirp', 'stirs', 'stive', 'stivy', 'stoae', 'stoai', 'stoas', 'stoat', 'stobs', 'stock', 'stoep', 'stogs', 'stogy', 'stoic', 'stoit', 'stoke', 'stole', 'stoln', 'stoma', 'stomp', 'stond', 'stone', 'stong', 'stonk', 'stonn', 'stony', 'stood', 'stook', 'stool', 'stoop', 'stoor', 'stope', 'stops', 'stopt', 'store', 'stork', 'storm', 'story', 'stoss', 'stots', 'stott', 'stoun', 'stoup', 'stour', 'stout', 'stove', 'stown', 'stowp', 'stows', 'strad', 'strae', 'strag', 'strak', 'strap', 'straw', 'stray', 'strep', 'strew', 'stria', 'strig', 'strim', 'strip', 'strop', 'strow', 'stroy', 'strum', 'strut', 'stubs', 'stuck', 'stucs', 'stude', 'studs', 'study', 'stuff', 'stull', 'stulm', 'stumm', 'stump', 'stums', 'stung', 'stunk', 'stuns', 'stunt', 'stupa', 'stupe', 'sture', 'sturt', 'stush', 'styed', 'styes', 'style', 'styli', 'stylo', 'styme', 'stymy', 'styre', 'styte', 'suave', 'subah', 'subak', 'subas', 'subby', 'suber', 'subha', 'succi', 'sucks', 'sucky', 'sucre', 'sudan', 'sudds', 'sudor', 'sudsy', 'suede', 'suent', 'suers', 'suete', 'suets', 'suety', 'sugan', 'sugar', 'sughs', 'sugos', 'suhur', 'suids', 'suing', 'suint', 'suite', 'suits', 'sujee', 'sukhs', 'sukis', 'sukuk', 'sulci', 'sulfa', 'sulfo', 'sulks', 'sulky', 'sulls', 'sully', 'sulph', 'sulus', 'sumac', 'sumis', 'summa', 'sumos', 'sumph', 'sumps', 'sunis', 'sunks', 'sunna', 'sunns', 'sunny', 'sunts', 'sunup', 'suona', 'suped', 'super', 'supes', 'supra', 'surah', 'sural', 'suras', 'surat', 'surds', 'sured', 'surer', 'sures', 'surfs', 'surfy', 'surge', 'surgy', 'surly', 'surra', 'sused', 'suses', 'sushi', 'susus', 'sutor', 'sutra', 'sutta', 'swabs', 'swack', 'swads', 'swage', 'swags', 'swail', 'swain', 'swale', 'swaly', 'swami', 'swamp', 'swamy', 'swang', 'swank', 'swans', 'swaps', 'swapt', 'sward', 'sware', 'swarf', 'swarm', 'swart', 'swash', 'swath', 'swats', 'swayl', 'sways', 'sweal', 'swear', 'sweat', 'swede', 'sweed', 'sweel', 'sweep', 'sweer', 'swees', 'sweet', 'sweir', 'swell', 'swelt', 'swept', 'swerf', 'sweys', 'swies', 'swift', 'swigs', 'swile', 'swill', 'swims', 'swine', 'swing', 'swink', 'swipe', 'swire', 'swirl', 'swish', 'swiss', 'swith', 'swits', 'swive', 'swizz', 'swobs', 'swole', 'swoll', 'swoln', 'swoon', 'swoop', 'swops', 'swopt', 'sword', 'swore', 'sworn', 'swots', 'swoun', 'swung', 'sybbe', 'sybil', 'syboe', 'sybow', 'sycee', 'syces', 'sycon', 'syeds', 'syens', 'syker', 'sykes', 'sylis', 'sylph', 'sylva', 'symar', 'synch', 'syncs', 'synds', 'syned', 'synes', 'synod', 'synth', 'syped', 'sypes', 'syphs', 'syrah', 'syren', 'syrup', 'sysop', 'sythe', 'syver', 'taals', 'taata', 'tabac', 'tabby', 'taber', 'tabes', 'tabid', 'tabis', 'tabla', 'table', 'tabls', 'taboo', 'tabor', 'tabos', 'tabun', 'tabus', 'tacan', 'taces', 'tacet', 'tache', 'tachi', 'tacho', 'tachs', 'tacit', 'tacks', 'tacky', 'tacos', 'tacts', 'tadah', 'taels', 'taffy', 'tafia', 'taggy', 'tagma', 'tagua', 'tahas', 'tahrs', 'taiga', 'taigs', 'taiko', 'tails', 'tains', 'taint', 'taira', 'taish', 'taits', 'tajes', 'takas', 'taken', 'taker', 'takes', 'takhi', 'takht', 'takin', 'takis', 'takky', 'talak', 'talaq', 'talar', 'talas', 'talcs', 'talcy', 'talea', 'taler', 'tales', 'talik', 'talks', 'talky', 'talls', 'tally', 'talma', 'talon', 'talpa', 'taluk', 'talus', 'tamal', 'tamas', 'tamed', 'tamer', 'tames', 'tamin', 'tamis', 'tammy', 'tamps', 'tanas', 'tanga', 'tangi', 'tango', 'tangs', 'tangy', 'tanhs', 'tania', 'tanka', 'tanks', 'tanky', 'tanna', 'tansu', 'tansy', 'tante', 'tanti', 'tanto', 'tanty', 'tapas', 'taped', 'tapen', 'taper', 'tapes', 'tapet', 'tapir', 'tapis', 'tappa', 'tapus', 'taras', 'tardo', 'tards', 'tardy', 'tared', 'tares', 'targa', 'targe', 'tarka', 'tarns', 'taroc', 'tarok', 'taros', 'tarot', 'tarps', 'tarre', 'tarry', 'tarse', 'tarsi', 'tarte', 'tarts', 'tarty', 'tarzy', 'tasar', 'tasca', 'tased', 'taser', 'tases', 'tasks', 'tassa', 'tasse', 'tasso', 'taste', 'tasto', 'tasty', 'tatar', 'tater', 'tates', 'taths', 'tatie', 'tatou', 'tatts', 'tatty', 'tatus', 'taube', 'tauld', 'taunt', 'tauon', 'taupe', 'tauts', 'tauty', 'tavah', 'tavas', 'taver', 'tawaf', 'tawai', 'tawas', 'tawed', 'tawer', 'tawie', 'tawny', 'tawse', 'tawts', 'taxed', 'taxer', 'taxes', 'taxis', 'taxol', 'taxon', 'taxor', 'taxus', 'tayra', 'tazza', 'tazze', 'teach', 'teade', 'teads', 'teaed', 'teaks', 'teals', 'teams', 'tears', 'teary', 'tease', 'teats', 'teaze', 'techs', 'techy', 'tecta', 'tecum', 'teddy', 'teels', 'teems', 'teend', 'teene', 'teens', 'teeny', 'teers', 'teeth', 'teets', 'teffs', 'teggs', 'tegua', 'tegus', 'tehee', 'tehrs', 'teiid', 'teils', 'teind', 'teins', 'tekke', 'telae', 'telco', 'teles', 'telex', 'telia', 'telic', 'tells', 'telly', 'teloi', 'telos', 'temed', 'temes', 'tempi', 'tempo', 'temps', 'tempt', 'temse', 'tench', 'tends', 'tendu', 'tenes', 'tenet', 'tenge', 'tenia', 'tenne', 'tenno', 'tenny', 'tenon', 'tenor', 'tense', 'tenth', 'tents', 'tenty', 'tenue', 'tepal', 'tepas', 'tepee', 'tepid', 'tepoy', 'terai', 'teras', 'terce', 'terek', 'teres', 'terfe', 'terfs', 'terga', 'terms', 'terne', 'terns', 'terra', 'terre', 'terry', 'terse', 'terts', 'terza', 'tesla', 'testa', 'teste', 'tests', 'testy', 'tetes', 'teths', 'tetra', 'tetri', 'teuch', 'teugh', 'tewed', 'tewel', 'tewit', 'texas', 'texes', 'texta', 'texts', 'thack', 'thagi', 'thaim', 'thale', 'thali', 'thana', 'thane', 'thang', 'thank', 'thans', 'thanx', 'tharm', 'thars', 'thaws', 'thawt', 'thawy', 'thebe', 'theca', 'theed', 'theek', 'thees', 'theft', 'thegn', 'theic', 'thein', 'their', 'thelf', 'thema', 'theme', 'thens', 'theor', 'theow', 'there', 'therm', 'these', 'thesp', 'theta', 'thete', 'thews', 'thewy', 'thick', 'thief', 'thigh', 'thigs', 'thilk', 'thill', 'thine', 'thing', 'think', 'thins', 'thiol', 'third', 'thirl', 'thoft', 'thole', 'tholi', 'thong', 'thorn', 'thoro', 'thorp', 'those', 'thots', 'thous', 'thowl', 'thrae', 'thraw', 'three', 'threw', 'thrid', 'thrip', 'throb', 'throe', 'throw', 'thrum', 'thuds', 'thugs', 'thuja', 'thumb', 'thump', 'thunk', 'thurl', 'thuya', 'thyme', 'thymi', 'thymy', 'tians', 'tiara', 'tiare', 'tiars', 'tibia', 'tical', 'ticca', 'ticed', 'tices', 'tichy', 'ticks', 'ticky', 'tidal', 'tiddy', 'tided', 'tides', 'tiefs', 'tiers', 'tiffs', 'tifos', 'tifts', 'tiger', 'tiges', 'tight', 'tigon', 'tikas', 'tikes', 'tikia', 'tikis', 'tikka', 'tilak', 'tilde', 'tiled', 'tiler', 'tiles', 'tills', 'tilly', 'tilth', 'tilts', 'timbo', 'timed', 'timer', 'times', 'timid', 'timon', 'timps', 'tinas', 'tinct', 'tinds', 'tinea', 'tined', 'tines', 'tinge', 'tings', 'tinks', 'tinny', 'tinto', 'tints', 'tinty', 'tipis', 'tippy', 'tipsy', 'tipup', 'tired', 'tires', 'tirls', 'tiros', 'tirrs', 'tirth', 'titan', 'titar', 'titas', 'titch', 'titer', 'tithe', 'tithi', 'titin', 'titir', 'titis', 'title', 'titre', 'titty', 'titup', 'tiyin', 'tiyns', 'tizes', 'tizzy', 'toads', 'toady', 'toast', 'toaze', 'tocks', 'tocky', 'tocos', 'today', 'todde', 'toddy', 'todea', 'todos', 'toeas', 'toffs', 'toffy', 'tofts', 'tofus', 'togae', 'togas', 'toged', 'toges', 'togue', 'tohos', 'toidy', 'toile', 'toils', 'toing', 'toise', 'toits', 'toity', 'tokay', 'toked', 'token', 'toker', 'tokes', 'tokos', 'tolan', 'tolar', 'tolas', 'toled', 'toles', 'tolls', 'tolly', 'tolts', 'tolus', 'tolyl', 'toman', 'tombo', 'tombs', 'tomen', 'tomes', 'tomia', 'tomin', 'tomme', 'tommy', 'tomos', 'tomoz', 'tonal', 'tondi', 'tondo', 'toned', 'toner', 'tones', 'toney', 'tonga', 'tongs', 'tonic', 'tonka', 'tonks', 'tonne', 'tonus', 'tools', 'tooms', 'toons', 'tooth', 'toots', 'topaz', 'toped', 'topee', 'topek', 'toper', 'topes', 'tophe', 'tophi', 'tophs', 'topic', 'topis', 'topoi', 'topos', 'toppy', 'toque', 'torah', 'toran', 'toras', 'torch', 'torcs', 'tores', 'toric', 'torii', 'toros', 'torot', 'torrs', 'torse', 'torsi', 'torsk', 'torso', 'torta', 'torte', 'torts', 'torus', 'tosas', 'tosed', 'toses', 'toshy', 'tossy', 'tosyl', 'total', 'toted', 'totem', 'toter', 'totes', 'totty', 'touch', 'tough', 'touks', 'touns', 'tours', 'touse', 'tousy', 'touts', 'touze', 'touzy', 'towai', 'towed', 'towel', 'tower', 'towie', 'towno', 'towns', 'towny', 'towse', 'towsy', 'towts', 'towze', 'towzy', 'toxic', 'toxin', 'toyed', 'toyer', 'toyon', 'toyos', 'tozed', 'tozes', 'tozie', 'trabs', 'trace', 'track', 'tract', 'trade', 'trads', 'trady', 'traga', 'tragi', 'trags', 'tragu', 'traik', 'trail', 'train', 'trait', 'tramp', 'trams', 'trank', 'tranq', 'trans', 'trant', 'trape', 'trapo', 'traps', 'trapt', 'trash', 'trass', 'trats', 'tratt', 'trave', 'trawl', 'trayf', 'trays', 'tread', 'treat', 'treck', 'treed', 'treen', 'trees', 'trefa', 'treif', 'treks', 'trema', 'trems', 'trend', 'tress', 'trest', 'trets', 'trews', 'treyf', 'treys', 'triac', 'triad', 'trial', 'tribe', 'trice', 'trick', 'tride', 'tried', 'trier', 'tries', 'trifa', 'triff', 'trigo', 'trigs', 'trike', 'trild', 'trill', 'trims', 'trine', 'trins', 'triol', 'trior', 'trios', 'tripe', 'trips', 'tripy', 'trist', 'trite', 'troad', 'troak', 'troat', 'trock', 'trode', 'trods', 'trogs', 'trois', 'troke', 'troll', 'tromp', 'trona', 'tronc', 'trone', 'tronk', 'trons', 'troop', 'trooz', 'trope', 'tropo', 'troth', 'trots', 'trout', 'trove', 'trows', 'troys', 'truce', 'truck', 'trued', 'truer', 'trues', 'trugo', 'trugs', 'trull', 'truly', 'trump', 'trunk', 'truss', 'trust', 'truth', 'tryer', 'tryke', 'tryma', 'tryps', 'tryst', 'tsade', 'tsadi', 'tsars', 'tsked', 'tsuba', 'tsubo', 'tuans', 'tuart', 'tuath', 'tubae', 'tubal', 'tubar', 'tubas', 'tubby', 'tubed', 'tuber', 'tubes', 'tucks', 'tufas', 'tuffe', 'tuffs', 'tufts', 'tufty', 'tugra', 'tuile', 'tuina', 'tuism', 'tuktu', 'tules', 'tulip', 'tulle', 'tulpa', 'tulps', 'tulsi', 'tumid', 'tummy', 'tumor', 'tumps', 'tumpy', 'tunas', 'tunds', 'tuned', 'tuner', 'tunes', 'tungs', 'tunic', 'tunny', 'tupek', 'tupik', 'tuple', 'tuque', 'turbo', 'turds', 'turfs', 'turfy', 'turks', 'turme', 'turms', 'turns', 'turnt', 'turon', 'turps', 'turrs', 'tushy', 'tusks', 'tusky', 'tutee', 'tutes', 'tutor', 'tutti', 'tutty', 'tutus', 'tuxes', 'tuyer', 'twaes', 'twain', 'twals', 'twang', 'twank', 'twats', 'tways', 'tweak', 'tweed', 'tweel', 'tween', 'tweep', 'tweer', 'tweet', 'twerk', 'twerp', 'twice', 'twier', 'twigs', 'twill', 'twilt', 'twine', 'twink', 'twins', 'twiny', 'twire', 'twirk', 'twirl', 'twirp', 'twist', 'twite', 'twits', 'twixt', 'twocs', 'twoer', 'twonk', 'twyer', 'tyees', 'tyers', 'tying', 'tyiyn', 'tykes', 'tyler', 'tymps', 'tynde', 'tyned', 'tynes', 'typal', 'typed', 'types', 'typey', 'typic', 'typos', 'typps', 'typto', 'tyran', 'tyred', 'tyres', 'tyros', 'tythe', 'tzars', 'ubacs', 'ubity', 'udals', 'udder', 'udons', 'udyog', 'ugali', 'ugged', 'uhlan', 'uhuru', 'ukase', 'ulama', 'ulans', 'ulcer', 'ulema', 'ulmin', 'ulmos', 'ulnad', 'ulnae', 'ulnar', 'ulnas', 'ulpan', 'ultra', 'ulvas', 'ulyie', 'ulzie', 'umami', 'umbel', 'umber', 'umble', 'umbos', 'umbra', 'umbre', 'umiac', 'umiak', 'umiaq', 'ummah', 'ummas', 'ummed', 'umped', 'umphs', 'umpie', 'umpty', 'umrah', 'umras', 'unagi', 'unais', 'unapt', 'unarm', 'unary', 'unaus', 'unbag', 'unban', 'unbar', 'unbed', 'unbid', 'unbox', 'uncap', 'unces', 'uncia', 'uncle', 'uncos', 'uncoy', 'uncus', 'uncut', 'undam', 'undee', 'under', 'undid', 'undos', 'undue', 'undug', 'uneth', 'unfed', 'unfit', 'unfix', 'ungag', 'unget', 'ungod', 'ungot', 'ungum', 'unhat', 'unhip', 'unica', 'unify', 'union', 'unios', 'unite', 'units', 'unity', 'unjam', 'unked', 'unket', 'unkey', 'unkid', 'unkut', 'unlap', 'unlaw', 'unlay', 'unled', 'unleg', 'unlet', 'unlid', 'unlit', 'unmad', 'unman', 'unmet', 'unmew', 'unmix', 'unode', 'unold', 'unown', 'unpay', 'unpeg', 'unpen', 'unpin', 'unply', 'unpot', 'unput', 'unred', 'unrid', 'unrig', 'unrip', 'unsaw', 'unsay', 'unsee', 'unset', 'unsew', 'unsex', 'unsod', 'unsub', 'untag', 'untax', 'untie', 'until', 'untin', 'unwed', 'unwet', 'unwit', 'unwon', 'unzip', 'upbow', 'upbye', 'updos', 'updry', 'upend', 'upful', 'upjet', 'uplay', 'upled', 'uplit', 'upped', 'upper', 'upran', 'uprun', 'upsee', 'upset', 'upsey', 'uptak', 'upter', 'uptie', 'uraei', 'urali', 'uraos', 'urare', 'urari', 'urase', 'urate', 'urban', 'urbex', 'urbia', 'urdee', 'ureal', 'ureas', 'uredo', 'ureic', 'ureid', 'urena', 'urent', 'urged', 'urger', 'urges', 'urial', 'urine', 'urite', 'urman', 'urnal', 'urned', 'urped', 'ursae', 'ursid', 'urson', 'urubu', 'urupa', 'urvas', 'usage', 'usens', 'users', 'useta', 'usher', 'using', 'usnea', 'usnic', 'usque', 'ustad', 'uster', 'usual', 'usure', 'usurp', 'usury', 'uteri', 'utero', 'utile', 'utter', 'uveal', 'uveas', 'uvula', 'vacas', 'vacay', 'vacua', 'vacui', 'vacuo', 'vadas', 'vaded', 'vades', 'vadge', 'vagal', 'vague', 'vagus', 'vaids', 'vails', 'vaire', 'vairs', 'vairy', 'vajra', 'vakas', 'vakil', 'vales', 'valet', 'valid', 'valis', 'valli', 'valor', 'valse', 'value', 'valve', 'vamps', 'vampy', 'vanda', 'vaned', 'vanes', 'vanga', 'vangs', 'vants', 'vaped', 'vaper', 'vapes', 'vapid', 'vapor', 'varan', 'varas', 'varda', 'vardo', 'vardy', 'varec', 'vares', 'varia', 'varix', 'varna', 'varus', 'varve', 'vasal', 'vases', 'vasts', 'vasty', 'vatas', 'vatha', 'vatic', 'vatje', 'vatos', 'vatus', 'vauch', 'vault', 'vaunt', 'vaute', 'vauts', 'vawte', 'vaxes', 'veale', 'veals', 'vealy', 'veena', 'veeps', 'veers', 'veery', 'vegan', 'vegas', 'veges', 'veggo', 'vegie', 'vegos', 'vehme', 'veils', 'veily', 'veins', 'veiny', 'velar', 'velds', 'veldt', 'veles', 'vells', 'velum', 'venae', 'venal', 'venas', 'vends', 'vendu', 'veney', 'venge', 'venin', 'venom', 'venti', 'vents', 'venue', 'venus', 'verba', 'verbs', 'verde', 'verge', 'verra', 'verre', 'verry', 'versa', 'verse', 'verso', 'verst', 'verte', 'verts', 'vertu', 'verve', 'vespa', 'vesta', 'vests', 'vetch', 'veuve', 'veves', 'vexed', 'vexer', 'vexes', 'vexil', 'vezir', 'vials', 'viand', 'vibed', 'vibes', 'vibex', 'vibey', 'vicar', 'viced', 'vices', 'vichy', 'vicus', 'video', 'viers', 'vieux', 'views', 'viewy', 'vifda', 'viffs', 'vigas', 'vigia', 'vigil', 'vigor', 'vilde', 'viler', 'villa', 'ville', 'villi', 'vills', 'vimen', 'vinal', 'vinas', 'vinca', 'vined', 'viner', 'vines', 'vinew', 'vinho', 'vinic', 'vinny', 'vinos', 'vints', 'vinyl', 'viola', 'viold', 'viols', 'viper', 'viral', 'vired', 'vireo', 'vires', 'virga', 'virge', 'virgo', 'virid', 'virls', 'virtu', 'virus', 'visas', 'vised', 'vises', 'visie', 'visit', 'visna', 'visne', 'vison', 'visor', 'vista', 'visto', 'vitae', 'vital', 'vitas', 'vitex', 'vitro', 'vitta', 'vivas', 'vivat', 'vivda', 'viver', 'vives', 'vivid', 'vivos', 'vivre', 'vixen', 'vizir', 'vizor', 'vlast', 'vleis', 'vlies', 'vlogs', 'voars', 'vobla', 'vocab', 'vocal', 'voces', 'voddy', 'vodka', 'vodou', 'vodun', 'voema', 'vogie', 'vogue', 'voice', 'voici', 'voids', 'voila', 'voile', 'voips', 'volae', 'volar', 'voled', 'voles', 'volet', 'volke', 'volks', 'volta', 'volte', 'volti', 'volts', 'volva', 'volve', 'vomer', 'vomit', 'voted', 'voter', 'votes', 'vouch', 'vouge', 'voulu', 'vowed', 'vowel', 'vower', 'voxel', 'voxes', 'vozhd', 'vraic', 'vrils', 'vroom', 'vrous', 'vrouw', 'vrows', 'vuggs', 'vuggy', 'vughs', 'vughy', 'vulgo', 'vulns', 'vulva', 'vutty', 'vygie', 'vying', 'waacs', 'wacke', 'wacko', 'wacks', 'wacky', 'wadas', 'wadds', 'waddy', 'waded', 'wader', 'wades', 'wadge', 'wadis', 'wadts', 'wafer', 'waffs', 'wafts', 'waged', 'wager', 'wages', 'wagga', 'wagon', 'wagyu', 'wahay', 'wahey', 'wahoo', 'waide', 'waifs', 'waift', 'wails', 'wains', 'wairs', 'waist', 'waite', 'waits', 'waive', 'wakas', 'waked', 'waken', 'waker', 'wakes', 'wakfs', 'waldo', 'walds', 'waled', 'waler', 'wales', 'walie', 'walis', 'walks', 'walla', 'walls', 'wally', 'walty', 'waltz', 'wamed', 'wames', 'wamus', 'wands', 'waned', 'wanes', 'waney', 'wangs', 'wanks', 'wanky', 'wanle', 'wanly', 'wanna', 'wanta', 'wants', 'wanty', 'wanze', 'waqfs', 'warbs', 'warby', 'wards', 'wared', 'wares', 'warez', 'warks', 'warms', 'warns', 'warps', 'warre', 'warst', 'warts', 'warty', 'wases', 'washi', 'washy', 'wasms', 'wasps', 'waspy', 'waste', 'wasts', 'watap', 'watch', 'water', 'watts', 'wauff', 'waugh', 'wauks', 'waulk', 'wauls', 'waurs', 'waved', 'waver', 'waves', 'wavey', 'wawas', 'wawes', 'wawls', 'waxed', 'waxen', 'waxer', 'waxes', 'wayed', 'wazir', 'wazoo', 'weald', 'weals', 'weamb', 'weans', 'wears', 'weary', 'weave', 'webby', 'weber', 'wecht', 'wedel', 'wedge', 'wedgy', 'weeds', 'weedy', 'weeis', 'weeke', 'weeks', 'weels', 'weems', 'weens', 'weeny', 'weeps', 'weepy', 'weest', 'weete', 'weets', 'wefte', 'wefts', 'weids', 'weigh', 'weils', 'weird', 'weirs', 'weise', 'weize', 'wekas', 'welch', 'welds', 'welke', 'welks', 'welkt', 'wells', 'welly', 'welsh', 'welts', 'wembs', 'wench', 'wends', 'wenge', 'wenny', 'wents', 'werfs', 'weros', 'wersh', 'wests', 'wetas', 'wetly', 'wexed', 'wexes', 'whack', 'whale', 'whamo', 'whams', 'whang', 'whaps', 'whare', 'wharf', 'whata', 'whats', 'whaup', 'whaur', 'wheal', 'whear', 'wheat', 'wheek', 'wheel', 'wheen', 'wheep', 'wheft', 'whelk', 'whelm', 'whelp', 'whens', 'where', 'whets', 'whews', 'wheys', 'which', 'whids', 'whies', 'whiff', 'whift', 'whigs', 'while', 'whilk', 'whims', 'whine', 'whins', 'whiny', 'whios', 'whips', 'whipt', 'whirl', 'whirr', 'whirs', 'whish', 'whisk', 'whiss', 'whist', 'white', 'whits', 'whity', 'whizz', 'whole', 'whomp', 'whoof', 'whoop', 'whoot', 'whops', 'whore', 'whorl', 'whort', 'whose', 'whoso', 'whows', 'whump', 'whups', 'whyda', 'wicca', 'wicks', 'wicky', 'widdy', 'widen', 'wider', 'wides', 'widow', 'width', 'wield', 'wiels', 'wifed', 'wifes', 'wifey', 'wifie', 'wifts', 'wifty', 'wigan', 'wigga', 'wiggy', 'wight', 'wikis', 'wilco', 'wilds', 'wiled', 'wiles', 'wilga', 'wilis', 'wilja', 'wills', 'willy', 'wilts', 'wimps', 'wimpy', 'wince', 'winch', 'winds', 'windy', 'wined', 'wines', 'winey', 'winge', 'wings', 'wingy', 'winks', 'winky', 'winna', 'winns', 'winos', 'winze', 'wiped', 'wiper', 'wipes', 'wired', 'wirer', 'wires', 'wirra', 'wirri', 'wised', 'wiser', 'wises', 'wisha', 'wisht', 'wisps', 'wispy', 'wists', 'witan', 'witch', 'wited', 'wites', 'withe', 'withs', 'withy', 'witty', 'wived', 'wiver', 'wives', 'wizen', 'wizes', 'wizzo', 'woads', 'woady', 'woald', 'wocks', 'wodge', 'wodgy', 'woful', 'wojus', 'woken', 'woker', 'wokka', 'wolds', 'wolfs', 'wolly', 'wolve', 'woman', 'womas', 'wombs', 'womby', 'women', 'womyn', 'wonga', 'wongi', 'wonks', 'wonky', 'wonts', 'woods', 'woody', 'wooed', 'wooer', 'woofs', 'woofy', 'woold', 'wools', 'wooly', 'woons', 'woops', 'woopy', 'woose', 'woosh', 'wootz', 'woozy', 'words', 'wordy', 'works', 'worky', 'world', 'worms', 'wormy', 'worry', 'worse', 'worst', 'worth', 'worts', 'would', 'wound', 'woven', 'wowed', 'wowee', 'wowse', 'woxen', 'wrack', 'wrang', 'wraps', 'wrapt', 'wrast', 'wrate', 'wrath', 'wrawl', 'wreak', 'wreck', 'wrens', 'wrest', 'wrick', 'wried', 'wrier', 'wries', 'wring', 'wrist', 'write', 'writs', 'wroke', 'wrong', 'wroot', 'wrote', 'wroth', 'wrung', 'wryer', 'wryly', 'wuddy', 'wudus', 'wuffs', 'wulls', 'wunga', 'wurst', 'wuses', 'wushu', 'wussy', 'wuxia', 'wyled', 'wyles', 'wynds', 'wynns', 'wyted', 'wytes', 'wythe', 'xebec', 'xenia', 'xenic', 'xenon', 'xeric', 'xerox', 'xerus', 'xoana', 'xolos', 'xrays', 'xviii', 'xylan', 'xylem', 'xylic', 'xylol', 'xylyl', 'xysti', 'xysts', 'yaars', 'yaass', 'yabas', 'yabba', 'yabby', 'yacca', 'yacht', 'yacka', 'yacks', 'yadda', 'yaffs', 'yager', 'yages', 'yagis', 'yagna', 'yahoo', 'yaird', 'yajna', 'yakka', 'yakow', 'yales', 'yamen', 'yampa', 'yampy', 'yamun', 'yandy', 'yangs', 'yanks', 'yapok', 'yapon', 'yapps', 'yappy', 'yarak', 'yarco', 'yards', 'yarer', 'yarfa', 'yarks', 'yarns', 'yarra', 'yarrs', 'yarta', 'yarto', 'yates', 'yatra', 'yauds', 'yauld', 'yaups', 'yawed', 'yawey', 'yawls', 'yawns', 'yawny', 'yawps', 'yayas', 'ybore', 'yclad', 'ycled', 'ycond', 'ydrad', 'ydred', 'yeads', 'yeahs', 'yealm', 'yeans', 'yeard', 'yearn', 'years', 'yeast', 'yecch', 'yechs', 'yechy', 'yedes', 'yeeds', 'yeeek', 'yeesh', 'yeggs', 'yelks', 'yells', 'yelms', 'yelps', 'yelts', 'yenta', 'yente', 'yerba', 'yerds', 'yerks', 'yeses', 'yesks', 'yests', 'yesty', 'yetis', 'yetts', 'yeuch', 'yeuks', 'yeuky', 'yeven', 'yeves', 'yewen', 'yexed', 'yexes', 'yfere', 'yield', 'yiked', 'yikes', 'yills', 'yince', 'yipes', 'yippy', 'yirds', 'yirks', 'yirrs', 'yirth', 'yites', 'yitie', 'ylems', 'ylide', 'ylids', 'ylike', 'ylkes', 'ymolt', 'ympes', 'yobbo', 'yobby', 'yocks', 'yodel', 'yodhs', 'yodle', 'yogas', 'yogee', 'yoghs', 'yogic', 'yogin', 'yogis', 'yohah', 'yohay', 'yoick', 'yojan', 'yokan', 'yoked', 'yokeg', 'yokel', 'yoker', 'yokes', 'yokul', 'yolks', 'yolky', 'yolps', 'yomim', 'yomps', 'yonic', 'yonis', 'yonks', 'yonny', 'yoofs', 'yoops', 'yopos', 'yoppo', 'yores', 'yorga', 'yorks', 'yorps', 'youks', 'young', 'yourn', 'yours', 'yourt', 'youse', 'youth', 'yowed', 'yowes', 'yowie', 'yowls', 'yowsa', 'yowza', 'yoyos', 'yrapt', 'yrent', 'yrivd', 'yrneh', 'ysame', 'ytost', 'yuans', 'yucas', 'yucca', 'yucch', 'yucko', 'yucks', 'yucky', 'yufts', 'yugas', 'yuked', 'yukes', 'yukky', 'yukos', 'yulan', 'yules', 'yummo', 'yummy', 'yumps', 'yupon', 'yuppy', 'yurta', 'yurts', 'yuzus', 'zabra', 'zacks', 'zaida', 'zaide', 'zaidy', 'zaire', 'zakat', 'zamac', 'zamak', 'zaman', 'zambo', 'zamia', 'zamis', 'zanja', 'zante', 'zanza', 'zanze', 'zappy', 'zarda', 'zarfs', 'zaris', 'zatis', 'zawns', 'zaxes', 'zayde', 'zayin', 'zazen', 'zeals', 'zebec', 'zebra', 'zebub', 'zebus', 'zedas', 'zeera', 'zeins', 'zendo', 'zerda', 'zerks', 'zeros', 'zests', 'zesty', 'zetas', 'zexes', 'zezes', 'zhomo', 'zhush', 'zhuzh', 'zibet', 'ziffs', 'zigan', 'zikrs', 'zilas', 'zilch', 'zilla', 'zills', 'zimbi', 'zimbs', 'zinco', 'zincs', 'zincy', 'zineb', 'zines', 'zings', 'zingy', 'zinke', 'zinky', 'zinos', 'zippo', 'zippy', 'ziram', 'zitis', 'zitty', 'zizel', 'zizit', 'zlote', 'zloty', 'zoaea', 'zobos', 'zobus', 'zocco', 'zoeae', 'zoeal', 'zoeas', 'zoism', 'zoist', 'zokor', 'zolle', 'zombi', 'zonae', 'zonal', 'zonda', 'zoned', 'zoner', 'zones', 'zonks', 'zooea', 'zooey', 'zooid', 'zooks', 'zooms', 'zoomy', 'zoons', 'zooty', 'zoppa', 'zoppo', 'zoril', 'zoris', 'zorro', 'zorse', 'zouks', 'zowee', 'zowie', 'zulus', 'zupan', 'zupas', 'zuppa', 'zurfs', 'zuzim', 'zygal', 'zygon', 'zymes', 'zymic'
]

print(f'{len(guess_words_long) = }')


len(guess_words_long) = 14855


In [2]:
### Wordle word list

secret_candidates = [
    'aback', 'abase', 'abate', 'abbey', 'abbot', 'abhor', 'abide', 'abled', 'abode', 'abort', 'about', 'above', 'abuse', 'abyss', 'acorn', 'acrid', 'actor', 'acute', 'adage', 'adapt', 'adept', 'admin', 'admit', 'adobe', 'adopt', 'adore', 'adorn', 'adult', 'affix', 'afire', 'afoot', 'afoul', 'after', 'again', 'agape', 'agate', 'agent', 'agile', 'aging', 'aglow', 'agony', 'agora', 'agree', 'ahead', 'aider', 'aisle', 'alarm', 'album', 'alert', 'algae', 'alibi', 'alien', 'align', 'alike', 'alive', 'allay', 'alley', 'allot', 'allow', 'alloy', 'aloft', 'alone', 'along', 'aloof', 'aloud', 'alpha', 'altar', 'alter', 'amass', 'amaze', 'amber', 'amble', 'amend', 'amiss', 'amity', 'among', 'ample', 'amply', 'amuse', 'angel', 'anger', 'angle', 'angry', 'angst', 'anime', 'ankle', 'annex', 'annoy', 'annul', 'anode', 'antic', 'anvil', 'aorta', 'apart', 'aphid', 'aping', 'apnea', 'apple', 'apply', 'apron', 'aptly', 'arbor', 'ardor', 'arena', 'argue', 'arise', 'armor', 'aroma', 'arose', 'array', 'arrow', 'arson', 'artsy', 'ascot', 'ashen', 'aside', 'askew', 'assay', 'asset', 'atoll', 'atone', 'attic', 'audio', 'audit', 'augur', 'aunty', 'avail', 'avert', 'avian', 'avoid', 'await', 'awake', 'award', 'aware', 'awash', 'awful', 'awoke', 'axial', 'axiom', 'axion', 'azure', 'bacon', 'badge', 'badly', 'bagel', 'baggy', 'baker', 'baler', 'balmy', 'banal', 'banjo', 'barge', 'baron', 'basal', 'basic', 'basil', 'basin', 'basis', 'baste', 'batch', 'bathe', 'baton', 'batty', 'bawdy', 'bayou', 'beach', 'beady', 'beard', 'beast', 'beech', 'beefy', 'befit', 'began', 'begat', 'beget', 'begin', 'begun', 'being', 'belch', 'belie', 'belle', 'belly', 'below', 'bench', 'beret', 'berry', 'berth', 'beset', 'betel', 'bevel', 'bezel', 'bible', 'bicep', 'biddy', 'bigot', 'bilge', 'billy', 'binge', 'bingo', 'biome', 'birch', 'birth', 'bison', 'bitty', 'black', 'blade', 'blame', 'bland', 'blank', 'blare', 'blast', 'blaze', 'bleak', 'bleat', 'bleed', 'bleep', 'blend', 'bless', 'blimp', 'blind', 'blink', 'bliss', 'blitz', 'bloat', 'block', 'bloke', 'blond', 'blood', 'bloom', 'blown', 'bluer', 'bluff', 'blunt', 'blurb', 'blurt', 'blush', 'board', 'boast', 'bobby', 'boney', 'bongo', 'bonus', 'booby', 'boost', 'booth', 'booty', 'booze', 'boozy', 'borax', 'borne', 'bosom', 'bossy', 'botch', 'bough', 'boule', 'bound', 'bowel', 'boxer', 'brace', 'braid', 'brain', 'brake', 'brand', 'brash', 'brass', 'brave', 'bravo', 'brawl', 'brawn', 'bread', 'break', 'breed', 'briar', 'bribe', 'brick', 'bride', 'brief', 'brine', 'bring', 'brink', 'briny', 'brisk', 'broad', 'broil', 'broke', 'brood', 'brook', 'broom', 'broth', 'brown', 'brunt', 'brush', 'brute', 'buddy', 'budge', 'buggy', 'bugle', 'build', 'built', 'bulge', 'bulky', 'bully', 'bunch', 'bunny', 'burly', 'burnt', 'burst', 'bused', 'bushy', 'butch', 'butte', 'buxom', 'buyer', 'bylaw', 'cabal', 'cabby', 'cabin', 'cable', 'cacao', 'cache', 'cacti', 'caddy', 'cadet', 'cagey', 'cairn', 'camel', 'cameo', 'canal', 'candy', 'canny', 'canoe', 'canon', 'caper', 'caput', 'carat', 'cargo', 'carol', 'carry', 'carve', 'caste', 'catch', 'cater', 'catty', 'caulk', 'cause', 'cavil', 'cease', 'cedar', 'cello', 'chafe', 'chaff', 'chain', 'chair', 'chalk', 'champ', 'chant', 'chaos', 'chard', 'charm', 'chart', 'chase', 'chasm', 'cheap', 'cheat', 'check', 'cheek', 'cheer', 'chess', 'chest', 'chick', 'chide', 'chief', 'child', 'chili', 'chill', 'chime', 'china', 'chirp', 'chock', 'choir', 'choke', 'chord', 'chore', 'chose', 'chuck', 'chump', 'chunk', 'churn', 'chute', 'cider', 'cigar', 'cinch', 'circa', 'civic', 'civil', 'clack', 'claim', 'clamp', 'clang', 'clank', 'clash', 'clasp', 'class', 'clean', 'clear', 'cleat', 'cleft', 'clerk', 'click', 'cliff', 'climb', 'cling', 'clink', 'cloak', 'clock', 'clone', 'close', 'cloth', 'cloud', 'clout', 'clove', 'clown', 'cluck', 'clued', 'clump', 'clung', 'coach', 'coast', 'cobra', 'cocoa', 'colon', 'color', 'comet', 'comfy', 'comic', 'comma', 'conch', 'condo', 'conic', 'copse', 'coral', 'corer', 'corny', 'couch', 'cough', 'could', 'count', 'coupe', 'court', 'coven', 'cover', 'covet', 'covey', 'cower', 'coyly', 'crack', 'craft', 'cramp', 'crane', 'crank', 'crash', 'crass', 'crate', 'crave', 'crawl', 'craze', 'crazy', 'creak', 'cream', 'credo', 'creed', 'creek', 'creep', 'creme', 'crepe', 'crept', 'cress', 'crest', 'crick', 'cried', 'crier', 'crime', 'crimp', 'crisp', 'croak', 'crock', 'crone', 'crony', 'crook', 'cross', 'croup', 'crowd', 'crown', 'crude', 'cruel', 'crumb', 'crump', 'crush', 'crust', 'crypt', 'cubic', 'cumin', 'curio', 'curly', 'curry', 'curse', 'curve', 'curvy', 'cutie', 'cyber', 'cycle', 'cynic', 'daddy', 'daily', 'dairy', 'daisy', 'dally', 'dance', 'dandy', 'datum', 'daunt', 'dealt', 'death', 'debar', 'debit', 'debug', 'debut', 'decal', 'decay', 'decor', 'decoy', 'decry', 'defer', 'deign', 'deity', 'delay', 'delta', 'delve', 'demon', 'demur', 'denim', 'dense', 'depot', 'depth', 'derby', 'deter', 'detox', 'deuce', 'devil', 'diary', 'dicey', 'digit', 'dilly', 'dimly', 'diner', 'dingo', 'dingy', 'diode', 'dirge', 'dirty', 'disco', 'ditch', 'ditto', 'ditty', 'diver', 'dizzy', 'dodge', 'dodgy', 'dogma', 'doing', 'dolly', 'donor', 'donut', 'dopey', 'doubt', 'dough', 'dowdy', 'dowel', 'downy', 'dowry', 'dozen', 'draft', 'drain', 'drake', 'drama', 'drank', 'drape', 'drawl', 'drawn', 'dread', 'dream', 'dress', 'dried', 'drier', 'drift', 'drill', 'drink', 'drive', 'droit', 'droll', 'drone', 'drool', 'droop', 'dross', 'drove', 'drown', 'druid', 'drunk', 'dryer', 'dryly', 'duchy', 'dully', 'dummy', 'dumpy', 'dunce', 'dusky', 'dusty', 'dutch', 'duvet', 'dwarf', 'dwell', 'dwelt', 'dying', 'eager', 'eagle', 'early', 'earth', 'easel', 'eaten', 'eater', 'ebony', 'eclat', 'edict', 'edify', 'eerie', 'egret', 'eight', 'eject', 'eking', 'elate', 'elbow', 'elder', 'elect', 'elegy', 'elfin', 'elide', 'elite', 'elope', 'elude', 'email', 'embed', 'ember', 'emcee', 'empty', 'enact', 'endow', 'enema', 'enemy', 'enjoy', 'ennui', 'ensue', 'enter', 'entry', 'envoy', 'epoch', 'epoxy', 'equal', 'equip', 'erase', 'erect', 'erode', 'error', 'erupt', 'essay', 'ester', 'ether', 'ethic', 'ethos', 'etude', 'evade', 'event', 'every', 'evict', 'evoke', 'exact', 'exalt', 'excel', 'exert', 'exile', 'exist', 'expel', 'extol', 'extra', 'exult', 'eying', 'fable', 'facet', 'faint', 'fairy', 'faith', 'false', 'fancy', 'fanny', 'farce', 'fatal', 'fatty', 'fault', 'fauna', 'favor', 'feast', 'fecal', 'feign', 'fella', 'felon', 'femme', 'femur', 'fence', 'feral', 'ferry', 'fetal', 'fetch', 'fetid', 'fetus', 'fever', 'fewer', 'fiber', 'fibre', 'ficus', 'field', 'fiend', 'fiery', 'fifth', 'fifty', 'fight', 'filer', 'filet', 'filly', 'filmy', 'filth', 'final', 'finch', 'finer', 'first', 'fishy', 'fixer', 'fizzy', 'fjord', 'flack', 'flail', 'flair', 'flake', 'flaky', 'flame', 'flank', 'flare', 'flash', 'flask', 'fleck', 'fleet', 'flesh', 'flick', 'flier', 'fling', 'flint', 'flirt', 'float', 'flock', 'flood', 'floor', 'flora', 'floss', 'flour', 'flout', 'flown', 'fluff', 'fluid', 'fluke', 'flume', 'flung', 'flunk', 'flush', 'flute', 'flyer', 'foamy', 'focal', 'focus', 'foggy', 'foist', 'folio', 'folly', 'foray', 'force', 'forge', 'forgo', 'forte', 'forth', 'forty', 'forum', 'found', 'foyer', 'frail', 'frame', 'frank', 'fraud', 'freak', 'freed', 'freer', 'fresh', 'friar', 'fried', 'frill', 'frisk', 'fritz', 'frock', 'frond', 'front', 'frost', 'froth', 'frown', 'froze', 'fruit', 'fudge', 'fugue', 'fully', 'fungi', 'funky', 'funny', 'furor', 'furry', 'fussy', 'fuzzy', 'gaffe', 'gaily', 'gamer', 'gamma', 'gamut', 'gassy', 'gaudy', 'gauge', 'gaunt', 'gauze', 'gavel', 'gawky', 'gayer', 'gayly', 'gazer', 'gecko', 'geeky', 'geese', 'genie', 'genre', 'ghost', 'ghoul', 'giant', 'giddy', 'gipsy', 'girly', 'girth', 'given', 'giver', 'glade', 'gland', 'glare', 'glass', 'glaze', 'gleam', 'glean', 'glide', 'glint', 'gloat', 'globe', 'gloom', 'glory', 'gloss', 'glove', 'glyph', 'gnash', 'gnome', 'godly', 'going', 'golem', 'golly', 'gonad', 'goner', 'goody', 'gooey', 'goofy', 'goose', 'gorge', 'gouge', 'gourd', 'grace', 'grade', 'graft', 'grail', 'grain', 'grand', 'grant', 'grape', 'graph', 'grasp', 'grass', 'grate', 'grave', 'gravy', 'graze', 'great', 'greed', 'green', 'greet', 'grief', 'grill', 'grime', 'grimy', 'grind', 'gripe', 'groan', 'groin', 'groom', 'grope', 'gross', 'group', 'grout', 'grove', 'growl', 'grown', 'gruel', 'gruff', 'grunt', 'guard', 'guava', 'guess', 'guest', 'guide', 'guild', 'guile', 'guilt', 'guise', 'gulch', 'gully', 'gumbo', 'gummy', 'guppy', 'gusto', 'gusty', 'gypsy', 'habit', 'hairy', 'halve', 'handy', 'happy', 'hardy', 'harem', 'harpy', 'harry', 'harsh', 'haste', 'hasty', 'hatch', 'hater', 'haunt', 'haute', 'haven', 'havoc', 'hazel', 'heady', 'heard', 'heart', 'heath', 'heave', 'heavy', 'hedge', 'hefty', 'heist', 'helix', 'hello', 'hence', 'heron', 'hilly', 'hinge', 'hippo', 'hippy', 'hitch', 'hoard', 'hobby', 'hoist', 'holly', 'homer', 'honey', 'honor', 'horde', 'horny', 'horse', 'hotel', 'hotly', 'hound', 'house', 'hovel', 'hover', 'howdy', 'human', 'humid', 'humor', 'humph', 'humus', 'hunch', 'hunky', 'hurry', 'husky', 'hussy', 'hutch', 'hydro', 'hyena', 'hymen', 'hyper', 'icily', 'icing', 'ideal', 'idiom', 'idiot', 'idler', 'idyll', 'igloo', 'iliac', 'image', 'imbue', 'impel', 'imply', 'inane', 'inbox', 'incur', 'index', 'inept', 'inert', 'infer', 'ingot', 'inlay', 'inlet', 'inner', 'input', 'inter', 'intro', 'ionic', 'irate', 'irony', 'islet', 'issue', 'itchy', 'ivory', 'jaunt', 'jazzy', 'jelly', 'jerky', 'jetty', 'jewel', 'jiffy', 'joint', 'joist', 'joker', 'jolly', 'joust', 'judge', 'juice', 'juicy', 'jumbo', 'jumpy', 'junta', 'junto', 'juror', 'kappa', 'karma', 'kayak', 'kebab', 'khaki', 'kinky', 'kiosk', 'kitty', 'knack', 'knave', 'knead', 'kneed', 'kneel', 'knelt', 'knife', 'knock', 'knoll', 'known', 'koala', 'krill', 'label', 'labor', 'laden', 'ladle', 'lager', 'lance', 'lanky', 'lapel', 'lapse', 'large', 'larva', 'lasso', 'latch', 'later', 'lathe', 'latte', 'laugh', 'layer', 'leach', 'leafy', 'leaky', 'leant', 'leapt', 'learn', 'lease', 'leash', 'least', 'leave', 'ledge', 'leech', 'leery', 'lefty', 'legal', 'leggy', 'lemon', 'lemur', 'leper', 'level', 'lever', 'libel', 'liege', 'light', 'liken', 'lilac', 'limbo', 'limit', 'linen', 'liner', 'lingo', 'lipid', 'lithe', 'liver', 'livid', 'llama', 'loamy', 'loath', 'lobby', 'local', 'locus', 'lodge', 'lofty', 'logic', 'login', 'loopy', 'loose', 'lorry', 'loser', 'louse', 'lousy', 'lover', 'lower', 'lowly', 'loyal', 'lucid', 'lucky', 'lumen', 'lumpy', 'lunar', 'lunch', 'lunge', 'lupus', 'lurch', 'lurid', 'lusty', 'lying', 'lymph', 'lynch', 'lyric', 'macaw', 'macho', 'macro', 'madam', 'madly', 'mafia', 'magic', 'magma', 'maize', 'major', 'maker', 'mambo', 'mamma', 'mammy', 'manga', 'mange', 'mango', 'mangy', 'mania', 'manic', 'manly', 'manor', 'maple', 'march', 'marry', 'marsh', 'mason', 'masse', 'match', 'matey', 'mauve', 'maxim', 'maybe', 'mayor', 'mealy', 'meant', 'meaty', 'mecca', 'medal', 'media', 'medic', 'melee', 'melon', 'mercy', 'merge', 'merit', 'merry', 'metal', 'meter', 'metro', 'micro', 'midge', 'midst', 'might', 'milky', 'mimic', 'mince', 'miner', 'minim', 'minor', 'minty', 'minus', 'mirth', 'miser', 'missy', 'mocha', 'modal', 'model', 'modem', 'mogul', 'moist', 'molar', 'moldy', 'money', 'month', 'moody', 'moose', 'moral', 'moron', 'morph', 'mossy', 'motel', 'motif', 'motor', 'motto', 'moult', 'mound', 'mount', 'mourn', 'mouse', 'mouth', 'mover', 'movie', 'mower', 'mucky', 'mucus', 'muddy', 'mulch', 'mummy', 'munch', 'mural', 'murky', 'mushy', 'music', 'musky', 'musty', 'myrrh', 'nadir', 'naive', 'nanny', 'nasal', 'nasty', 'natal', 'naval', 'navel', 'needy', 'neigh', 'nerdy', 'nerve', 'never', 'newer', 'newly', 'nicer', 'niche', 'niece', 'night', 'ninja', 'ninny', 'ninth', 'noble', 'nobly', 'noise', 'noisy', 'nomad', 'noose', 'north', 'nosey', 'notch', 'novel', 'nudge', 'nurse', 'nutty', 'nylon', 'nymph', 'oaken', 'obese', 'occur', 'ocean', 'octal', 'octet', 'odder', 'oddly', 'offal', 'offer', 'often', 'olden', 'older', 'olive', 'ombre', 'omega', 'onion', 'onset', 'opera', 'opine', 'opium', 'optic', 'orbit', 'order', 'organ', 'other', 'otter', 'ought', 'ounce', 'outdo', 'outer', 'outgo', 'ovary', 'ovate', 'overt', 'ovine', 'ovoid', 'owing', 'owner', 'oxide', 'ozone', 'paddy', 'pagan', 'paint', 'paler', 'palsy', 'panel', 'panic', 'pansy', 'papal', 'paper', 'parer', 'parka', 'parry', 'parse', 'party', 'pasta', 'paste', 'pasty', 'patch', 'patio', 'patsy', 'patty', 'pause', 'payee', 'payer', 'peace', 'peach', 'pearl', 'pecan', 'pedal', 'penal', 'pence', 'penne', 'penny', 'perch', 'peril', 'perky', 'pesky', 'pesto', 'petal', 'petty', 'phase', 'phone', 'phony', 'photo', 'piano', 'picky', 'piece', 'piety', 'piggy', 'pilot', 'pinch', 'piney', 'pinky', 'pinto', 'piper', 'pique', 'pitch', 'pithy', 'pivot', 'pixel', 'pixie', 'pizza', 'place', 'plaid', 'plain', 'plait', 'plane', 'plank', 'plant', 'plate', 'plaza', 'plead', 'pleat', 'plied', 'plier', 'pluck', 'plumb', 'plume', 'plump', 'plunk', 'plush', 'poesy', 'point', 'poise', 'poker', 'polar', 'polka', 'polyp', 'pooch', 'poppy', 'porch', 'poser', 'posit', 'posse', 'pouch', 'pound', 'pouty', 'power', 'prank', 'prawn', 'preen', 'press', 'price', 'prick', 'pride', 'pried', 'prime', 'primo', 'print', 'prior', 'prism', 'privy', 'prize', 'probe', 'prone', 'prong', 'proof', 'prose', 'proud', 'prove', 'prowl', 'proxy', 'prude', 'prune', 'psalm', 'pubic', 'pudgy', 'puffy', 'pulpy', 'pulse', 'punch', 'pupal', 'pupil', 'puppy', 'puree', 'purer', 'purge', 'purse', 'pushy', 'putty', 'pygmy', 'quack', 'quail', 'quake', 'qualm', 'quark', 'quart', 'quash', 'quasi', 'queen', 'queer', 'quell', 'query', 'quest', 'queue', 'quick', 'quiet', 'quill', 'quilt', 'quirk', 'quite', 'quota', 'quote', 'quoth', 'rabbi', 'rabid', 'racer', 'radar', 'radii', 'radio', 'rainy', 'raise', 'rajah', 'rally', 'ralph', 'ramen', 'ranch', 'randy', 'range', 'rapid', 'rarer', 'raspy', 'ratio', 'ratty', 'raven', 'rayon', 'razor', 'reach', 'react', 'ready', 'realm', 'rearm', 'rebar', 'rebel', 'rebus', 'rebut', 'recap', 'recur', 'recut', 'reedy', 'refer', 'refit', 'regal', 'rehab', 'reign', 'relax', 'relay', 'relic', 'remit', 'renal', 'renew', 'repay', 'repel', 'reply', 'rerun', 'reset', 'resin', 'retch', 'retro', 'retry', 'reuse', 'revel', 'revue', 'rhino', 'rhyme', 'rider', 'ridge', 'rifle', 'right', 'rigid', 'rigor', 'rinse', 'ripen', 'riper', 'risen', 'riser', 'risky', 'rival', 'river', 'rivet', 'roach', 'roast', 'robin', 'robot', 'rocky', 'rodeo', 'roger', 'rogue', 'roomy', 'roost', 'rotor', 'rouge', 'rough', 'round', 'rouse', 'route', 'rover', 'rowdy', 'rower', 'royal', 'ruddy', 'ruder', 'rugby', 'ruler', 'rumba', 'rumor', 'rupee', 'rural', 'rusty', 'sadly', 'safer', 'saint', 'salad', 'sally', 'salon', 'salsa', 'salty', 'salve', 'salvo', 'sandy', 'saner', 'sappy', 'sassy', 'satin', 'satyr', 'sauce', 'saucy', 'sauna', 'saute', 'savor', 'savoy', 'savvy', 'scald', 'scale', 'scalp', 'scaly', 'scamp', 'scant', 'scare', 'scarf', 'scary', 'scene', 'scent', 'scion', 'scoff', 'scold', 'scone', 'scoop', 'scope', 'score', 'scorn', 'scour', 'scout', 'scowl', 'scram', 'scrap', 'scree', 'screw', 'scrub', 'scrum', 'scuba', 'sedan', 'seedy', 'segue', 'seize', 'semen', 'sense', 'sepia', 'serif', 'serum', 'serve', 'setup', 'seven', 'sever', 'sewer', 'shack', 'shade', 'shady', 'shaft', 'shake', 'shaky', 'shale', 'shall', 'shalt', 'shame', 'shank', 'shape', 'shard', 'share', 'shark', 'sharp', 'shave', 'shawl', 'shear', 'sheen', 'sheep', 'sheer', 'sheet', 'sheik', 'shelf', 'shell', 'shied', 'shift', 'shine', 'shiny', 'shire', 'shirk', 'shirt', 'shoal', 'shock', 'shone', 'shook', 'shoot', 'shore', 'shorn', 'short', 'shout', 'shove', 'shown', 'showy', 'shrew', 'shrub', 'shrug', 'shuck', 'shunt', 'shush', 'shyly', 'siege', 'sieve', 'sight', 'sigma', 'silky', 'silly', 'since', 'sinew', 'singe', 'siren', 'sissy', 'sixth', 'sixty', 'skate', 'skier', 'skiff', 'skill', 'skimp', 'skirt', 'skulk', 'skull', 'skunk', 'slack', 'slain', 'slang', 'slant', 'slash', 'slate', 'slave', 'sleek', 'sleep', 'sleet', 'slept', 'slice', 'slick', 'slide', 'slime', 'slimy', 'sling', 'slink', 'sloop', 'slope', 'slosh', 'sloth', 'slump', 'slung', 'slunk', 'slurp', 'slush', 'slyly', 'smack', 'small', 'smart', 'smash', 'smear', 'smell', 'smelt', 'smile', 'smirk', 'smite', 'smith', 'smock', 'smoke', 'smoky', 'smote', 'snack', 'snail', 'snake', 'snaky', 'snare', 'snarl', 'sneak', 'sneer', 'snide', 'sniff', 'snipe', 'snoop', 'snore', 'snort', 'snout', 'snowy', 'snuck', 'snuff', 'soapy', 'sober', 'soggy', 'solar', 'solid', 'solve', 'sonar', 'sonic', 'sooth', 'sooty', 'sorry', 'sound', 'south', 'sower', 'space', 'spade', 'spank', 'spare', 'spark', 'spasm', 'spawn', 'speak', 'spear', 'speck', 'speed', 'spell', 'spelt', 'spend', 'spent', 'sperm', 'spice', 'spicy', 'spied', 'spiel', 'spike', 'spiky', 'spill', 'spilt', 'spine', 'spiny', 'spire', 'spite', 'splat', 'split', 'spoil', 'spoke', 'spoof', 'spook', 'spool', 'spoon', 'spore', 'sport', 'spout', 'spray', 'spree', 'sprig', 'spunk', 'spurn', 'spurt', 'squad', 'squat', 'squib', 'stack', 'staff', 'stage', 'staid', 'stain', 'stair', 'stake', 'stale', 'stalk', 'stall', 'stamp', 'stand', 'stank', 'stare', 'stark', 'start', 'stash', 'state', 'stave', 'stead', 'steak', 'steal', 'steam', 'steed', 'steel', 'steep', 'steer', 'stein', 'stern', 'stick', 'stiff', 'still', 'stilt', 'sting', 'stink', 'stint', 'stock', 'stoic', 'stoke', 'stole', 'stomp', 'stone', 'stony', 'stood', 'stool', 'stoop', 'store', 'stork', 'storm', 'story', 'stout', 'stove', 'strap', 'straw', 'stray', 'strip', 'strut', 'stuck', 'study', 'stuff', 'stump', 'stung', 'stunk', 'stunt', 'style', 'suave', 'sugar', 'suing', 'suite', 'sulky', 'sully', 'sumac', 'sunny', 'super', 'surer', 'surge', 'surly', 'sushi', 'swami', 'swamp', 'swarm', 'swash', 'swath', 'swear', 'sweat', 'sweep', 'sweet', 'swell', 'swept', 'swift', 'swill', 'swine', 'swing', 'swirl', 'swish', 'swoon', 'swoop', 'sword', 'swore', 'sworn', 'swung', 'synod', 'syrup', 'tabby', 'table', 'taboo', 'tacit', 'tacky', 'taffy', 'taint', 'taken', 'taker', 'tally', 'talon', 'tamer', 'tango', 'tangy', 'taper', 'tapir', 'tardy', 'tarot', 'taste', 'tasty', 'tatty', 'taunt', 'tawny', 'teach', 'teary', 'tease', 'teddy', 'teeth', 'tempo', 'tenet', 'tenor', 'tense', 'tenth', 'tepee', 'tepid', 'terra', 'terse', 'testy', 'thank', 'theft', 'their', 'theme', 'there', 'these', 'theta', 'thick', 'thief', 'thigh', 'thing', 'think', 'third', 'thong', 'thorn', 'those', 'three', 'threw', 'throb', 'throw', 'thrum', 'thumb', 'thump', 'thyme', 'tiara', 'tibia', 'tidal', 'tiger', 'tight', 'tilde', 'timer', 'timid', 'tipsy', 'titan', 'tithe', 'title', 'toast', 'today', 'toddy', 'token', 'tonal', 'tonga', 'tonic', 'tooth', 'topaz', 'topic', 'torch', 'torso', 'torus', 'total', 'totem', 'touch', 'tough', 'towel', 'tower', 'toxic', 'toxin', 'trace', 'track', 'tract', 'trade', 'trail', 'train', 'trait', 'tramp', 'trash', 'trawl', 'tread', 'treat', 'trend', 'triad', 'trial', 'tribe', 'trice', 'trick', 'tried', 'tripe', 'trite', 'troll', 'troop', 'trope', 'trout', 'trove', 'truce', 'truck', 'truer', 'truly', 'trump', 'trunk', 'truss', 'trust', 'truth', 'tryst', 'tubal', 'tuber', 'tulip', 'tulle', 'tumor', 'tunic', 'turbo', 'tutor', 'twang', 'tweak', 'tweed', 'tweet', 'twice', 'twine', 'twirl', 'twist', 'twixt', 'tying', 'udder', 'ulcer', 'ultra', 'umbra', 'uncle', 'uncut', 'under', 'undid', 'undue', 'unfed', 'unfit', 'unify', 'union', 'unite', 'unity', 'unlit', 'unmet', 'unset', 'untie', 'until', 'unwed', 'unzip', 'upper', 'upset', 'urban', 'urine', 'usage', 'usher', 'using', 'usual', 'usurp', 'utile', 'utter', 'vague', 'valet', 'valid', 'valor', 'value', 'valve', 'vapid', 'vapor', 'vault', 'vaunt', 'vegan', 'venom', 'venue', 'verge', 'verse', 'verso', 'verve', 'vicar', 'video', 'vigil', 'vigor', 'villa', 'vinyl', 'viola', 'viper', 'viral', 'virus', 'visit', 'visor', 'vista', 'vital', 'vivid', 'vixen', 'vocal', 'vodka', 'vogue', 'voice', 'voila', 'vomit', 'voter', 'vouch', 'vowel', 'vying', 'wacky', 'wafer', 'wager', 'wagon', 'waist', 'waive', 'waltz', 'warty', 'waste', 'watch', 'water', 'waver', 'waxen', 'weary', 'weave', 'wedge', 'weedy', 'weigh', 'weird', 'welch', 'welsh', 'wench', 'whack', 'whale', 'wharf', 'wheat', 'wheel', 'whelp', 'where', 'which', 'whiff', 'while', 'whine', 'whiny', 'whirl', 'whisk', 'white', 'whole', 'whoop', 'whose', 'widen', 'wider', 'widow', 'width', 'wield', 'wight', 'willy', 'wimpy', 'wince', 'winch', 'windy', 'wiser', 'wispy', 'witch', 'witty', 'woken', 'woman', 'women', 'woody', 'wooer', 'wooly', 'woozy', 'wordy', 'world', 'worry', 'worse', 'worst', 'worth', 'would', 'wound', 'woven', 'wrack', 'wrath', 'wreak', 'wreck', 'wrest', 'wring', 'wrist', 'write', 'wrong', 'wrote', 'wrung', 'wryly', 'yacht', 'yearn', 'yeast', 'yield', 'young', 'youth', 'zebra', 'zesty', 'zonal'
    ]


In [3]:
def alpha_segments(wrds, line_length=12, multiple_lines_per_letter=None):
    """
    Convert a list of words into a single string for printing.
    The string is broken into lines by '\n' characters.
    - line_length is number of words per line.
    - multiple_lines_per_letter: may a line of words be split among multiple lines?
      If not, truncate lines with more than line_length words
    Order the lines of words alphabetically.
    """
    if wrds == [] or line_length <= 0: return ''

    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    # lsts_of_wrds is lists of words, one list for words starting each letter of the alphabet.
    # Also, drop empy lists, i.e., lists corresponding to letters that start no words.
    lsts_of_wrds = [lst for lst in [[w for w in wrds if w[0] == ltr] for ltr in alphabet] if len(lst) > 0]
    lngth_of_longest_word_list = max([len(lst) for lst in lsts_of_wrds])

    # If multiple_lines_per_letter was set by the caller, keep it, otherwise set it
    # to depend on whether lngth_of_longest_word_list > line_length
    multiple_lines_per_letter = (multiple_lines_per_letter if multiple_lines_per_letter is not None
                                 else lngth_of_longest_word_list > line_length)

    if multiple_lines_per_letter:
        # For each list of words:
        # a) Add an extra '\n\t' in front of each list. This puts an empty line between the lines
        #    for each member of the alphabet--making it easier to read. Do this by adding '\n\t'
        #    as characters to the first word in each list.
        # b) Either break each list into multiple lists or truncate each list: the resulting lists
        #    will have length <= line_length.
        # bnded_lsts_of_wrds will be lists of words, each list of length <= line_length
        bnded_lsts_of_wrds = []
        for lst in lsts_of_wrds:
            # lst is originally a list of words starting with the same letter
            lst = ['\n\t' + lst[0]] + lst[1:]
            # Like bnded_lsts_of_wrds, lsts will be a list of lists of words,
            # each no lonter than line_length. Add those lists to bnded_lsts_of_wrds
            lsts = [lst[i:i + line_length] for i in range(0, len(lst), line_length)]
            bnded_lsts_of_wrds += lsts
    else:  # multiple_lines_per_letter = False
        # Since we are not splitting up long lists, truncate them and add '...' if too long.
        # Also, add no extra space between lines as we did with the extra '\n\t' additions.
        bnded_lsts_of_wrds = [(lst[:line_length] + ([] if len(lst) <= line_length else ['...']))
                                    for lst in lsts_of_wrds]

    # Join each list of words into a string with the words separated by ', '
    # Add \t in front of each string.
    # Add a comma at the end of each string if the end is not '...'.
    # Use the pack/unpack (*) operator to isolate the last string.
    *list_of_strings, last_string = [('\t' + ', '.join(lst) + (',' if lst[-1] != '...' else ''))
                                            for lst in bnded_lsts_of_wrds]
    # If the last_string ends with a ',', remove it.
    if last_string[-1] == ',':
        last_string = last_string[:-1]

    # Combine all the strings into one, separated by '\n'
    final_string = "\n".join(list_of_strings + [last_string])

    # If multiple_lines_per_letter, final_string will begin with an extraneous '\n\t'. Remove it.
    final_string = final_string[2:] if multiple_lines_per_letter else final_string

    return final_string

print(f'{len(secret_candidates) = }\n')

# print('alpha_segments(secret_candidates[2200:], multiple_lines_per_letter=True):')
# print(alpha_segments(secret_candidates[2200:], multiple_lines_per_letter=True))

# print('\nalpha_segments(secret_candidates[2200:], multiple_lines_per_letter=False):')
# print(alpha_segments(secret_candidates[2200:], multiple_lines_per_letter=False))

# print('\nalpha_segments(secret_candidates[2200:]):')
# print(alpha_segments(secret_candidates[2200:]))

# print('\nalpha_segments(secret_candidates[2295:]):')
# print(alpha_segments(secret_candidates[2295:]))

# print('\nalpha_segments(secret_candidates[2293:]):')
# print(alpha_segments(secret_candidates[2293:]))


len(secret_candidates) = 2315



In [4]:
from collections import Counter
from typing import List, Optional, Tuple

class Filter():
    def __init__(self, word_length):

        self.word_length = word_length

        # self.greens are the known letters; '_' if not known.
        # Initially none of the letters are known.
        self.greens = list('_____')

        # self.grays are letters known not to be in the word. These are lettters that were guessed
        # and returned gray, i.e., not in the word. Not included are lettters that returned gray
        # because they are known greens or known to be unmatched from a yellow.
        self.grays = set()

        # self.disallowed are the letters that are known not to be at specific positions,
        # e.g., letters that have been guessed at a position and have been yellow
        self.disallowed: List[set] = [set() for _ in range(self.word_length)]

        # A Counter of Counters. A count of its still-hidden instances for each hidden letter.
        self.yellow_counters = Counter()

        # # These are set in generate_letter_sets(), which is called in Secret_Candidates_Profile.__init__().
        # # This awkwardness is requored because the profiles are needed to compute them.
        # self.included_leters, self.excluded_letters, self.known_letters, self.uncertain_letters = None*4


    def __str__(self):
        return f'*** {self.greens = }\n*** {self.grays = }\n*** {self.disallowed = }\n*** {self.yellow_counters = }'


    def edit_filter(self, guess, score, verbose=False):
        # The score char should be interpreted as follows.
        # 'g': the letter is at its position in guess.
        # 'y': the letter is not at its position in guess but is elsewhere in the word
        # '#': the letter is not at any position in the word that is neither green nor assigned

        # Since we use the iterator enumerate(zip(self.guess, self.score))
        # multiple times, make it a concrete list to allow reuse.
        enumerated_guess_score = list(enumerate(zip(guess, score)))

        # Update the greens first.
        for index, (guess_ltr, score_color) in enumerated_guess_score:

            # If this is a new green: self.greens[index] == '_'
            if score_color == 'g' and self.greens[index] == '_':
                # Record the green value
                self.greens[index] = guess_ltr

                # Decrement the yellow_counter for guess_ltr if there is one
                if self.yellow_counters[guess_ltr] <= 1:
                    del self.yellow_counters[guess_ltr]
                else: # self.yellow_counters[guess_ltr] > 1:
                    self.yellow_counters[guess_ltr] -= 1
                # If this is an existing green, do nothing

        # Update the 'y's, i.e., update self.disallowed and self.yellow_counters.

        # Update self.disallowed by adding the yellow letters to the disallowed sets for each yellow letter position
        for index, (guess_ltr, score_color) in enumerated_guess_score:
            if score_color == 'y':
                self.disallowed[index].add(guess_ltr)

        # Update self.yellow_counters:
        # First, make a local yellow Counter for the y-letters in the score for this guess.
        # The following will count the number of occurrences of each yellow letter.
        yellow_counters = Counter([guess_ltr for _, (guess_ltr, score_color) in enumerated_guess_score
                                  if score_color == 'y'])

        # Update yellow_counters.
        # The new value is the max value from yellow_counters and self.yellow_counters
        for guess_ltr in yellow_counters:
            self.yellow_counters[guess_ltr] = max(self.yellow_counters[guess_ltr], yellow_counters[guess_ltr])

        # Update self.grays.
        # Add guess_ltr to self.grays if it is marked # and is in neither self.greens nor self.yellow_counters
        # If guess_ltr is in self.greens or self.yellow_counters, add it to self.disallowed[index] but not to self.grays
        for index, (guess_ltr, score_color) in enumerated_guess_score:
           if score_color == '#':
            if guess_ltr in self.greens or guess_ltr in self.yellow_counters:
                self.disallowed[index].add(guess_ltr)
            else:
                self.grays.add(guess_ltr)


    def edit_filter_from_profiles(self, cndt_prfl, verbose=False):
        greens_profile_keys = list(zip(self.greens, [list(prfl.keys()) for prfl in cndt_prfl.position_profiles]))
        new_greens = [(keys[0] if ltr == '_' and len(keys) == 1 else ltr) for ltr, keys in greens_profile_keys]

        # Identify the new green letters
        new_green_ltrs = [new_ltr for new_ltr, old_ltr in zip(new_greens, self.greens) if new_ltr != old_ltr]
        self.greens = new_greens

        # No need to test whether new_green_ltrs is empty.
        # If len(new_green_ltrs) == 0, the following loop will do nothing.
        for ltr in new_green_ltrs:
            self.yellow_counters[ltr] -= 1
            if self.yellow_counters[ltr] <= 0:
                del self.yellow_counters[ltr]
            # else: # self.yellow_counters[guess_ltr] > 1:

            # self.greens = new_greens
            cndt_prfl.included_letters = cndt_prfl.included_letters.union(new_green_ltrs)
            cndt_prfl.uncertain_letters = Counter({key: cndt_prfl.uncertain_letters[key]
                                            for key in (set(cndt_prfl.uncertain_letters.keys()) - set(new_green_ltrs))})
                                            #   if key not in new_green_ltrs})


    # def generate_letter_sets(self, candidates_profile):
    #     # The letters known to be included in the secret word
    #     self.included_letters = set(self.greens).difference("_").union(self.yellow_counters.keys())
    #     # The letters known not to be included in the secret word
    #     self.excluded_letters = set(self.grays)
    #     # The letters known to be included or excluded
    #     known_letters = self.included_letters.union(self.excluded_letters)
    #     # uncertain_letters are leters in the secret word candidate that are, so far,
    #     # not known to be either included or excluded
    #     all_letters = candidates_profile.letter_profiles
    #     self.uncertain_letters = Counter({ltr: all_letters[ltr] for ltr in set(all_letters) - known_letters})

    #     return included_leters, excluded_letters, known_letters, uncertain_letters


    def print_what_we_know_about_the_secret_word(self, cndt_prfl, guess_nbr):
        """
        Prints informstion (mainly from this Filter) about what we know about the secret word.
        Need the parameter guesses_profile for guesses_profile.excluded_letters and guesses_profile.uncertain_letters
        """
        print('\n' + '-'*20 + f' What we {"initially" if guess_nbr == 1 else "now"} know about the secret word ' + '-'*20)
        print(f'Placed letters: {"".join(self.greens)}')
        print(f'Required but unplaced letters: {", ".join(list(sorted(self.yellow_counters.keys())))}')
        print(f'Excluded letters: {", ".join(sorted(cndt_prfl.excluded_letters))}')

        len_uncert = len(cndt_prfl.uncertain_letters)
        str_1 = '' if len_uncert <= 9 else f'(out of {len_uncert})'
        print(f'Uncertain letter{"" if len_uncert == 1 else "s"}' +
              ("" if len_uncert == 0 else f':\n{dict(cndt_prfl.uncertain_letters.most_common()[:9])} {str_1}'))


    def word_passes(self, word):
        '''
        A word fails if:
        - It contains a gray letter
        - It fails to match a known (i.e., green) letter
        - It includes a letter in one of the disallowed sets
        - It is missing a required hidden yellow
        '''
        # This loop makes the first three tests.
        for index, ltr in enumerate(word):
            if (ltr in self.grays or
                # If self.greens[index] not in {'_', ltr}, the secret word's
                # letter at index is known to be something other than ltr.
                self.greens[index] not in {'_', ltr} or
                ltr in self.disallowed[index]
                ): return False

        # The following makes the yellow_counters test.

        # Count the instances of the letters in word
        letter_counter = Counter(word)
        # Count the instances of the letters in self.greens
        green_counters = Counter(self.greens)

        # self.yellow_counters[ltr] is the known number of hidden positions in the secret word where ltr appears.
        # green_counters[ltr] is the number of known positions in the secret word where ltr appears.
        # I.e., the secret word requires at least self.yellow_counters[ltr] + green_counters[ltr] ltr instances.
        # If letter_counter[ltr] is less than self.yellow_counters[ltr] + green_counters[ltr], word fails
        # because it has fewer instances of ltr than the secret word is known to require.
        for ltr in self.yellow_counters:
            if letter_counter[ltr] < self.yellow_counters[ltr] + green_counters[ltr]: return False

        return True



In [5]:
import random

class Secret_Candidates_Profile():
    """
    Given a set of possible Wordle answers, a profile of those words is:
    (a) a list of 5 Counters, one for each letter position in the 5-character Wordle words.
        Each counter has the English letters as keys. The  values are the number of times that letter
        appears at the Counter's position in the set of words.
    (b) a counter for the freqencies of the letters in all the words

    After these Counters are calculated, the words are evaluated against them. The result can be used
    to select a Wordle guess for these words
    """
    def __init__(self, secret_candidates, guess_words, filter):
        """
        Extract scores for the positions and letters from words. (These are the remaining
        possible secret words.)
        self.position_profiles and self.letter_profiles characterize the collection of words.
        a) position_profiles are profiles for each letter position. For each position, it counts
           the number of occurrences of the various letters in that position in all the words.
        b) letter_profiles is a single Counter for the letters in all the words. It counts the
           number of occurrences of the various letters in all the words.

        First build the position_profiles and the letter_profile and then evaluate the words
        against them.
        """

        if secret_candidates == []: return

        self.secret_candidates = secret_candidates

        # Generate the Profiles based on a collection of candidate secret words.
        self.position_profiles, self.letter_profiles = self.gen_words_profiles(secret_candidates)

        # ===============================================================

        self.filter = filter

        self.included_letters, self.excluded_letters, \
        self.known_letters, self.uncertain_letters = self.generate_letter_sets(self.filter)

        self.filter.edit_filter_from_profiles(self, verbose=False)

        self.best_position_guess = self.best_pos_guess(guess_words)
        self.best_unknown_position_guess = self.best_pos_guess(guess_words, discount_knowns=True)


    def best_knockout_guess(self, len_candidates):
        # guess = (''.join([ltr for ltr, _ in self.filter.uncertain_letters.most_common()[:5]])*4)[:5]
        # return guess

        ltr_prfls = self.letter_profiles
        xxx = [ltr for ltr, _ in ltr_prfls.most_common() if ltr_prfls[ltr]*2 <= len_candidates]
        print(f'{len_candidates = } {xxx = }')
        guess = (''.join(xxx[:5])*4)[:5]
        # guess = (''.join([ltr for ltr, _ in self.letter_profiles.most_common() ][:5])*4)[:5]
        return guess


    def best_pos_guess(self, guess_words, discount_knowns=False):
        """
        Find and return the best position-based guess.
        If discount_knowns=True, discount known letters in scoring.
        """
        # If we've narrowed the remaining candidates down to 3 or fewer, don't use discount_knowns
        if len(self.secret_candidates) <= 3 and discount_knowns:
            discount_knowns = False

        # Create Counters and evaluate the candidate guesses against the Profiles.
        guess_pos_scores = Counter()
        guess_letters_scores = Counter()

        # The overall guess score.
        guess_scores = Counter()

        for guess in guess_words:
           guess_scores[guess] = self.eval_guess_against_profiles(guess, discount_knowns)

        # Order the words by their scores
        best_guesses: List[Tuple[str, int]] = guess_scores.most_common()
        best_score = best_guesses[0][1]

        # All words with the best score
        best_scoring_guesses = [guess for (guess, score) in best_guesses if score == best_score]

        # One of the words with the best score
        selected_guess = random.choice(best_scoring_guesses)

        return selected_guess


    def eval_guess_against_profiles(self, guess, discount_knowns) -> int:
        '''
        Given a guess, find (a) the sum of its word-position-vals according to the position_profiles
        and (b) its letters-val according to the letter_profiles
        A word gets credit for its letters in proportion to the frequency with which those
        letters correspond the the overall letter frequency at each position.
        '''
        # positions_score is the score for guess based on its letters at their positions
        # letters_score is the score for guess based on its letters at any position
        positions_score, letters_score = 0, 0

        # Only count the letter score once even if a letter appears multiple times in a guess
        used_ltrs = set()
        # pos_counter is a Counter of frequencies of letters in a specific position.
        # We want to know for each ltr in a guess, how often that ltr appears in its position in
        # all remaining candidate secret words.
        # The zip expression pairs the letters with the Counter for that position
        for (ltr, pos_counter) in zip(guess, self.position_profiles):
            if not discount_knowns or discount_knowns and ltr in self.uncertain_letters:
                positions_score += pos_counter[ltr]
                if ltr not in used_ltrs:
                    letters_score += self.letter_profiles[ltr]
                    used_ltrs.add(ltr)
        # A word's score is its position score + 1/4 * its letter score
        score = positions_score + int(round(letters_score/4))
        return score

    def generate_letter_sets(self, filter):
        # The letters known to be included in the secret word
        included_letters = set(filter.greens).difference("_").union(filter.yellow_counters.keys())
        # The letters known not to be included in the secret word
        excluded_letters = set(filter.grays)
        # The letters known to be included or excluded
        known_letters = included_letters.union(excluded_letters)
        # uncertain_letters are leters in the secret word candidate that are, so far,
        # not known to be either included or excluded
        ltr_prfls = self.letter_profiles
        uncertain_letters = Counter({ltr: ltr_prfls[ltr] for ltr in set(ltr_prfls) - known_letters})

        return included_letters, excluded_letters, known_letters, uncertain_letters


    def gen_words_profiles(self, words) -> Tuple[List[Counter], Counter]:
        '''
        Calculate scores for the positions and letters of the words.
        words are the candidate secret words
        '''
        # position_profiles are profiles for each letter position. For each position, a profile
        # counts the number of occurrences of the various letters in that position in all the words.
        position_profiles = [Counter(), Counter(), Counter(), Counter(), Counter()]

        # letter_profiles counts the number of occurrences of the various letters in all the words.
        letter_profiles = Counter()
        # Build the position_profiles and the letter_profiles
        for word in words:
            # Use zip to pair the word positions with its associated position Counter
            for (ltr, position_profile) in zip(word, position_profiles):
                position_profile[ltr] += 1
                letter_profiles[ltr] += 1
        return position_profiles, letter_profiles


    def print_secret_candidates_profile(self, guess_nbr, secret_candidates):
        init_or_rem = "" if guess_nbr == 1 else "remaining "
        print('\n' + '-'*20 +
              f' What we {"initially " if guess_nbr == 1 else ""}know about' +
              f" the {init_or_rem}secret candidate{'s' if len(secret_candidates) > 1 else ''} " +
              '-'*20)

        init_or_rem = 'initial' if guess_nbr == 1 else 'remaining'
        print(f'{len(secret_candidates)} {init_or_rem} secret candidates')
        if guess_nbr > 1:
            print(alpha_segments(secret_candidates))

        print(f'\n{"Initial position frequencies" if guess_nbr == 1 else "Position frequencies in remaining words"}')
        for (pos, counter) in enumerate(self.position_profiles):
            most_common = dict(counter.most_common()[:3])
            print(f"{pos + 1}. {most_common}")
        str_1 = f"Most frequent letters in {'initial' if guess_nbr == 1 else 'remaining'} words:\n"
        str_2 = '' if len(self.letter_profiles) <= 9 else f'(out of {len(self.letter_profiles)})'
        print(f'\n{str_1}{dict(self.letter_profiles.most_common()[:9])} {str_2}')



In [6]:
from itertools import product

class Wordle():

    def __init__(self, secret_candidates=secret_candidates, guess_words=[],
                 secret=None, nbr_of_games=1, guess_strategies=['pos_based'],
                 game_nbr=None, verbose=False):
        '''
        Play Wordle
        '''
        self.verbose = verbose

        # The fixed universe of allowable secret words
        self.all_secret_words = secret_candidates

        self.guess_words = guess_words
        self.guess_limit = 6

        # The currently allowable secret words
        self.secret_candidates = self.all_secret_words

        # self.secret_candidates_profile is set in select_a_guess().
        self.secret_candidates_profile: Secret_Candidates_Profile

        # The list of possible_guess_strategies is not fully determined
        self.possible_guess_strategies = ['interactive', 'online', 'pos_based', 'unk_pos_based']

        self.guess_strategies = guess_strategies

        # See make_a_guess().  Possible guess strategies are
        self.guess_strategy = ''

        self.discount_knowns = False

        self.guess = '*****'
        self.score = '#####'

        self.word_length = len(self.guess)
        self.secret = secret if secret is not None else \
                      self.get_user_secret_word() if self.guess_strategy == 'interactive' else \
                      random.choice(self.guess_words)


        self.word_length = len(self.guess)

        self.nbr_of_games = nbr_of_games
        self.game_nbr = game_nbr

        self.many_games = 5

        self.filter: Filter


    def __str__(self):
        return  f'Wordle:\n{alpha_segments(self.secret_candidates)}\n{self.filter}'


    def get_user_secret_word(self):

        while True:
            print('Please enter a 5 letter secret word.')
            print('Or enter "help" for the list of possible secret words.')
            print('Or enter "no" to have the system pick a secret word.\n')
            user_secret = input(' secret word > ').lower()

            match user_secret:
                case 'no':
                    user_secret = random.choice(self.secret_candidates)
                    print('A secret word has been selected.')
                    return user_secret
                case 'help':
                    print(alpha_segments(self.guess_words, line_length=19, multiple_lines_per_letter=True))
                    continue

            if len(user_secret) != self.word_length:
                    comparator = "more" if len(user_secret) > self.word_length else "fewer"
                    print(f'"{user_secret}" has {comparator} than 5 characters.')
                    continue

            if user_secret not in self.secret_candidates:
                    print(f'"{user_secret}" isn\'t in the list of possible secret words.')
                    continue

            # Since we found no reason to reject user_secret, use it.
            return user_secret


    @classmethod
    def long_test(cls, secret_candidates=secret_candidates, guess_words=secret_candidates,
                  guess_strategy='pos_based', verbose=False):
        total_guesses = 0
        subtotal_count = subtotal_guesses = 0
        nbr_of_games = len(secret_candidates)
        guess_strategy = guess_strategy
        print(f'\t{nbr_of_games = }\n\t{guess_strategy = }')
        print_counter = 0
        for n in range(1, nbr_of_games + 1):
            subtotal_count += 1
            if n > 0 and n%100 == 0:
                print(f'{n} [{round(total_guesses/n, 2)}, ({round(subtotal_guesses/subtotal_count, 2)})]; ', end='')
                subtotal_count = subtotal_guesses = 0
                print_counter += 1
                if print_counter%5 == 0: print()

            wordle = Wordle(secret_candidates=secret_candidates, guess_words=guess_words,
                            nbr_of_games=nbr_of_games, guess_strategies=[guess_strategy],
                            verbose=verbose)

            game_info = wordle.play_a_game()
            print(f'{game_info = }')
            game_guesses = game_info['game_instancess']['game_guesses']
            subtotal_guesses += game_guesses
            total_guesses += game_guesses
            print(f'\n\nAfter {nbr_of_games} trials: average number of guesses/trial: {round(total_guesses/nbr_of_games, 2)}')


    def make_a_guess(self, guess_nbr, verbose=True) -> bool:
        """
        This function runs once each guess. See self.play_a_game().
        It returns True/False depending on whether the secret word is/isn't found.
        """
        # ======================================

        # Make a new secret_candidates_Profile based on the current candidate secret words.
        self.secret_candidates_profile = Secret_Candidates_Profile(secret_candidates=self.secret_candidates,
                                                                   guess_words=self.guess_words, filter=self.filter)

        if verbose and self.nbr_of_games <= self.many_games:
            self.secret_candidates_profile.print_secret_candidates_profile(guess_nbr, self.secret_candidates)
            self.filter.print_what_we_know_about_the_secret_word(self.secret_candidates_profile, guess_nbr)

        # ======================================
        # Begin a round for a new guess.
        # If guess_nbr == 1 this is the start of a new game.
        if verbose and self.nbr_of_games <= self.many_games:
            prefix = '\n'
            if guess_nbr == 1:
                print(f'\n\n{"="*38} Strategy: {self.guess_strategy} {"="*40}')
                prefix += '\n'

            game_id = '' if self.game_nbr is None else f' Game {self.game_nbr},'
            print(f'{prefix}{"="*30} Strategy: {self.guess_strategy}; {game_id} Guess {guess_nbr} {"="*30}')

        # Select a guess. This displays the guess on the screen.
        self.guess = self.select_a_guess(guess_nbr)

        # Score the guess
        self.score = self.retrieve_score() if self.guess_strategy == 'online' else \
                     self.score_a_guess(self.secret, self.guess)


        # When not playing multiple games and if neither interactive nor online, display the secret word and the score.
        if verbose and self.nbr_of_games <= self.many_games and self.guess_strategy != 'online':
            print_prefix = "" if self.guess_strategy == 'interactive' else f' secret   = {self.secret}\n'
            print(f'{print_prefix}  -> score: {self.score}')

        # We found the secret word. If not playing multiple games, tell the user and exit this game.
        if set(self.score) == {'g'}:
            if verbose and self.nbr_of_games <= self.many_games or self.guess_strategy in ['interactive', 'online']:
                print(f'\n => Found the secret word: {self.guess}\n{"="*100}\n{"="*100}\n')
            return True

        # We didn't find the secret word.
        # Update the filter
        self.filter.edit_filter(self.guess, self.score)

        # Use the updated filter to filter self.secret_candidates
        remaining_words = [word for word in self.secret_candidates if self.filter.word_passes(word)]

        # If the secret word is lost (a bug!), display the state
        if (len(remaining_words) == 0 or self.guess_strategy != 'online' and
            self.secret not in remaining_words):
            if verbose: print(f'\n\n\n{self.filter}\n\n\n')
            if len(remaining_words) == 0:
                raise Exception('Error: No candidate secret words remainings')
            else:
                if verbose: print(f'\n\n###### {self.secret = }\n###### {self.guess  = }\n###### {self.score  = }')
                raise Exception(f'Error: {self.secret} is not in the remaining words')
            return False

        # Update self.secret_candidates to the remaining words
        self.secret_candidates = remaining_words
        self.guess_words = self.secret_candidates if self.secret_candidates == self.guess_words else self.guess_words

        return False


    def play_a_game(self) -> dict:
        """
        Plays a game. Returns game_info, containing the guess_strategy used and the number of guesses required
        to find the secret word.
        May play the same game, i.e., with the same secret word, multiple times with different guess_strategies.
        """
        game_info = {'secret_word': self.secret, 'game_instancess': []}

        for self.guess_strategy in self.guess_strategies:
            this_game = {'guess_strategy': self.guess_strategy, 'guesses': -1}

            # The initial words for this game
            self.secret_candidates = self.all_secret_words

            # An empty Filter; blocks nothing.
            self.filter = Filter(self.word_length)

            # The following loop, loops through guesses. It ends internally when
            # either we find the secret word or reach self.guess_limit guesses.
            for guess_nbr in range(1, self.guess_limit + 1):

                # self.make_a_guess() returns True/Flse if the secret word is/is not found
                if self.make_a_guess(guess_nbr):
                    this_game['guesses'] = guess_nbr
                    break

            # This game is over using the current self.guess_strategy.
            # Try it with the next self.guess_strategy
            game_info['game_instancess'].append(this_game)

        # Tried all self.guess_strategies
        return game_info


    def retrieve_score(self):
        print(f'\n\t\t\tPlease enter a 5-character score for the guess: {self.guess}:')
        print("\t\t\t\t 'g' (green)  - right letter in right position")
        print("\t\t\t\t 'y' (yellow) - right letter in wrong position")
        print("\t\t\t\t '#' (gray)   - letter not in secret word")

        while True:
            score = input('\n\t\t\tScore > ').lower()

            if len(score) != 5:
                print(f'{score} has {len(score)} letters')
            elif not set(score).issubset({'g', 'y', '#'}):
                print(f'{score} has invalid letters')
            else:
                return score


    def score_a_guess(self, secret, guess, verbose=False):
        '''
        Build a list with one char for each guess element.
        g = Letter in Correct position (green);
        y = Letter in Incorrect position (yellow);
        # = Letter not in secret word. (These are the gray results. Can't use 'g' since we use it for green)

        In the following example: secret    = 'hpaay'
                                  guess     = 'aaapz'
                                  -> score  = 'y#gy#'

        The third letter in both the secret and the guess is "a".
            So the third letter in the score must be "g".
            (The matching letters must be determined first.)
        The first letter in the guess, 'a', is associated with the fourth letter in the secret, 'a'.
            So the first letter in the score is "y".
        The second letter in the guess, 'a', has nothing in secret to be associated with.
            So the second letter in the score is "#".
        The other two letter in the score should be self explanatory.
        '''

        # Generate the score list with the letters in the correct position marked 'g'.
        # All non-green elements are marked '#'. These will be changed to 'y' later if appropriate.
        score = [('g' if guess[i]==secret[i] else '#') for i in range(self.word_length)]

        # Generate a list of the secret letters that do not have correct corresponding guess letters.
        secret_avail = [secret[i] for i in range(self.word_length) if score[i] == '#' and self.filter.greens[i] == '_']

        # Print intermediate results
        if verbose:
            greens = score
            print(f'\nScore of correct letters:\n\t{secret = }\n\t{guess  = }\n\t{greens  = }\n\t{secret_avail = }')

        # For each available secret letter find matching guess letters. Ignore guess letters that are paired with
        # their secret letters. Score the first one as 'y' and don't look further
        for ltr in secret_avail:
            for i in range(self.word_length):
                if ltr == guess[i] and score[i] == '#':
                    score[i] = 'y'
                    break

        # Return score as a string
        score_string = ''.join(score)
        if verbose:
            print(f'\n\t{score_string = }\n\n')
        return score_string


    def select_a_guess(self, guess_nbr, discount_knowns=False):

        knockout_guess = user_guess = pos_unk_guess = None

        # Always find the position_based guess
        pos_guess = self.secret_candidates_profile.best_position_guess

        if self.guess_strategy == 'unk_pos_based':
            pos_unk_guess = self.secret_candidates_profile.best_unknown_position_guess

        # # Always consider the knockout_guess
        # print(f'{(self.secret_candidates_profile.uncertain_letters) = }  {len(self.secret_candidates) = }')
        if 3 <= len(self.secret_candidates_profile.uncertain_letters) and 3 <= len(self.secret_candidates) and False:
            knockout_guess = self.secret_candidates_profile.best_knockout_guess(len(self.secret_candidates))

        # If we are using either interactive or online as guess_strategy, suggest a guess and read the user's input
        if self.guess_strategy in ['interactive', 'online']:
            my_choice_guess = knockout_guess if knockout_guess else pos_guess
            print(f'My choice:  {my_choice_guess}')
            print('Enter ok to use the suggested guess')
            user_guess = (input(f'Guess {guess_nbr}   > ') + '_'*5)[:5].lower()

            # Entering 'ok', means the user wants to use the suggested guess, either knockout_guess or pos_guess
            if user_guess == 'ok___': user_guess = my_choice_guess

            print(f'Your guess: {user_guess}')

        the_guess = knockout_guess if knockout_guess else \
                    user_guess     if user_guess     else \
                    pos_unk_guess  if pos_unk_guess  else \
                    pos_guess

        # Print the guess if the user didn't enter it.
        if user_guess is None and self.nbr_of_games <= self.many_games:
            print(f'Guess {guess_nbr}   > {the_guess}')

        return the_guess



In [7]:
# alpha_segments() tests
def test_alpha_segments(verbose=False):
    print(f'1. {alpha_segments([])}\n')
    print(f'2. {alpha_segments(secret_candidates[:150])}\n')
    print(f'3. {alpha_segments(secret_candidates)}\n')


# Candidates_Profiles test
def test_Candidates_Profile(verbose=False):
    test_words = ['arise', 'crane', 'crate', 'irate', 'least', 'plate', 'raise',
                  'sauce', 'sauce', 'saute', 'shine', 'slate', 'trace']

    # Test Words_Profile on both words and test_words
    for wrds in [test_words]:
        candidates_profile = Secret_Candidates_Profile(secret_candidates=secret_candidates,
                                                       guess_words=secret_candidates,
                                                       filter=Filter(5))
        # print(f'\n{candidates_profile.best_guesses = }')
        print(f'{candidates_profile.best_pos_guess = }\n\n')



# This data is used in the following two tests.
secret_guess_exp_list = [('hpaax', 'aaapz', "y#gy#"), ('aabbb', 'ccaaa', "##yy#"),
                         ('godly', 'dolly', "yg#gg"), ('madam', 'mamma', 'ggy#y'),
                         ('madam', 'magma', 'gg#yy'), ('moose', 'onion', 'y##y#')
                         ]

# Scoring test
def test_Scoring(verbose=False):
    """
    Scoring: g (green)  = Letter in correct position;
             y (yellow) = Letter in incorrect position;
             # (gray)   = Letter not in word.

    In the following example:  guess  = 'aaapz'
                               secret = 'hpaay'
                            -> score  = 'y#gy#'

    First check the matching letters.

    The third letter in both the guess and the secret is "a". So the third letter in the score must be "g".

    The first letter in the guess, 'a', is associated with the fourth letter in the secret, 'a'. (This is
    why the matching letters must be determined first.) So the first letter in the score is "y".

    The second letter in the guess, 'a', has nothing in secret to be associated with. So the second letter
    in the score is "#".

    The fourth letter in the guess, 'p', is associated with the second letter in guess. So the fourth letter
    in the score is "y".

    The fifth letter in the guess, 'z', is not in secret. So the fifth letter in the score is "#".
    """

    w = Wordle()
    for secret, guess, exp_score in secret_guess_exp_list:
        score = w.score_a_guess(secret=secret, guess=guess)
        print(f' {secret = } :: {guess = } -> {score = } <{exp_score = }>\t[{score == exp_score = }]')


# edit_filter() Test
def test_edit_filter(verbose=False):
    w = Wordle()
    for secret, guess, exp_score in secret_guess_exp_list:
        w.__init__(secret=secret)
        w.score = w.score_a_guess(secret=w.secret, guess=w.guess)
        print(f'\n    {w.secret = }' +
              f'\n    {w.guess  = }' +
              f'\n -> {w.score  = } <{exp_score = }>\t[{w.score == exp_score = }]')
        w.filter.edit_filter(guess, w.score, verbose=False)
        print(w.filter)


# word_passes() test
def test_word_passes(verbose=False):
    wordle = Wordle(secret_candidates)
    wordle.filter.yellow_counters = Counter({'d': 1, 'c': 2})
    wordle.filter.greens =['a', '_', '_', '_', '_']
    print('\n', wordle.filter, sep='', end='\n\n')
    print(f"1. {wordle.filter.word_passes('abcdc') = } -> True")
    print(f"2. {wordle.filter.word_passes('acdce') = } -> True")
    print(f"3. {wordle.filter.word_passes('abcde') = } -> False")
    wordle.filter.disallowed[3].add('g')
    print(f'\n{wordle.filter}\n\n')
    print(f"4. {wordle.filter.word_passes('fcbac') = } -> False")
    print(f"5. {wordle.filter.word_passes('fcbge') = } -> False")
    wordle.filter.greens =['_', '_', '_', '_', '_']
    wordle.filter.yellow_counters = Counter({'c':3, 'd':1})
    print('\n', wordle.filter, sep='', end='\n\n')
    print(f"6. {wordle.filter.word_passes('ccccd') = } -> True")


# Best starting words REALS, BLAHS (??)
# Wordle test
def run_Wordle(verbose=False):
    nbr_of_games = 1
    # The possible guess_strategies are: interactive, online, pos_based, unk_pos_based
    # Could also supply a list of strategies.
    guess_strategies = ['pos_based', 'unk_pos_based']
    print(f'\t{nbr_of_games = }\n\t{guess_strategies = }')
    for n in range(1, nbr_of_games + 1):
        wordle = Wordle(secret_candidates=secret_candidates, guess_words=secret_candidates,
                        nbr_of_games=nbr_of_games, guess_strategies=guess_strategies, game_nbr=n)
        game_info = wordle.play_a_game()
        print(f'Secret word: {game_info["secret_word"]}')
        for instance in game_info['game_instancess']:
            print(f"\tstrategy: {instance['guess_strategy']}: {instance['guesses']} guesses")


tests = [(False, test_alpha_segments, False),     (False, test_edit_filter, False),
         (False, test_Candidates_Profile, False), (False, test_word_passes, False),
         (False, test_Scoring, False),
         (True , run_Wordle, False),              (False , Wordle.long_test, False),
         ]

for (run, test, verbose) in tests:
    if run:
        print(f'\tRunning: {test.__name__}(verbose={verbose})\n')
        test(verbose=verbose)
        print('\n' + ('='*100), end='\n\n')


	Running: run_Wordle(verbose=False)

	nbr_of_games = 1
	guess_strategies = ['pos_based', 'unk_pos_based']

-------------------- What we initially know about the secret candidates --------------------
2315 initial secret candidates

Initial position frequencies
1. {'s': 366, 'c': 198, 'b': 173}
2. {'a': 304, 'o': 279, 'r': 267}
3. {'a': 307, 'i': 266, 'o': 244}
4. {'e': 318, 'n': 182, 's': 171}
5. {'e': 424, 'y': 364, 't': 253}

Most frequent letters in initial words:
{'e': 1233, 'a': 979, 'r': 899, 'o': 754, 't': 729, 'l': 719, 'i': 671, 's': 669, 'n': 575} (out of 26)

-------------------- What we initially know about the secret word --------------------
Placed letters: _____
Required but unplaced letters: 
Excluded letters: 
Uncertain letters:
{'e': 1233, 'a': 979, 'r': 899, 'o': 754, 't': 729, 'l': 719, 'i': 671, 's': 669, 'n': 575} (out of 26)


====================================== Strategy: pos_based ========================================


============================== Strat